# Linking named entities
by Filip Gregora

In [1]:
import pandas as pd
from itertools import combinations
from string import punctuation, ascii_letters
import numpy as np
import math
import random
import os
import xml.etree.ElementTree as elt
from openai import OpenAI
import regex
import requests
import json

In [2]:
data = pd.read_csv("data/NER_entities.csv")
data.head(10)

,label,text
0,symptom,jemný fibrózní proužek
1,procedura,neoadjuvantní CHT
2,medikace,Novalgin
3,symptom,Označena SLU v levé axile.
4,procedura,st.p. totální ME + SNB vlevo
5,medikace,NOVALGIN
6,procedura,Založení TE l.sin
7,procedura,Cytostatika
8,NE symptom,"přiměřené echogenity,"
9,NE symptom,nezvětšena


## Data exploration
At the beginning we want to explore data.

- We can see that somewhere there is big letter at the beginning (but be carefull when whole first word is written in upper case)
- Somewhere at the end is interpuncion
- There are lots of duplicates
- the length of text is variable and the longest has 20 words (this can be problem in the future)


In [3]:
len(data[data.duplicated()])

2588

In [4]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
def clean(string):
    string = string.strip(" " + "".join(punctuation))
    # Remove first upper letter if not all letters are upper
    if len(string) >= 2:
        string = string[0].lower() + string[1:] if string[1].islower() else string
    # Replace multiple whitespaces with one
    return " ".join(string.split())

def clean_table(db):
    db_copy = db.copy()
    db_copy["text"] = db_copy["text"].apply(clean)
    db_copy["text"] = db_copy["text"].drop_duplicates()
    return db_copy.dropna()

former_len = len(data)
data = clean_table(data)
print(former_len, "->", len(data), ":", former_len - len(data))
data.head(10)

6034 -> 2803 : 3231


,label,text
0,symptom,jemný fibrózní proužek
1,procedura,neoadjuvantní CHT
2,medikace,novalgin
3,symptom,označena SLU v levé axile
4,procedura,st.p. totální ME + SNB vlevo
5,medikace,NOVALGIN
6,procedura,založení TE l.sin
7,procedura,cytostatika
8,NE symptom,přiměřené echogenity
9,NE symptom,nezvětšena


In [6]:
def comb_sum(j):
    sum = 0
    for i in range(j, 0, -1):
        sum += math.comb(j,i)
    return sum

for i in range(1, 21):
    print(i, comb_sum(i), sep = ": ", end = " | ")
    
lenght_data = data["text"].apply(lambda x: len(x.split(" ")))
len(lenght_data[lenght_data >= 7])

1: 1 | 2: 3 | 3: 7 | 4: 15 | 5: 31 | 6: 63 | 7: 127 | 8: 255 | 9: 511 | 10: 1023 | 11: 2047 | 12: 4095 | 13: 8191 | 14: 16383 | 15: 32767 | 16: 65535 | 17: 131071 | 18: 262143 | 19: 524287 | 20: 1048575 | 

140

## Linking
Our approach to linking entities is:
- Link words to some medicinal database.
- Let some pretrained language model to choose the best from them.

So I can compare I decided to use two approaches for linking. The first one is to use international mesh and access it via web API of NIH (National Institute of Health).

The second is to use czech mesh. I accessed it from predownloaded file.


### Linking to international MESH through NIH
Mash is international medical databaze: https://uts.nlm.nih.gov/uts/.

I tried search all combinations of words from text in databaze. The longer combinations have higher priority. 

There is one big problem, the complexity grows exponentially with the lenght of the words (in the worst case for lenght of 20 we have to try around 10^6 combinations). My solution for this problem is go from bottom up, start with lenght 1 and continue only with combinations which success.

In [7]:
# Do not search in databaze if it number or it is too short (shorter than 2)
def filter_short(string):
    return len(string) < 2 or string.isdigit()
    
    
def print_linking_stats(data_list):
    empty = len(data_list[data_list.apply(lambda x: len(x) == 0)])
    print(f"Number of empty: {empty} ({empty / len(data_list) * 100} %)")

    number_of_matches = data_list.apply(lambda x: len(x))
    print(f"Mean from number of matches: {number_of_matches.mean()}")
    print(f"Median from number of matches: {number_of_matches.median()}")
    print(f"Maximal of matches: {number_of_matches.max()}")

In [8]:
# From string to list which contains tuples
def from_string_to_list(string):
    result = []
    for j in string.strip("[]()").split("), ("):
        if len(j) == 0:
            continue
        result.append(tuple([s.strip("'\" \\") for s in j.split("', ")]))
                
    return result


#From string to tuple
def from_string_to_tuple(string):
    if string == "N/A":
        return
    string = string.strip("\\\" '")
    stripped_string = string[0].strip("(") + string[1:-1] + string[-1].strip(")")
    
    result = [i.strip("\\\"'") for i in stripped_string.split(", ")]
    return (result[0], result[1], ", ".join(result[2:]))


#From string to dictionary
def from_string_to_dict(string):
    result = {}
    for j in string.strip("{} ").split("], "):
        if j == "":
            continue
        i = list(j.split(": ["))
        assert len(i) == 2
        result[i[0].strip("\"\' \\")] = from_string_to_list(i[1])

    return result


#From string to dictionary which contains tuples
def from_string_to_dict_to_tuple(string):
    result = {}
    for j in regex.split("(\)| None), ('|\")", string.strip("{} ")):
        if j in ["", ')', ' None', "'", '"']:
            continue
        i = list(j.split(": ("))
        if len(i) == 1:
            i[0] = i[0].split(": None")[0].strip(": ")
            result[i[0].strip("\"\' \\")] = None
        else:
            result[i[0].strip("\"\' \\")] = from_string_to_tuple(i[1])

    return result

#From string to dictionary with ints
from_string_to_int_dict = (lambda x: {elem.split("': ")[0].strip("' \"") : int(elem.split("': ")[1])
                                      for elem in x.strip("{}\"' ").split(", '")})

In [9]:
with open("APIkeys/NIH", "r") as f:
    NIH_api = f.read()

def mash_slow_search(string):
    splitted_input = (string.split(" "))
    result = []
    for j in range(len(splitted_input), 0, -1):
        for string in combinations(splitted_input, j): 
            if filter_short(" ".join(string)):
                continue
                
            path = 'https://uts-ws.nlm.nih.gov/rest/search/current'
            query = {
                     'string': " ".join(string),
                     'apiKey':NIH_api,
            }
            res = requests.get(path, params=query)

            if res.status_code <= 200:
                data = json.loads(res.text)
                for j in data["result"]["results"]:
                    result.append((j["ui"], j["name"]))
            else:
                print(res.status_code, res.text)
        
        if len(result) != 0:
            break
                        
    return result
        
    
def search_from_bottom(string, func, output_state = False):
    if (output_state):
        global count
        count += 1
        if count % 100 == 0:
            print(count)
    
    splitted_input = (string.split(" "))
    result = []
    last_result = []
    lenght = len(splitted_input)
    
    for j in range(1, lenght + 1):
        splitted_dict={}
        for elem in splitted_input:
            splitted_dict[j] = False
                    
        for words in combinations(splitted_input, j):
            data = func(" ".join(words))
            if len(data) != 0:
                for j in words:
                    splitted_dict[j] = True
                if filter_short(" ".join(words)):
                    continue
                result.append(data)

        splitted_input = [j for j, i in splitted_dict.items() if i]
        if len(splitted_input) == 0:
            break
        else:
            last_result, result = result, []
        
    temp = []
    for j in last_result:
        temp += list(enumerate(j))
    return [j for (i, j) in sorted(temp)]
    
    
def mash_search(string):
    path = 'https://uts-ws.nlm.nih.gov/rest/search/current'
    query = {
             'string': string,
             'apiKey':NIH_api,
    }
    res = requests.get(path, params=query)

    if res.status_code <= 200:
        data = json.loads(res.text)          
        return [(j["ui"], j["name"]) for j in data["result"]["results"]]
    else:
        print(res.status_code, res.text)
        return []
    

def seach_table(db, func):
    db = db.copy()
    db["search"] = db["text"].apply(func)
    return db 
    

def mash_search_table(db):
    return seach_table(db, lambda x: search_from_bottom(x, mash_search))

In [10]:
if os.path.isfile("saved_search/data_mash.csv"):
    mash_linked_data = pd.read_csv("saved_search/data_mash.csv")
    mash_linked_data.index = mash_linked_data["Unnamed: 0"]
    mash_linked_data.drop(["Unnamed: 0"], axis=1, inplace=True)
    mash_linked_data["search"] = mash_linked_data["search"].apply(from_string_to_list)
else:
    mash_linked_data = mash_search_table(data)
    mash_linked_data.to_csv("saved_search/data_mash.csv")
    
print_linking_stats(mash_linked_data["search"])

Number of empty: 298 (10.631466286122013 %)
Mean from number of matches: 21.931858722797003
Median from number of matches: 25.0
Maximal of matches: 138


#### Not assigned
If we look at the random sample of 10 texts, which are not assigned, then we can see that in five of them there is typographical mistake (*"nejsou zn.plicní hpertenze"* = *"nejsou zn. plicní hypertenze"*, *"kumulce a nehomogenity"* = *"kumulace a nehomogenita"*, *"ceriucal"* = *"cerucal"*, *"paitace"* = *"palpitace"*, *"mamily klidné"* = ?). Others five are correct medical term, but in some non-typical grammatical form.

If we try to improve them we get 50 % improvement.

In [11]:
# # empty_sample = mash_linked_data[mash_linked_data["search"].apply(lambda x: len(x) == 0)].sample(10, random_state=42)

# # Because of my mistake (I had worser clean_table), the code above generate different sample than I have worked with.
# # So I have to create the sample by hand:
# empty_sample = mash_linked_data.loc[[878, 91, 5240, 3728, 1125, 2479, 4981, 1134, 5089, 1129]]

# empty_sample["text"][878] = "nejsou zn. plicní hypertenze"
# empty_sample["text"][91] = "hormostenické"
# empty_sample["text"][5240] = "kumulace a nehomogenita"
# empty_sample["text"][3728] = "biopsie"
# empty_sample["text"][1125] = "chemobioterapie"
# empty_sample["text"][2479] = "dysmorfické"
# empty_sample["text"][4981] = "anikterické"
# empty_sample["text"][1134] = "cerucal"
# empty_sample["text"][5089] = "palpitace"
# empty_sample["text"][1129] = "mamily klidné"

# empty_sample = mash_search_table(empty_sample)
# empty_sample

There is one mistake which we can correct automaticly and it is not having space after punctuation mark. We can see that if we have space after punctuation then it find something, else it didn't.

In [12]:
# print(len(search_from_bottom("zn. plicní", mash_search)))
# print(len(search_from_bottom("zn.plicní", mash_search)))

We can see that there is around 150 examples of this mistakes.

In [13]:
def is_space_after_punc(string):
    punctuation = [".", ",", "!", "?", ":", ";", "+"]
    for i in range(len(string) - 1):
        if string[i] in punctuation and string[i+1] != " " and string[i+1] not in punctuation:
            return False
 
    return True

def insert_space_after_punc(string):
    punctuation = [".", ",", "!", "?", ":", ";", "+"]
    for i in range(len(string) - 1):
        if string[i] in punctuation and string[i+1] != " " and string[i+1] not in punctuation:
            string = string[:i+1] + " " + string[i+1:]
 
    return string

inserted_space_data = data.copy()
inserted_space_data["text"] = inserted_space_data["text"].apply(insert_space_after_punc)
no_space = data[~data["text"].apply(is_space_after_punc)]
len(no_space)

155

In [14]:
if os.path.isfile("saved_search/data_mash_inserted_space.csv"):
    inserted_space_mash_linked_data = pd.read_csv("saved_search/data_mash_inserted_space.csv")
    inserted_space_mash_linked_data.index = inserted_space_mash_linked_data["Unnamed: 0"]
    inserted_space_mash_linked_data.drop(["Unnamed: 0"], axis=1, inplace=True)
    inserted_space_mash_linked_data["search"] = inserted_space_mash_linked_data["search"].apply(from_string_to_list)
else:
    inserted_space_mash_linked_data = mash_linked_data.copy()
    no_space["text"] = no_space["text"].apply(insert_space_after_punc)
    for i in no_space.index:
        inserted_space_mash_linked_data["search"][i] = search_from_bottom(no_space["text"][i], mash_search)
        inserted_space_mash_linked_data["text"][i] = no_space["text"][i]
    inserted_space_mash_linked_data.to_csv("saved_search/data_mash_inserted_space.csv")    
        
print_linking_stats(mash_linked_data["search"])
print()
print_linking_stats(inserted_space_mash_linked_data["search"])

Number of empty: 298 (10.631466286122013 %)
Mean from number of matches: 21.931858722797003
Median from number of matches: 25.0
Maximal of matches: 138

Number of empty: 256 (9.133071708883339 %)
Mean from number of matches: 22.350338922582946
Median from number of matches: 25.0
Maximal of matches: 138


Thanks to this upgrade we improved search by finding 40 new matches.

### Linking to CZ Mash through Medvik 

Now we try to link through czech mash, I have downloaded it from NLK (národní lékařská knihovna): https://nlk.cz/pro-knihovny/data/#mesh-cz

I called this linking as Medvik, because there is web service called Medvik: https://www.medvik.cz/bmc/subject.do, where you can search in czech mash.

I used already improved methods from Mash_search. First I experimenced with search, which tests if contains gived text.

In [15]:
offline_mshcz = elt.parse('databaze/MeSH2023_Marc21_Alma.xml').getroot()

In [16]:
def patternize(string):
    result = []
    for i in string:
        if i in '<([{\\^-=$!|]})?*+.>]':
            result.append("\\" + i)
        else:
            result.append(i)
    return "".join(result)


def medvik_search(string, test):
    result = []
    for child in offline_mshcz:
        for subchild in child.iter("{http://www.loc.gov/MARC21/slim}subfield"):
            if subchild.text and test(string, subchild.text):
                try:
                    code = [i for i in child.findall("{http://www.loc.gov/MARC21/slim}controlfield") if i.attrib["tag"] == "001" ][0].text
                    name = [i for i in child.findall("{http://www.loc.gov/MARC21/slim}datafield") if i.attrib["tag"] == "150" ][0][0].text
                    result.append((code, name))
                    break
                except IndexError:
                    break                
    return result


def medvik_exact_search(string):              
    return medvik_search(string, lambda x, y: x.lower() == y.lower())


def medvik_words_search(string):      
    return medvik_search(string, lambda x, y: (" " + x.lower() + " ") in y.lower())


def medvik_match_search(string):
    return medvik_search(string, lambda x, y: x.lower() in y.lower())


def medvik_combined_search(string):
    result = medvik_exact_search(string)
    if len(result) == 0:
        result = medvik_words_search(string)
    if len(result) == 0:
        result = medvik_match_search(string)
    
    return result


def medvik_exact_search_table(db):
    return seach_table(db, lambda x: search_from_bottom(x, medvik_exact_search, output_state=True))


def medvik_words_search_table(db):
    return seach_table(db, lambda x: search_from_bottom(x, medvik_words_search, output_state=True))
    

def medvik_match_search_table(db):
    return seach_table(db, lambda x: search_from_bottom(x, medvik_match_search, output_state=True))


def medvik_combined_search_table(db):
    return seach_table(db, lambda x: search_from_bottom(x, medvik_combined_search, output_state=True))

In [17]:
if os.path.isfile("saved_search/data_medvik_contains.csv"):
    medvik_match_search_data = pd.read_csv("saved_search/data_medvik_contains.csv")
    medvik_match_search_data.index = medvik_match_search_data["Unnamed: 0"]
    medvik_match_search_data.drop(["Unnamed: 0"], axis=1, inplace=True)
    medvik_match_search_data["search"] = medvik_match_search_data["search"].apply(from_string_to_list)
    
else:
    count = 0
    medvik_match_search_data = medvik_match_search_table(inserted_space_data)
    medvik_match_search_data.to_csv("saved_search/data_medvik_contains.csv")
    
print_linking_stats(inserted_space_mash_linked_data["search"])
print()
print_linking_stats(medvik_match_search_data["search"])

Number of empty: 256 (9.133071708883339 %)
Mean from number of matches: 22.350338922582946
Median from number of matches: 25.0
Maximal of matches: 138

Number of empty: 270 (9.632536567962898 %)
Mean from number of matches: 1670.0117731002497
Median from number of matches: 28.0
Maximal of matches: 43684


In [18]:
print([i for i in medvik_match_search_data["search"].apply(lambda x: len(x)).sample(10, random_state=42)])

temp = medvik_match_search_data["search"].apply(lambda x: len(x))
print(f"number of searches longer than 100 matches in medvik_match_search_data: {len(temp[temp > 100])}")

[1904, 1720, 0, 27715, 1722, 94, 9, 10724, 0, 112]
number of searches longer than 100 matches in medvik_match_search_data: 1017


We can see, that for some examples this is working well. But for some we have really lots of samples whose lenght grows exponentially.

For this reasons it might be better to use some different match method instead:
- First method is contain search (I used it before) - test if contains given text
- Next method is word search - test if contains given text as word (there are spaces around)
- Next method is exact search - test if contains exactly given text
- The last method is combined search - first text exact, then word, then contains (if some success then end).

In [19]:
test_data = inserted_space_data.sample(20, random_state=42)

if os.path.isfile("saved_search/test_medvik.csv"):
    test_data = pd.read_csv("saved_search/test_medvik.csv")
    test_data["search_match"] = test_data["search_match"].apply(from_string_to_list)
    test_data["search_exact"] = test_data["search_exact"].apply(from_string_to_list)
    test_data["search_words"] = test_data["search_words"].apply(from_string_to_list)
    test_data["search_combined"] = test_data["search_combined"].apply(from_string_to_list)
else:
    count = 0
    test_data["search_match"] = medvik_match_search_table(test_data)["search"]
    test_data["search_exact"] = medvik_exact_search_table(test_data)["search"]
    test_data["search_words"] = medvik_words_search_table(test_data)["search"]
    test_data["search_combined"] = medvik_combined_search_table(test_data)["search"]
    test_data.to_csv("saved_search/test_medvik.csv")
    
print("Contains match")
print_linking_stats(test_data["search_match"])
print("\nWords match")
print_linking_stats(test_data["search_words"])
print("\nExact match")
print_linking_stats(test_data["search_exact"])
print("\nCombined match")
print_linking_stats(test_data["search_combined"])

Contains match
Number of empty: 3 (15.0 %)
Mean from number of matches: 2265.2
Median from number of matches: 15.5
Maximal of matches: 27715

Words match
Number of empty: 6 (30.0 %)
Mean from number of matches: 194.0
Median from number of matches: 5.5
Maximal of matches: 1918

Exact match
Number of empty: 11 (55.00000000000001 %)
Mean from number of matches: 3.05
Median from number of matches: 0.0
Maximal of matches: 22

Combined match
Number of empty: 3 (15.0 %)
Mean from number of matches: 261.95
Median from number of matches: 6.5
Maximal of matches: 1904


We can see that using exact match we get rid of the long matches but it have quite low success rate. Using words match is something in the middle (not good in both ways).

As last option we used combined match (first try exact, if don't success then words, then only match). This seems as the best methods (this doesn't create too large lists and has the same number of empty matches as contains match) 

In [20]:
if os.path.isfile("saved_search/data_medvik_combined.csv"):
    medvik_combined_search_data = pd.read_csv("saved_search/data_medvik_combined.csv")
    medvik_combined_search_data.index = medvik_combined_search_data["Unnamed: 0"]
    medvik_combined_search_data.drop(["Unnamed: 0"], axis=1, inplace=True)
    medvik_combined_search_data["search"] = medvik_combined_search_data["search"].apply(from_string_to_list)
else:
    count = 0
    medvik_combined_search_data = medvik_combined_search_table(inserted_space_data)
    medvik_combined_search_data.to_csv("saved_search/data_medvik_combined.csv")
       
print_linking_stats(medvik_combined_search_data["search"])

Number of empty: 270 (9.632536567962898 %)
Mean from number of matches: 488.3999286478773
Median from number of matches: 7.0
Maximal of matches: 28651


#### Duplicates
It is possible to get duplicates in list of matches, when getting the same match from two different words from text (or combinations of the same lenght) 

In mash search there are few duplicates, but in medvik search it can be serious problem - we can see, that maximum of matches in contains search is reduced nearly by 15 000.

In [21]:
len(inserted_space_mash_linked_data[~inserted_space_mash_linked_data["search"].apply(lambda x: len(set(x)) == len(x))])

5

In [22]:
def remove_dup_preserve_order(l):
    seen = set()
    seen_add = seen.add
    return [x for x in l if not (x in seen or seen_add(x))]


print("Before removing duplicates:")
print("Mash search")
print_linking_stats(inserted_space_mash_linked_data["search"])
print("\nMedvik contains search")
print_linking_stats(medvik_match_search_data["search"])
print("\nMedvik combined search")
print_linking_stats(medvik_combined_search_data["search"])

inserted_space_mash_linked_data["search"] = inserted_space_mash_linked_data["search"].apply(remove_dup_preserve_order)
medvik_match_search_data["search"] = medvik_match_search_data["search"].apply(remove_dup_preserve_order)
medvik_combined_search_data["search"] = medvik_combined_search_data["search"].apply(remove_dup_preserve_order)

print("\n\nAfter removing duplicates:")
print("Mash search")
print_linking_stats(inserted_space_mash_linked_data["search"])
print("\nMedvik contains search")
print_linking_stats(medvik_match_search_data["search"])
print("\nMedvik combined search")
print_linking_stats(medvik_combined_search_data["search"])

Before removing duplicates:
Mash search
Number of empty: 256 (9.133071708883339 %)
Mean from number of matches: 22.350338922582946
Median from number of matches: 25.0
Maximal of matches: 138

Medvik contains search
Number of empty: 270 (9.632536567962898 %)
Mean from number of matches: 1670.0117731002497
Median from number of matches: 28.0
Maximal of matches: 43684

Medvik combined search
Number of empty: 270 (9.632536567962898 %)
Mean from number of matches: 488.3999286478773
Median from number of matches: 7.0
Maximal of matches: 28651


After removing duplicates:
Mash search
Number of empty: 256 (9.133071708883339 %)
Mean from number of matches: 22.33927934356047
Median from number of matches: 25.0
Maximal of matches: 138

Medvik contains search
Number of empty: 270 (9.632536567962898 %)
Mean from number of matches: 1625.7213699607564
Median from number of matches: 28.0
Maximal of matches: 29327

Medvik combined search
Number of empty: 270 (9.632536567962898 %)
Mean from number of ma

## Choosing best match with Chat GPT
The idea behind our model is first link term to database and then choose the best one by some pretrained language model.

I use GPT-3.5, because it is free to access with limitations (There are some limits of access per day. And there is limited number of access per account. Then we have to pay.), it is fast and it is well known.

The message to GPT is in this format:

Který z uvedených lékařských pojmů s jeho popisem nejlépe odpovídá pojmu: "[MEDICAL TERM]":

    1. [DESCRIPTION_N.1] (pojem: [TERM_N.1])
    2. [DESCRIPTION_N.2] (pojem: [TERM_N.2])
    ...
    
Jako odpověď mi pošli pouze číslo odpovědi. Pokud to nebude žádná z možností, pak odpověz NONE.

In [23]:
def medvik_find_by_code(string):
    if len(string) == 0:
        return ""
    
    for child in offline_mshcz:
        try:
            code = [i for i in child.findall("{http://www.loc.gov/MARC21/slim}controlfield") if i.attrib["tag"] == "001" ][0].text
            if code == string:
                d = [i.iter("{http://www.loc.gov/MARC21/slim}subfield") for i in child.findall("{http://www.loc.gov/MARC21/slim}datafield") if i.attrib["tag"] == "680"][0]
                return next(d).text
        except IndexError:
            continue      
            
    return ""


def mash_find_by_code(string):
    path = f'https://uts-ws.nlm.nih.gov/rest/content/current/CUI/{string}'
    query = {
             'apiKey':NIH_api,
    }
    res = requests.get(path, params=query)
    
    if res.status_code <= 200:
        try:
            data = json.loads(res.text)
            definition = data["result"]["definitions"]

            if "https://uts-ws.nlm.nih.gov/" in definition:
                path = definition
                res = requests.get(path,params=query)
                try:
                    return [i["value"] for i in json.loads(res.text)["result"] if i["rootSource"] == "MSHCZE"][0]
                except IndexError:
                    pass
                try:
                    return [i["value"] for i in json.loads(res.text)["result"] if i["rootSource"] == "MSH"][0]
                except IndexError:
                    definition = "NONE"
        except Exception:
            definition = "NONE"
            print(string, res.text)
        
        if definition == "NONE":
            return data["result"]["name"]
        
        return definition
    else:
        print(string)
        print(res.status_code, res.text)
        
    return ""

In [24]:
with open("APIkeys/NIH", "r") as f:
    NIH_api = f.read()

def send_to_GPT(message):
    if message == "":
        return ""

    with open("APIkeys/chatGTP", "r") as f:
        chatgpt_api = f.read()

    client = OpenAI(api_key=chatgpt_api)
    return client.chat.completions.create(
                model="gpt-3.5-turbo-16k",
                messages=[{"role": "user", "content": message}],
                stream=False)


def message_for_GPT(string, li, find, context=None):
    if len(li) == 0:
        return ""
            
    if context is None:
        result = [f"Který z uvedených lékařských pojmů s jeho popisem nejlépe odpovídá pojmu: \"{string}\":\n"]
    else:
        result = [f"Který z uvedených lékařských pojmů s jeho popisem nejlépe odpovídá pojmu: \"{string}\" v kontextu:  \"{context}\":\n"]
        
    j = 1
    for i in li:
        result.append(f"{j}. {find(i[0])} (pojem: {i[1]})\n")
        j += 1
        
    result.append("Jako odpověď mi pošli pouze číslo odpovědi. Pokud to nebude žádná z možností, pak odpověz NONE. Pokud to není lékařský pojem odpověz taky NONE.")
    
    return "".join(result)
    
def find_int(string):
    result = []
    for i in string:
        if i.isdigit():
            result.append(i)

    return int("".join(result))


def from_GPT(result, li, find):
    try:
        i = find_int(result.choices[0].message.content) - 1
        # int(result.choices[0].message.content.split(".")[0].strip(" " + ascii_letters + punctuation)) - 1
        return (li[i][0], li[i][1], find(li[i][0]))
    except ValueError:
        pass
    except IndexError:
        pass

    return None

Because ChatGPT has problem with long messages (and sometimes we get really long results with medvik_combined), we have to restrict these messages and drop them.

In [25]:
if os.path.isfile("saved_search/explanation_sample.csv"):
    explanation = pd.read_csv("saved_search/explanation_sample.csv")
    explanation.index = explanation["Unnamed: 0"]
    explanation.drop(["Unnamed: 0"], axis=1, inplace=True)
    explanation["mash_explanation"] = explanation["mash_explanation"].fillna(value="N/A").apply(from_string_to_tuple)
    explanation["medvik_explanation"] = explanation["medvik_explanation"].fillna(value="N/A").apply(from_string_to_tuple)
    explanation["mash_search"] = explanation["mash_search"].apply(from_string_to_list)
    explanation["medvik_search_combined"] = explanation["medvik_search_combined"].apply(from_string_to_list)
else:
    explanation = results.sample(100, random_state=38)
    explanation["mash_explanation"] = "N/A"
    for j in explanation.index:
        message = message_chatGTP(explanation["text"][j], explanation["mash_search"][j], mash_find_by_code)
        response = send_to_GPT(message)
        explanation["mash_explanation"][j] = from_GPT(response, explanation["mash_search"][j], mash_find_by_code)     
        
#         If the message is too long we cannot send it to chatGPT, so we drop shorter messages.
        if len(explanation["medvik_search_combined"][j]) > 50:
            continue
        message = message_for_GPT(explanation["text"][j], explanation["medvik_search_combined"][j], medvik_find_by_code)
        response = send_to_GPT(message)
        explanation["medvik_explanation"][j] = from_GPT(response, explanation["medvik_search_combined"][j], medvik_find_by_code)
    explanation.to_csv("saved_search/explanation_sample.csv")

explanation.head()

,label,text,mash_search,medvik_search_combined,mash_explanation,medvik_explanation
Unnamed: 0,,,,,,
1633,NE symptom,KI100,[],[],None,None
5536,procedura,st. p. tru-cut biopsii,"[(C1170898, Companion P/ST 1000ML), (C4015802,...","[(D000039, peritonzilární absces), (D000081182...","(C5700874, Percutaneous pulmonary artery revas...",None
412,osobní anamnéza,konizace čípku,"[(C0195324, Conization)]","[(D002583, nádory děložního čípku), (D019092, ...","(C0195324, Conization, Kruhovité kuželovité vy...","(D019092, konizace děložního čípku, Kruhovité ..."
4517,procedura,operace: ITP,"[(C0398650, Immune thrombocytopenic purpura), ...","[(D007293, inosintrifosfát)]","(C3842543, Idiopathic thrombocytopenia (ITP), ...",None
5757,symptom,neostře konturované ložisko 7x5mm,"[(C0241148, Cutaneous plaque), (C1533591, Calc...","[(D001253, astrocyty), (D002833, choroiditida)...","(C0235456, Thyroid nodular, Thyroid nodular)","(D002833, choroiditida, Zánět cévnatky, zadní ..."


In [26]:
len(explanation[explanation["medvik_search_combined"].apply(lambda x: len(x) > 50)])

22

## Conclusion Basic Access
There I am going to evaluate the basic access for linking entities.

### Results for Linking
Now we look how we have been successful with linking to database with respect to different labels.

In [27]:
results = inserted_space_data.copy()
results["mash_search"] = inserted_space_mash_linked_data["search"]
results["medvik_search_combined"] = medvik_combined_search_data["search"]
results.head(3)

,label,text,mash_search,medvik_search_combined
0,symptom,jemný fibrózní proužek,"[(C0030848, Peyronie Disease), (C0227365, Taen...","[(D000077275, fibrózní dysplazie kraniofaciáln..."
1,procedura,neoadjuvantní CHT,"[(C0600558, Neoadjuvant Therapy), (C1422359, S...","[(D000014, abnormality vyvolané léky), (D00313..."
2,medikace,novalgin,"[(C0917937, Novalgin)]","[(D004177, metamizol)]"


In [28]:
os_anamneza = results[results.label == "osobní anamnéza"]
ne_os_anamneza = results[results.label == "NE osobní anamnéza"]
medikace = results[results.label == "medikace"]
ne_medikace = results[results.label == "NE medikace"]
symptom = results[results.label == "symptom"]
ne_symptom = results[results.label == "NE symptom"]
procedura = results[results.label == "procedura"]
    
print("osobní anamnéza:")    
print_linking_stats(os_anamneza["mash_search"])
print_linking_stats(os_anamneza["medvik_search_combined"])

print("\nNE osobní anamnéza:")
print_linking_stats(ne_os_anamneza["mash_search"])
print_linking_stats(ne_os_anamneza["medvik_search_combined"])

print("\nmedikace:")
print_linking_stats(medikace["mash_search"])
print_linking_stats(medikace["medvik_search_combined"])

print("\nNE medikace:")
print_linking_stats(ne_medikace["mash_search"])
print_linking_stats(ne_medikace["medvik_search_combined"])

print("\nsymptom:")
print_linking_stats(symptom["mash_search"])
print_linking_stats(symptom["medvik_search_combined"])

print("\nNE symptom:")
print_linking_stats(ne_symptom["mash_search"])
print_linking_stats(ne_symptom["medvik_search_combined"])

print("\nprocedura:")
print_linking_stats(procedura["mash_search"])
print_linking_stats(procedura["medvik_search_combined"])

print("\n\nall:")
print_linking_stats(results["mash_search"])
print_linking_stats(results["medvik_search_combined"])
print("Number of empty in both search:", len(results[(results["mash_search"].apply(lambda x: len(x) == 0)) & (results["medvik_search_combined"].apply(lambda x: len(x) == 0))]))

osobní anamnéza:
Number of empty: 11 (5.14018691588785 %)
Mean from number of matches: 22.939252336448597
Median from number of matches: 24.0
Maximal of matches: 86
Number of empty: 18 (8.411214953271028 %)
Mean from number of matches: 459.1588785046729
Median from number of matches: 11.5
Maximal of matches: 13185

NE osobní anamnéza:
Number of empty: 4 (6.557377049180328 %)
Mean from number of matches: 20.83606557377049
Median from number of matches: 25.0
Maximal of matches: 75
Number of empty: 14 (22.950819672131146 %)
Mean from number of matches: 190.34426229508196
Median from number of matches: 1.0
Maximal of matches: 5777

medikace:
Number of empty: 73 (23.934426229508198 %)
Mean from number of matches: 14.462295081967213
Median from number of matches: 8.0
Maximal of matches: 82
Number of empty: 101 (33.114754098360656 %)
Mean from number of matches: 96.88196721311475
Median from number of matches: 1.0
Maximal of matches: 5134

NE medikace:
Number of empty: 1 (6.25 %)
Mean from nu

We can see, that most labels work quite similar to each others. One big exception is label "medikace" (in english medication). This label have significantly higher empty rate, but have less of matches. 

I this is caused by shorter text (have less of words), but these words are usually more concrete (as names of medicine).

### Results for Mash and Medvik search
Now I am going to evaluate results from GPT for linking.

For this I have created a sample of 100 entries, which I have send to GPT to find best match. From them I picked up another sample of 35 entries which I have evaluated manually.

In [29]:
print("Number of examples: {}".format(len(explanation)))

print("Number of empty Linking for medvik_search_combined: {}".format(explanation["medvik_search_combined"].apply(lambda x: len(x) == 0).sum()))
print("Number of empty Linking for Mash_search: {}".format(explanation["mash_search"].apply(lambda x: len(x) == 0).sum()))

print("Number of not assigned for Medvik: {}".format(explanation["medvik_explanation"].apply(lambda x: x is None).sum()))
print("Number of not assigned for Mash: {}".format(explanation["mash_explanation"].apply(lambda x: x is None).sum()))

Number of examples: 100
Number of empty Linking for medvik_search_combined: 11
Number of empty Linking for Mash_search: 10
Number of not assigned for Medvik: 39
Number of not assigned for Mash: 24


We can see, that there is high number of not assigned in both searches. But in Medvik it is much higher. The higher number in Medvik is because we have to drop very long searches. In the future accesses it is important to handle the long.

In [30]:
asign = []
for j in explanation.sample(35, random_state=42).index:   
    if explanation["medvik_explanation"][j] is not None:
        x = explanation["medvik_explanation"][j]
        asign.append((explanation["text"][j], "Medvik", "{} ({})".format(x[1], x[2])))
    if explanation["mash_explanation"][j] is not None:
        x = explanation["mash_explanation"][j]
        asign.append((explanation["text"][j], "Mash", "{} ({})".format(x[1], x[2])))
    if explanation["mash_explanation"][j] is None and explanation["medvik_explanation"][j] is None:
        asign.append((explanation["text"][j], "None", "Empty"))

i = 0
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)

assert len(set([i[3] for i in asign])) == 4

In [31]:
print("Not assigned medvik:", 35 - len([i for i in asign if i[1] == "Medvik"]))
print("Mistakes from medvik:", len([i for i in asign if i[3] == 'Wrong' and i[1] == "Medvik"]))
print("Partially right from medvik:", len([i for i in asign if i[3] == 'Partially' and i[1] == "Medvik"]))
print("Right from medvik:", len([i for i in asign if i[3] == 'Right' and i[1] == "Medvik"]))

print("\nNot assigned mash:", 35 - len([i for i in asign if i[1] == "Mash"]))
print("Mistakes from mash:", len([i for i in asign if i[3] == 'Wrong' and i[1] == "Mash"]))
print("Partially right from mash:", len([i for i in asign if i[3] == 'Partially' and i[1] == "Mash"]))
print("Right from mash:", len([i for i in asign if i[3] == 'Right' and i[1] == "Mash"]))

Not assigned medvik: 15
Mistakes from medvik: 13
Partially right from medvik: 5
Right from medvik: 2

Not assigned mash: 11
Mistakes from mash: 13
Partially right from mash: 5
Right from mash: 6


The results are not so good (but it was base line).

For medvik we have got 20 % for at least partially right linking. And 6 % for completely right linking.

For mash we have got 31 % for at least partially right linking. And 17 % for completely right linking.

These results need improvement. For this reason I create new Improved linking based on different method. But before it I look at the wrong, partially and right results and observe if there is some pattern.


In [32]:
med = [i[0] for i in asign if i[3] == 'Wrong' and i[1] == "Medvik"]
mash = [i[0] for i in asign if i[3] == 'Wrong' and i[1] == "Mash"]
print("\ntext wrong in medvik and mash:", [i for i in med if i in mash])
print("text wrong only in medvik", [i for i in med if i not in mash])
print("text wrong only in mash", [i for i in mash if i not in med])

med = [i[0] for i in asign if i[3] == 'Partially' and i[1] == "Medvik"]
mash = [i[0] for i in asign if i[3] == 'Partially' and i[1] == "Mash"]
print("\ntext partially right in medvik and mash:", [i for i in med if i in mash])
print("text partially right only in medvik", [i for i in med if i not in mash])
print("text partially right only in mash", [i for i in mash if i not in med])

med = [i[0] for i in asign if i[3] == 'Right' and i[1] == "Medvik"]
mash = [i[0] for i in asign if i[3] == 'Right' and i[1] == "Mash"]
print("\ntext right in medvik and mash:", [i for i in med if i in mash])
print("text right only in medvik", [i for i in med if i not in mash])
print("text right only in mash", [i for i in mash if i not in med])


text wrong in medvik and mash: ['MG vlevo', 'bez šelestu', 'neostře konturované ložisko 7x5mm', 'bez poruchy kinetiky myokardu', 'norm. velikost srdeč. oddílů', 'jizevnaté změny v ZDK', 'tumorozní ložisko']
text wrong only in medvik ['benigní verifikované ložisko v HKK', 'játra bez solidních patologických ložiskových změn', 'USG pravé mammy a axilly', 'mírný sekund lymfedém pod axilou, v zadní axil. řase', 'apokrinní metaplazií a místy i adenóza', 'mastitis: ne']
text wrong only in mash ['někdy tahv oblasti jizvy', 'subjektivně bez bolestí', 'močový měchýř hypodenzní homogenní náplně', 'bránice hladká', 'fibropleurální změny', 'dlouhodobě stac. nález']

text partially right in medvik and mash: ['parciální mastektomie vlevo']
text partially right only in medvik ['močový měchýř hypodenzní homogenní náplně', 'bránice hladká', 'zn. krvácení', 'průjmy']
text partially right only in mash ['játra bez solidních patologických ložiskových změn', 'USG pravé mammy a axilly', 'uzliny fyziologické'

- We can see, that there are higher percentage of partially right. This is because we have been looking for one sumarizing linking. But very often such linking doesn't exists. At these cases we only find linking which corresponds to part of the text not for the whole. 
- The completely right one, are usualy very short (I think the reason is same as above).

## Improved Search

Now I am going to try another access, where I try handle former mistakes. The biggest change is not to have one list of links for whole text, but to have one list for each word from the text. And then try to explain this word.

In [33]:
def search_from_bottom_no_drop(string, func):
    splitted_input = (string.split(" "))
    lenght = len(splitted_input)
    
    result_dict = {}
    for word in splitted_input:
        result_dict[word] = []
        
    for j in range(1, lenght + 1):
        splitted_dict={}
        for elem in splitted_input:
            splitted_dict[j] = False
                    
        for words in combinations(splitted_input, j):
            data = func(" ".join(words))
            if len(data) == 0:
                continue
            for j in words:
                splitted_dict[j] = True
                result_dict[j] += [(i[0], i[1], words) for i in data]
                    
        splitted_input = [j for j, i in splitted_dict.items() if i]
        if len(splitted_input) == 0:
            break
            
    for k, v in result_dict.copy().items():
        result_dict[k] = [i for i in v if len(i[2]) == len(v[-1][2])]
        pop_key = True
        for n in set([i[2] for i in result_dict[k]]):
            string = " ".join(n)
            if string == k:
                pop_key = False
            if string in result_dict:
                continue
            result_dict[string] = [(i[0], i[1]) for i in result_dict[k] if " ".join(i[2]) == string]
            
        if pop_key:
            result_dict.pop(k)
    
    return result_dict

First we need to link them to databases.

In [34]:
if os.path.isfile("saved_search/new_access.csv"):
    test_new_access = pd.read_csv("saved_search/new_access.csv")
    test_new_access.index = test_new_access["Unnamed: 0"]
    test_new_access.drop(["Unnamed: 0"], axis=1, inplace=True)
    test_new_access["mash_search"] = test_new_access["mash_search"].apply(from_string_to_dict)
    test_new_access["medvik_search"] = test_new_access["medvik_search"].apply(from_string_to_dict)

else:
    test_new_access = results[["text"]].sample(100, random_state=25)
    test_new_access["mash_search"] = "N/A"
    for j in test_new_access.index:
        test_new_access["mash_search"][j] = search_from_bottom_no_drop(test_new_access["text"][j], mash_search)

    test_new_access["medvik_search"] = "N/A"
    for j in test_new_access.index:
        test_new_access["medvik_search"][j] = search_from_bottom_no_drop(test_new_access["text"][j], medvik_combined_search)

    test_new_access.to_csv("saved_search/new_access.csv")

To send message to GPT we need not to exceed certain length. We try to discover some lenght, by which the medvik search returns only noice (or really probably).

In [35]:
def print_long_searches(db, limit=20):
    temp = db.apply(lambda x: [(i, len(x[i])) for i in x if len(x[i]) > limit])
    print(sorted(list(temp[temp.apply(lambda x: len(x) != 0)]), key=(lambda x: x[0][1])))

print_long_searches(test_new_access["medvik_search"])

[[('vlně', 22)], [('krvácení', 23)], [('mírné', 23)], [('stomatologické', 27), ('vyš', 874)], [('nových', 30)], [('stabilní', 32)], [('strukturou', 36)], [('laloku', 38)], [('susp', 52)], [('i na', 58), ('tlustého střeva', 30)], [('operace', 58)], [('nebol', 66)], [('léčí s', 76)], [('spíš', 77)], [('příl', 84)], [('e', 95)], [('genetické', 99)], [('p.', 119)], [('NACT', 128), ('-', 5334)], [('plicní', 162)], [('klinické', 164)], [('ME s', 166)], [('ME s', 166)], [('léčba', 181)], [('vyšetření', 186)], [('příznaky', 314)], [('není', 327)], [('TAD', 331), ('l. I', 72)], [('- po', 386)], [('pomocí', 393)], [('pm', 548)], [('PM', 548), ('se', 7588)], [('negat', 740)], [('Cor', 1304)], [('patol', 6932)], [('v', 7248)], [('toxicita', 11218)], [('v', 14496)]]


We can see, that for longer length than 40 we get mostly non-medical terms (or general medical terms).

In [36]:
def drop_long_searches(dictionary, limit=40):
    for key in dictionary.copy():
        if len(dictionary[key]) > limit:
            dictionary[key] = []
    return dictionary

test_new_access["medvik_search"] = test_new_access["medvik_search"].apply(drop_long_searches)

The second part is to choose the best one by GPT.

In [37]:
if os.path.isfile("saved_search/new_access_explanation.csv"):
    test_new_access = pd.read_csv("saved_search/new_access_explanation.csv")
    test_new_access.index = test_new_access["Unnamed: 0"]
    test_new_access.drop(["Unnamed: 0"], axis=1, inplace=True)
    test_new_access["mash_search"] = test_new_access["mash_search"].apply(from_string_to_dict)
    test_new_access["medvik_search"] = test_new_access["medvik_search"].apply(from_string_to_dict)
    test_new_access["mash_explanation"] = test_new_access["mash_explanation"].fillna("{}").apply(from_string_to_dict_to_tuple)
    test_new_access["medvik_explanation"] = test_new_access["medvik_explanation"].fillna("{}").apply(from_string_to_dict_to_tuple)

else:   
    test_new_access["medvik_explanation"] = "N/A"
    for i in test_new_access.index:
        l = test_new_access["medvik_search"][i]
        result = {}
        for text in l:
            message = message_for_GPT(text, l[text], medvik_find_by_code)
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
        test_new_access["medvik_explanation"][i] = result

    test_new_access["mash_explanation"] = "N/A"
    for i in test_new_access.index:
        l = test_new_access["mash_search"][i]
        result = {}
        for text in l:
            message = message_for_GPT(text, l[text], medvik_find_by_code)
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
        test_new_access["mash_explanation"][i] = result

    test_new_access.to_csv("saved_search/new_access_explanation.csv")

### Results
Now its time to examine the results of improved search.

In [38]:
print("Number of examples: {}".format(len(test_new_access)))

print("Number of empty Linking for Medvik_combined_search: {}".format(test_new_access["medvik_search"].apply(lambda x: len(x) == 0).sum()))
print("Number of empty Linking for Mash_search: {}".format(test_new_access["mash_search"].apply(lambda x: len(x) == 0).sum()))

print("Number of not assigned for Medvik: {}".format(test_new_access["medvik_explanation"].apply(lambda x: len(x) == 0).sum()))
print("Number of not assigned for Mash: {}".format(test_new_access["mash_explanation"].apply(lambda x: len(x) == 0).sum()))

Number of examples: 100
Number of empty Linking for Medvik_combined_search: 12
Number of empty Linking for Mash_search: 10
Number of not assigned for Medvik: 12
Number of not assigned for Mash: 15


There we have got much more better results than in basic access. We were able to choose the best match for nearly everything, what we have been able to find in database.

In [39]:
asign = []
for j in test_new_access.sample(35, random_state=42).index:   
    if test_new_access["medvik_explanation"][j] is not None:
        x = test_new_access["medvik_explanation"][j]
        asign.append((test_new_access["text"][j], "Medvik", "{}".format([(e, x[e]) for e in x])))
    if test_new_access["mash_explanation"][j] is not None:
        x = test_new_access["mash_explanation"][j]
        asign.append((test_new_access["text"][j], "Mash", "{}".format([(e, x[e]) for e in x])))
    if test_new_access["mash_explanation"][j] is None and test_new_access["medvik_explanation"][j] is None:
        asign.append((test_new_access["text"][j], "None", "Empty"))

i = 0
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)

In [40]:
print("Not assigned medvik:", len([i for i in asign if i[1] == "Medvik" and i[3] == ""]))
print("Mistakes from medvik:", len([i for i in asign if i[3] == 'Wrong' and i[1] == "Medvik"]))
print("Partially right from medvik:", len([i for i in asign if i[3] == 'Partially' and i[1] == "Medvik"]))
print("Right from medvik:", len([i for i in asign if i[3] == 'Right' and i[1] == "Medvik"]))

print("\nNot assigned mash:", len([i for i in asign if i[1] == "Mash" and i[3] == ""]))
print("Mistakes from mash:", len([i for i in asign if i[3] == 'Wrong' and i[1] == "Mash"]))
print("Partially right from mash:", len([i for i in asign if i[3] == 'Partially' and i[1] == "Mash"]))
print("Right from mash:", len([i for i in asign if i[3] == 'Right' and i[1] == "Mash"]))

Not assigned medvik: 1
Mistakes from medvik: 20
Partially right from medvik: 6
Right from medvik: 8

Not assigned mash: 2
Mistakes from mash: 18
Partially right from mash: 10
Right from mash: 5


The results are better than previous access:

For medvik we have got 40 % for at least partially right linking. And 23 % for completely right linking.

For mash we have got 43 % for at least partially right linking. And 14 % for completely right linking.

But even these results are very good, they need next improvement. For this reason I would like to try another improved linking. But first I look at the wrong, partially and right results and observe if there is some pattern.


In [41]:
med = [i[0] for i in asign if i[3] == 'Wrong' and i[1] == "Medvik"]
mash = [i[0] for i in asign if i[3] == 'Wrong' and i[1] == "Mash"]
print("text wrong in medvik and mash:", [i for i in med if i in mash])
print("text wrong only in medvik", [i for i in med if i not in mash])
print("text wrong only in mash", [i for i in mash if i not in med])

med = [i[0] for i in asign if i[3] == 'Partially' and i[1] == "Medvik"]
mash = [i[0] for i in asign if i[3] == 'Partially' and i[1] == "Mash"]
print("\ntext partially right in medvik and mash:", [i for i in med if i in mash])
print("text partially right only in medvik", [i for i in med if i not in mash])
print("text partially right only in mash", [i for i in mash if i not in med])

med = [i[0] for i in asign if i[3] == 'Right' and i[1] == "Medvik"]
mash = [i[0] for i in asign if i[3] == 'Right' and i[1] == "Mash"]
print("\ntext right in medvik and mash:", [i for i in med if i in mash])
print("text right only in medvik", [i for i in med if i not in mash])
print("text right only in mash", [i for i in mash if i not in med])

text wrong in medvik and mash: ['DKK: bez otoků', 'kličky tenkého i tlustého střeva na necíleném vyšetření přiměřeného kalibru i norm. šíře stěny', 'fibrocystické změny s mnohočetnými intraduktálními papilomy', 'beze změny zdra. satvu', 'oboustranné totální mastektomii', 'vpačování bradavek 0', 'bez patrných MTS', 'menzes no', 'mírné velikostní progresi', 'tamoxifenu', 'mamila: pravidelné stavby', 'mutace v genu NBN', 'vlevo bez patol', 'bez nových poíží']
text wrong only in medvik ['gynekologické operace', 'AS reg', 'normě', 'váha stabilní', 'kompletní klinické regrese', 'regrese v prsu']
text wrong only in mash ['parc. ME s disekcí axily', 'anastrozol', 'jizva v ZHQ zhojena', 'neurotoxicita']

text partially right in medvik and mash: ['kůže intaktní', 'stolice spíš zácpovitá', 'hysterectomii pro krvácení', 'klinicky lipom při sternu', 'GIT toxicita G1']
text partially right only in medvik ['parc. ME s disekcí axily']
text partially right only in mash ['USG (Mamma, Axilla', 'váha stab

- I have noticed, that most of the words which have been linked correctly have been in basic form (Sg 1).
- Some from the wrong one have mistake or there a punctuation is doing some mess.


### Some Adititional comments on results
The results above aren't pretty good. Some changes and improvements can be done in the medical text. But lots of these changes will be label specific.

During labeling I have noticed, that there can be problem with punctuation (because of punctuation we usualy don't find match). For this reason I tried to discover if there is some punctuation we can remove.

In [42]:
punc_in_data = set()
data["text"].apply(lambda x: punc_in_data.update(set([i for i in x if i in punctuation])))
punc_in_data

{'%', '(', ')', '+', ',', '-', '.', '/', ':', ';', '['}

In [43]:
# temp = data.sample(200, random_state=10)
for p in sorted(list(punc_in_data)):
    print(f"Punctuation: {p}")
    data[data["text"].apply(lambda x: p in x)].sample(frac=1, random_state=10).head(10)["text"].apply(lambda x: print(x))
    print()


Punctuation: %
ki 100%, aloepcie,sliznice prokrveny, oběh. komp
ki 100%,alopecie,sliznice prokrveny, oběh. komp
inf. glucosi 10% 500 ml
růst trabekulárně, solidně, s polarizací do 10 %, jádra s jemným chromatinem
KI 100% lucidní

Punctuation: (
proliferace dle Ki67 (automat) 35
RTG (Plíce
cílenou axilární disekcí (SNB + klipovaná uzlina
biopsie sentinelové uzliny (SNB
vyrážka na kůži (kopřivka
re-resekce - laterální části (kůže + podkoží) + disekce pravé axily - en bloc
adjuvantní radioterapii na hrudní stěnu vpravo + axilu vpravo ( I-IV.etáž
páteře C+ L( diskopatie
USG (Břicho
alergie: Ketazon (urtika

Punctuation: )
proliferace dle Ki67 (automat) 35
odstranění klipované (a ev. sentinelové) uzliny
re-resekce - laterální části (kůže + podkoží) + disekce pravé axily - en bloc

Punctuation: +
adjuv CHt paclitaxel weekly 12x + trastuzuab
ko + trombo + dif
LHRH+IA
H+L nehmatné
hye+ae
UZ+MMG
totální ME + SNB vlevo
cílenou axilární disekcí (SNB + klipovaná uzlina
LDK, st.p. HYE+AE
páteře C+ 

We can see, that most of the punctuation have no specific meaning, so we can substitute them with space (most of them stands there instead of space).

On the other hand colon has specific meaning, its meaning is specific some category and the rest from the text is about it.

And dot have specific meaning, which specify that the word is only shortcut or dash which is part of some words.

In [44]:
data_improved_punctuation = data.copy()
data_improved_punctuation["original_text"] = data["text"]
data_improved_punctuation["text"] = data_improved_punctuation["text"].apply(
    lambda text: "".join([l if l not in punctuation or l in ["."] else " " for l in text]))
data_improved_punctuation["text"] = data_improved_punctuation["text"].apply(clean)

If we inspect the longer matches, we can see, that many times we've got there nonsense combination (like word with conjunction behind it). For this reason I am going to modify the algorithm, where it will preserve even the shorter matches.

In [45]:
temp = test_new_access["medvik_explanation"].apply(lambda x: {k: x[k][1] for k in x if len(k.split(" ")) >= 2 if x[k] is not None})
print(list(temp[temp != {}].head(5)))
temp = test_new_access["mash_explanation"].apply(lambda x: {k: x[k][1] for k in x if len(k.split(" ")) >= 2 if x[k] is not None})
print(list(temp[temp != {}].head(5)))

[{'beze změny': 'tropismus'}, {'sekund a': 'syndromy spánkové apnoe', 'lymfedém a': 'syndrom žlutých nehtů', 'pod hrudní': 'bederní obratle', 'pod a': 'parotis', 'hrudní a': 'bránice', 'stěnou a': 'sagitální abdominální rozměr'}, {'změny s': 'incontinentia pigmenti', 's mnohočetnými': 'vrozené srdeční vady'}, {'vše v': 'naučená bezmocnost'}, {'v kloubech': 'synoviální cysta', 'v a': 'parciální tromboplastinový čas', 'a kloubech': 'juvenilní artritida', 'a kyčelních': 'artróza kyčelních kloubů'}]
[{'hrudní a': 'Thoracoabdominal aortic aneurysm'}, {'jizva v': "Vaccination site scar',"}, {'Baker. cysta': "Popliteal Cyst',"}, {'změny s': 'Mood alterations with depressive symptoms\',",'}, {'TEN 0': "WHODAS 2.0 12-item Version Proxy-administered - Concentrating for Ten Minutes',"}]


## Label specific Search
The access, where we have been handling all labels the same way have the issues. Because we cannot do label focus improvements. So all improvements we will do, we will only focus on the most common labels.

### New functions used there

In [46]:
def search_words_bottom(string, func):
    splitted_input = (string.split(" "))
    lenght = len(splitted_input)
    
    result_dict = {}
    for j in range(1, lenght + 1):
        splitted_dict={}
        for elem in splitted_input:
            splitted_dict[j] = False
                    
        for words in combinations(splitted_input, j):
            data = func(" ".join(words))

            if len(data) == 0:
                continue
            for j in words:
                splitted_dict[j] = True
            result_dict[" ".join(words)] = [(i[0], i[1]) for i in data]
                    
        splitted_input = [j for j, i in splitted_dict.items() if i]
        if len(splitted_input) == 0:
            break 
            
    return result_dict


def search_words_mistake_bottom(string, func1, func2, output_errors=False, limit=0.5):
    result_dict = {}
    for word in string.split(" "):
        n = 0
        search = func1(word)
        if len(search) == 0:
            while len(search) == 0 and n <= math.floor(len(word) * (limit)):
                n += 1
                search = func2(word, n)

        if output_errors:
            result_dict[(word, n)] = search
        else:
            result_dict[word] = search
        
    return result_dict

Function for searching in database

In [47]:
def medvik_without_descr_search(test, database):
    result = []
    for_loop_stopped = False
    
    for child in database:
        for subchild in child.iter("{http://www.loc.gov/MARC21/slim}datafield"):
            tag = int(subchild.attrib["tag"])
            if tag != 150 and tag != 450 and tag != 750:
                continue
                
            for subsubchild in subchild.iter("{http://www.loc.gov/MARC21/slim}subfield"):
                if subsubchild.text and test(subsubchild.text):
                    try:
                        code = [i for i in child.findall("{http://www.loc.gov/MARC21/slim}controlfield") if i.attrib["tag"] == "001" ][0].text
                        name = [i for i in child.findall("{http://www.loc.gov/MARC21/slim}datafield") if i.attrib["tag"] == "150" ][0][0].text
                        result.append((code, name))
                        for_loop_stopped = True
                        break
                    except IndexError:
                        for_loop_stopped = True
                        break
            if for_loop_stopped:
                for_loop_stopped = False
                break 
    
    return result


def medvik_combined_without_descr_search(string, database=offline_mshcz):
    string = string.lower()
    result = medvik_without_descr_search(lambda x: string == x.lower(), database)
    if len(result) == 0:
        result = medvik_without_descr_search(lambda x: (f" {string} " in x.lower()) or 
                                                       (f"{string} " == x.lower()[:(len(string) + 1)]) or
                                                       (f" {string}" == x.lower()[(-len(string) - 1):]),
                                                   database)
    if len(result) == 0:
        result = medvik_without_descr_search(lambda x: string in x.lower(), database)
    
    return result

Functions for searching with mistake:

In [48]:
def medvik_mistakes_search(string, mistakes, database):
    string = patternize(string)
    pattern = regex.compile(f"({string}){{e<={mistakes}}}", regex.IGNORECASE)
    return medvik_without_descr_search(lambda x: pattern.search(x) is not None, database)


def medvik_find_mistakes_search(dictionary, test=False, database=offline_mshcz):
    for w in dictionary:
        if dictionary[w] != []:
            continue

        n = 1
        r = []
        while r == [] and n <= len(w) // 2:
            r = medvik_mistakes_search(w, n, database)
            n += 1
            
        if r != [] and test:
            test_right.append(w)
            print("Find for:", w, ", mistakes:", n - 1)
        elif test:
            print("Don't find for:", w, "mistakes:", n - 1)
            
        dictionary[w] = r
    return dictionary

Help function for testing how we were good at finding mistake

In [49]:
def add_all_searches(string, dictionary):
    for w in string.split(" "):
        if w not in dictionary:
            dictionary[w] = []

    return dictionary


def drop_empty_searches(dictionary):
    for w in dictionary.copy():
        if dictionary[w] == []:
            dictionary.pop(w)

    return dictionary

Function for more precise asking GPT

In [50]:
def message_for_GPT_with_empty(string, li, find, context=None):
    if len(li) == 0:
        return ""
            
    if context is None:
        result = [f"Který z uvedených lékařských pojmů s jeho popisem nejlépe odpovídá pojmu: \"{string}\":\n"]
    else:
        result = [f"Který z uvedených lékařských pojmů s jeho popisem nejlépe odpovídá pojmu: \"{string}\" v kontextu:  \"{context}\":\n"]
        
    j = 1
    for i in li:
        result.append(f"{j}. {find(i[0])} (pojem: {i[1]})\n")
        j += 1

    result.append(f"{j}. Žádná z výše uvedených možností plně nevystihuje daný pojem.\n")
    result.append("Jako odpověď mi pošli pouze číslo odpovědi. Pokud to nebude žádná z možností, pak odpověz NONE. "
                  "Pokud to není lékařský pojem odpověz taky NONE.")
    
    return "".join(result)

Function for combining words

In [51]:
def print_long_searches_len_dep(db, limit=20):
    temp = db.apply(lambda x: [(i, len(x[i]), len(x[i]) / len(i)) for i in x if len(x[i]) > limit * len(i)])
    print(sorted(list(temp[temp.apply(lambda x: len(x) != 0)]), key=(lambda x: x[0][2])))


def drop_long_searches_len_dep(dictionary, limit=50):
    for key in dictionary.copy():
        if len(dictionary[key]) > limit * len(key):
            dictionary[key] = []
    return dictionary


def create_doubles(dictionary):
    result = {}
    for k_1 in dictionary:
        for k_2 in dictionary:
            if k_1 == k_2:
                continue

            r = list(set(dictionary[k_1]).intersection(
                     set(dictionary[k_2])))
            if len(r) > 0 and (k_2 + " " + k_1) not in result:
                result[k_1 + " " + k_2] = r

    return result


def combine_searches(dictionary):
    combined_results = {}
    result = create_doubles(dictionary)
    combined_results.update(result)
    while result != {}:
        result = create_doubles(result)
        result = {" ".join(set(k.split(" "))) :result[k] for k in result}
        combined_results.update(result)

    return combined_results

In [52]:
def is_negative_word(string):
    string = string.strip(" ")
    return string in ["0", "O", "bez", "ne", "neguje", "neg"] or " ne" in f" {string}"


def is_soft_negative_word(string):
    string = string.strip(" ")
    return string in ["0", "O", "bez", "ne", "neguje", "neg"] or " negat" in f" {string}"


def make_negation_table(db):
    db = db.copy()
    db["neg"] = db["text"].apply(lambda x: any(True for word in x.split(" ") if is_soft_negative_word(word)))
    db["text"] = db["text"].apply(lambda text: " ".join(
        [x.strip(":") for x in text.split(" ") if not(is_soft_negative_word(x))]))

    return db

In [53]:
def shortcuts(string):
    result = []
    for word in string.split(" "):
        if "." in word:
            result += word.strip(" .").split(".")
    return " ".join(result)


def make_shortcuts_table(db):
    db = db.copy()
    db["shortcuts"] = db["text"].apply(shortcuts)
    db["text"] = db["text"].apply(lambda x: " ".join([word for word in x.split(" ") if "." not in word]))
    return db


def shortcuts_medvik_without_descr_search(string, database=offline_mshcz):
    return medvik_without_descr_search(lambda x: (f" {string.lower()}" in x.lower()) or
                                                    (f"{string.lower()}" == x.lower()[:(len(string) + 1)]), database)


def shortcuts_medvik_mistakes_without_descr_search(string, mistakes, database):
    string = patternize(string)
    pattern_start = regex.compile(f"({string}){{e<={mistakes}}}", regex.IGNORECASE)
    pattern_in = regex.compile(f"( {string}){{e<={mistakes}}}", regex.IGNORECASE)
    test = (lambda x: pattern_start.search(x[:len(string) + mistakes]) is not None or
            pattern_in.search(x) is not None)
    return medvik_without_descr_search(test, database)

Interprets results

In [54]:
def interprete_explanation(dictionary):
    dictionary = {k: dictionary[k] for k in dictionary if dictionary[k] is not None}
    long_words = set()
    [k for k in dictionary if len(k.split(" ")) >= 2 and long_words.update(k.split(" "))]

    return {k : dictionary[k] for k in dictionary if k not in long_words}

In [55]:
def medvik_with_mistake_all_search_table(db, database, output_errors=False):
    db = db.copy()
    db["medvik_search"] = "N/A"
    
    for j in db.index:
        x = {}
        if db["text"][j] != "":
            x.update(search_words_mistake_bottom(
                db["text"][j], 
                lambda x: medvik_combined_without_descr_search(x, database),
                lambda x, y: medvik_mistakes_search(x, y, database=database),
                output_errors=output_errors,
                limit=0.25))
        
        if db["shortcuts"][j] != "": 
            x.update(search_words_mistake_bottom(
                db["shortcuts"][j],
                lambda x: shortcuts_medvik_without_descr_search(x, database),
                lambda x, y: shortcuts_medvik_mistakes_without_descr_search(x, y, database=database),
                output_errors=output_errors,
                limit=0.25))
            
        db.at[j, "medvik_search"] = x

    if output_errors:
        db["medvik_search_errors"] = db["medvik_search"].apply(lambda x: {k[0]:k[1] for k in x})
        db["medvik_search"] = db["medvik_search"].apply(lambda x: {k[0]:x[k] for k in x})

    return db


def medvik_search_all_read_csv(path):
    db = pd.read_csv(path)
    db.index = db["Unnamed: 0"]
    db.drop(["Unnamed: 0"], axis=1, inplace=True)
    db["medvik_search"] = db["medvik_search"].apply(from_string_to_dict)
    db["shortcuts"] = db["shortcuts"].fillna("")

    return db


def medvik_choose_GPT_table(db):
    db = db.copy()
    db["medvik_explanation"] = "N/A"
    
    for i in db.index:
        result = {}
        combined_level = -np.infty
        is_find = set()
        
        l = db["combined_medvik"][i]
        for text in sorted(l, key=lambda x: len(x.split(" ")), reverse=True):
            if combined_level > len(text.split(" ")) and all([w in is_find for w in text.split(" ")]):
                continue
            message = message_for_GPT(text, l[text], medvik_find_by_code, context=db["original_text"][i])
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
            if result[text] is not None:
                is_find.update(text.split(" "))
                combined_level = len(text.split(" "))

        l = db["medvik_search"][i]
        for text in l:
            if text in is_find:
                continue
            message = message_for_GPT(text, l[text], medvik_find_by_code, context=db["original_text"][i])
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
        db.at[i, "medvik_explanation"] = result
        
    return db


def medvik_explanation_all_read_csv(path):
    db = pd.read_csv(path)
    db.index = db["Unnamed: 0"]
    db.drop(["Unnamed: 0"], axis=1, inplace=True)
    db["medvik_search"] = db["medvik_search"].apply(from_string_to_dict)
    db["combined_medvik"] = db["combined_medvik"].apply(from_string_to_dict)
    db["medvik_explanation"] = db["medvik_explanation"].fillna("{}").apply(from_string_to_dict_to_tuple)
    db["shortcuts"] = db["shortcuts"].fillna("")

    return db

### Label procedura 01

In [56]:
improved_procedura = data_improved_punctuation[(data_improved_punctuation["label"] == "procedura")].copy()
print("Number of 'procedura' label:", len(improved_procedura))
procedura_test = improved_procedura.sample(35, random_state=34)
procedura_test.head(3)

Number of 'procedura' label: 596


,label,text,original_text
4791,procedura,genet. testování,genet. testování
1223,procedura,parciální mastektomie,parciální mastektomie
108,procedura,rekonstrukce,rekonstrukce


#### Linking and linking with mistake

In [57]:
procedury = []
for child in offline_mshcz:
    try:
        d = [i for i in child.findall("{http://www.loc.gov/MARC21/slim}datafield") if i.attrib["tag"] == "072" ][0][0].text
        if d is not None and d[0] == "E":
            procedury.append(child)
    except IndexError:
        pass                
print(len(procedury))

if os.path.isfile("saved_search/procedura_test.csv"):
    procedura_test = pd.read_csv("saved_search/procedura_test.csv")
    procedura_test.index = procedura_test["Unnamed: 0"]
    procedura_test.drop(["Unnamed: 0"], axis=1, inplace=True)
    procedura_test["mash_search"] = procedura_test["mash_search"].apply(from_string_to_dict)
    procedura_test["medvik_search"] = procedura_test["medvik_search"].apply(from_string_to_dict)

else:
    procedura_test = improved_procedura.sample(35, random_state=34)
    procedura_test["mash_search"] = "N/A"
    for j in procedura_test.index:
        procedura_test.at[j, "mash_search"] = search_words_bottom(procedura_test["text"][j], mash_search)
        
    procedura_test["medvik_search"] = "N/A"
    for j in procedura_test.index:
        procedura_test.at[j, "medvik_search"] = search_words_bottom(procedura_test["text"][j], lambda x: medvik_combined_without_descr_search(x, procedury))

    procedura_test.to_csv("saved_search/procedura_test.csv")

procedura_test.head(3)

3019


,label,text,original_text,about,mash_search,medvik_search
Unnamed: 0,,,,,,
4791,procedura,genet. testování,genet. testování,NaN,"{'genet.': [('C0325074', 'Genets'), ('C0032246...","{'testování': [('D000067108', 'DTC screening a..."
1223,procedura,parciální mastektomie,parciální mastektomie,NaN,"{'parciální': [('C0020608', 'Hypodontia'), ('C...","{'parciální': [('D010314', 'parciální trombopl..."
108,procedura,rekonstrukce,rekonstrukce,NaN,"{'rekonstrukce': [('C0085076', 'Mammaplasty'),...","{'rekonstrukce': [('D000070638', 'rekonstrukce..."


In [58]:
# for j in procedura_test.index:
#     procedura_test.at[j, "mash_search"] = add_all_searches(procedura_test["text"][j], procedura_test["mash_search"][j])
#     procedura_test.at[j, "medvik_search"] = add_all_searches(procedura_test["text"][j], procedura_test["medvik_search"][j])

# for j in procedura_test.index:
#     procedura_test.at[j, "medvik_search"] = medvik_find_mistakes_search(procedura_test["medvik_search"][j], test=False, database=procedury)

find_with_mistake = [('genet.', 1), ('nechemo', 1), ('adjuv.', 1), ('CT1', 1), ('epigastria', 3), ('disekci', 1),
            ('prsou', 1), ('mastektomii', 1), ('vpravo', 1), ('vyšších', 2), ('etáží', 1), ('parc.', 1), ('SNb', 1),
            ('l.dx', 2), ('st.p.NACT', 4), ('vpravo', 1), ('BRCA1', 2), ('BRCA2', 2), ('PALB2', 2), ('CHEK2', 2), ('TP53', 2),
            ('dalších', 2), ('parc.', 1), ('SNB', 1), ('adjuvantním',              1), ('simplexní', 2), ('vlevo', 1), ('parc.', 1),
            ('mastektomii', 1), ('SNB', 1), ('bilat.', 1), ('SNB', 1), ('vlevo', 1), ('mastectomia', 2), ('simplex', 1),
            ('mammae', 1), ('l.', 1), ('etáži', 1)]
print(find_with_mistake)

[('genet.', 1), ('nechemo', 1), ('adjuv.', 1), ('CT1', 1), ('epigastria', 3), ('disekci', 1), ('prsou', 1), ('mastektomii', 1), ('vpravo', 1), ('vyšších', 2), ('etáží', 1), ('parc.', 1), ('SNb', 1), ('l.dx', 2), ('st.p.NACT', 4), ('vpravo', 1), ('BRCA1', 2), ('BRCA2', 2), ('PALB2', 2), ('CHEK2', 2), ('TP53', 2), ('dalších', 2), ('parc.', 1), ('SNB', 1), ('adjuvantním', 1), ('simplexní', 2), ('vlevo', 1), ('parc.', 1), ('mastektomii', 1), ('SNB', 1), ('bilat.', 1), ('SNB', 1), ('vlevo', 1), ('mastectomia', 2), ('simplex', 1), ('mammae', 1), ('l.', 1), ('etáži', 1)]


#### Combining words

In [59]:
print("Lenght_dependent:")
print_long_searches_len_dep(procedura_test["medvik_search"], limit=40)
print("\nLength_independent:")
print_long_searches(procedura_test["medvik_search"], limit=40)

Lenght_dependent:
[[('TM', 86, 43.0)], [('do', 237, 118.5)], [('s', 129, 129.0)], [('a', 405, 405.0)], [('a', 405, 405.0)], [('a', 405, 405.0)], [('ME', 1076, 538.0)], [('ME', 1076, 538.0)], [('ME', 1076, 538.0)], [('ME', 1076, 538.0)], [('i', 2725, 2725.0), ('s', 129, 129.0)]]

Length_independent:
[[('NAC', 66)], [('HT', 70)], [('TM', 86)], [('s', 129)], [('do', 237)], [('a', 405)], [('a', 405)], [('a', 405)], [('ME', 1076)], [('ME', 1076)], [('ME', 1076)], [('ME', 1076)], [('i', 2725), ('s', 129)]]


We can notice that there is limit, where we can remove single letters or common words, which don't have medical meaning. The limit is around 50. This filtered words we will use to create doubles (or more), where we will have probably real medical terms.

In [60]:
procedura_test["medvik_search"] = procedura_test["medvik_search"].apply(lambda x: drop_long_searches_len_dep(x, limit=40))

In [61]:
temp = procedura_test["medvik_search"].apply(lambda x: {k : x[k] for k in x if not len(k.split(" ")) > 1}).apply(combine_searches)
temp[temp != {}]

Series([], Name: medvik_search, dtype: object)

In [62]:
procedura_test["combined_medvik"] = procedura_test["medvik_search"].apply(
    lambda x: {k : x[k] for k in x if not len(k.split(" ")) > 1}).apply(combine_searches)

procedura_test["medvik_search"] = procedura_test["medvik_search"].apply(lambda x: {k : x[k] for k in x if not len(k.split(" ")) > 1})
procedura_test.head(3)

,label,text,original_text,about,mash_search,medvik_search,combined_medvik
Unnamed: 0,,,,,,,
4791,procedura,genet. testování,genet. testování,NaN,"{'genet.': [('C0325074', 'Genets'), ('C0032246...","{'testování': [('D000067108', 'DTC screening a...",{}
1223,procedura,parciální mastektomie,parciální mastektomie,NaN,"{'parciální': [('C0020608', 'Hypodontia'), ('C...","{'parciální': [('D010314', 'parciální trombopl...",{}
108,procedura,rekonstrukce,rekonstrukce,NaN,"{'rekonstrukce': [('C0085076', 'Mammaplasty'),...","{'rekonstrukce': [('D000070638', 'rekonstrukce...",{}


#### Choosing best match

In [63]:
print_long_searches(procedura_test["medvik_search"], limit=10)

[[('rekonstrukce', 12)], [('se', 13)], [('se', 13)], [('RAME', 22)], [('amb', 25)], [('BT', 26)], [('PM', 33)], [('sono', 36), ('axil', 20)], [('NAC', 66)], [('HT', 70)]]


In [64]:
procedura_test["medvik_search"] = procedura_test["medvik_search"].apply(lambda x: drop_long_searches(x, 40))
procedura_test["medvik_search"] = procedura_test["medvik_search"].apply(drop_empty_searches)
procedura_test["mash_search"] = procedura_test["mash_search"].apply(drop_empty_searches)

In [65]:
if os.path.isfile("saved_search/procedura_test_explanation.csv"):
    procedura_test = pd.read_csv("saved_search/procedura_test_explanation.csv")
    procedura_test.index = procedura_test["Unnamed: 0"]
    procedura_test.drop(["Unnamed: 0"], axis=1, inplace=True)
    procedura_test["mash_search"] = procedura_test["mash_search"].apply(from_string_to_dict)
    procedura_test["medvik_search"] = procedura_test["medvik_search"].apply(from_string_to_dict)
    procedura_test["combined_medvik"] = procedura_test["combined_medvik"].apply(from_string_to_dict)
    procedura_test["mash_explanation"] = procedura_test["mash_explanation"].fillna("{}").apply(from_string_to_dict_to_tuple)
    procedura_test["medvik_explanation"] = procedura_test["medvik_explanation"].fillna("{}").apply(from_string_to_dict_to_tuple)
    
else:
    procedura_test["medvik_explanation"] = "N/A"
    for i in procedura_test.index:
        l = procedura_test["medvik_search"][i]
        result = {}
        for text in l:
            message = message_for_GPT(text, l[text], medvik_find_by_code)
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
        procedura_test.at[i, "medvik_explanation"] = result
        
    procedura_test["mash_explanation"] = "N/A"
    for i in procedura_test.index:
        l = procedura_test["mash_search"][i]
        result = {}
        for text in l:
            message = message_for_GPT(text, l[text], medvik_find_by_code)
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
            
        l = procedura_test["combined_medvik"][i]
        for text in l:
            message = message_for_GPT(text, l[text], medvik_find_by_code)
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
        
        procedura_test.at[i, "mash_explanation"] = result
    procedura_test.to_csv("saved_search/procedura_test_explanation.csv")

procedura_test.head(3)

,label,text,original_text,about,mash_search,medvik_search,combined_medvik,medvik_explanation,mash_explanation
Unnamed: 0,,,,,,,,,
4791,procedura,genet. testování,genet. testování,NaN,"{'genet.': [('C0325074', 'Genets'), ('C0032246...","{'testování': [('D000067108', 'DTC screening a...",{},"{'testování': ('D000086742', 'testování na COV...","{'genet.': ('C0314603', 'Genetic',', ''), 'tes..."
1223,procedura,parciální mastektomie,parciální mastektomie,NaN,"{'parciální': [('C0020608', 'Hypodontia'), ('C...","{'parciální': [('D010314', 'parciální trombopl...",{},"{'parciální': None, 'mastektomie': ('D015409',...","{'parciální': ('C0080024', 'Piebaldism',', '')..."
108,procedura,rekonstrukce,rekonstrukce,NaN,"{'rekonstrukce': [('C0085076', 'Mammaplasty'),...","{'rekonstrukce': [('D000070638', 'rekonstrukce...",{},"{'rekonstrukce': ('D001178', 'artroplastika', ...","{'rekonstrukce': ('C0195196', 'Reconstruction ..."


#### Results

In [66]:
asign = []
for j in procedura_test.sample(35, random_state=42).index:   
    if procedura_test["medvik_explanation"][j] is not None:
        x = procedura_test["medvik_explanation"][j]
        asign.append((procedura_test["text"][j], "Medvik", "{}".format([(e, x[e]) for e in x])))
    if procedura_test["mash_explanation"][j] is not None:
        x = procedura_test["mash_explanation"][j]
        asign.append((procedura_test["text"][j], "Mash", "{}".format([(e, x[e]) for e in x])))
    if procedura_test["mash_explanation"][j] is None and procedura_test["medvik_explanation"][j] is None:
        asign.append((procedura_test["text"][j], "None", "Empty"))

i = 0
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = ""
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)

In [67]:
print("Number of examples: {}".format(len(procedura_test)))

print("Number of empty Linking for Medvik_combined_search: {}".format(procedura_test["medvik_search"].apply(lambda x: len(x) == 0).sum()))
print("Number of empty Linking for Mash_search: {}".format(procedura_test["mash_search"].apply(lambda x: len(x) == 0).sum()))

Number of examples: 35
Number of empty Linking for Medvik_combined_search: 4
Number of empty Linking for Mash_search: 3


In [68]:
print("Not assigned medvik:", len([i for i in asign if i[1] == "Medvik" and i[3] == ""]))
print("Mistakes from medvik:", len([i for i in asign if i[3] == 'Wrong' and i[1] == "Medvik"]))
print("Partially right from medvik:", len([i for i in asign if i[3] == 'Partially' and i[1] == "Medvik"]))
print("Right from medvik:", len([i for i in asign if i[3] == 'Right' and i[1] == "Medvik"]))

print("\nNot assigned mash:", len([i for i in asign if i[1] == "Mash" and i[3] == ""]))
print("Mistakes from mash:", len([i for i in asign if i[3] == 'Wrong' and i[1] == "Mash"]))
print("Partially right from mash:", len([i for i in asign if i[3] == 'Partially' and i[1] == "Mash"]))
print("Right from mash:", len([i for i in asign if i[3] == 'Right' and i[1] == "Mash"]))

Not assigned medvik: 5
Mistakes from medvik: 10
Partially right from medvik: 17
Right from medvik: 3

Not assigned mash: 3
Mistakes from mash: 21
Partially right from mash: 9
Right from mash: 2


- Medvik Right: 9 %
- Medvik Partially: 57 %

- Mash Right: 6 %
- Mash Partially: 31 %

Problems
- shorcuts
- too afraid to say don't know
- Lots of parts of body, don't recognize them.

Good
- All combined words were right

### Label procedura 02

In [69]:
procedura_test2 = improved_procedura.sample(35, random_state=123)       
procedura_test2.head(3)

,label,text,original_text
2769,procedura,z core biopsie,z core biopsie
45,procedura,krve ve stolici hemokult test,krve ve stolici - hemokult test
2517,procedura,adjuvance,adjuvance


#### Linking with mistakes

In [70]:
procedury_anathomy = []
for child in offline_mshcz:
    try:
        d = [i for i in child.findall("{http://www.loc.gov/MARC21/slim}datafield") if i.attrib["tag"] == "072" ][0][0].text
        if d is not None and d[0] in ["E", "A"]:
            procedury_anathomy.append(child)
    except IndexError:
        pass                
print(len(procedury_anathomy))

4915


In [71]:
if os.path.isfile("saved_search/procedura_test2.csv"):
    procedura_test2 = pd.read_csv("saved_search/procedura_test2.csv")
    procedura_test2.index = procedura_test2["Unnamed: 0"]
    procedura_test2.drop(["Unnamed: 0"], axis=1, inplace=True)
    procedura_test2["medvik_search"] = procedura_test2["medvik_search"].apply(from_string_to_dict)

else:
    procedura_test2["medvik_search"] = "N/A"
    for j in procedura_test2.index:
        procedura_test2.at[j, "medvik_search"] = search_words_mistake_bottom(
            procedura_test2["text"][j], lambda x: medvik_combined_without_descr_search(x, procedury_anathomy), 
                                        lambda x, y: medvik_mistakes_search(x, y, database=procedury_anathomy))
        
    procedura_test2.to_csv("saved_search/procedura_test2.csv")

procedura_test2.head(3)

,label,text,original_text,about,medvik_search
Unnamed: 0,,,,,
2769,procedura,z core biopsie,z core biopsie,NaN,"{'z': [('D000071936', 'výtok z bradavky'), ('D..."
45,procedura,krve ve stolici hemokult test,krve ve stolici - hemokult test,NaN,"{'krve': [('D000071997', 'kultivační vyšetření..."
2517,procedura,adjuvance,adjuvance,NaN,"{'adjuvance': [('D017358', 'skelet buněčné stě..."


In [72]:
print("Lenght_dependent:")
print_long_searches_len_dep(procedura_test2["medvik_search"], limit=30)
print("\nLength_independent:")
print_long_searches(procedura_test2["medvik_search"], limit=50)

procedura_test2["medvik_search"] = procedura_test2["medvik_search"].apply(lambda x: drop_long_searches_len_dep(x, 30))

Lenght_dependent:
[[('test', 129, 32.25)], [('sin.', 136, 34.0), ('I', 4470, 4470.0)], [('z', 42, 42.0)], [('z', 42, 42.0), ('v', 136, 136.0)], [('oper', 173, 43.25), ('bilt', 295, 73.75)], [('sin', 135, 45.0)], [('SNB', 722, 240.66666666666666), ('sin.', 136, 34.0), ('No', 77, 38.5), ('I', 4470, 4470.0)], [('SNB', 722, 240.66666666666666)], [('op.', 1378, 459.3333333333333)], [('USG', 2178, 726.0), ('a', 609, 609.0)], [('USG', 2178, 726.0)], [('RT', 1472, 736.0)], [('RT', 1472, 736.0), ('na', 149, 74.5)], [('ME', 1805, 902.5), ('s', 140, 140.0)], [('ko', 2050, 1025.0), ('dif', 152, 50.666666666666664)]]

Length_independent:
[[('krve', 61), ('test', 129)], [('terapie', 84)], [('Cov', 86)], [('sin', 135)], [('sin.', 136), ('I', 4470)], [('v', 136)], [('oper', 173), ('bilt', 295)], [('SNB', 722), ('sin.', 136), ('No', 77), ('I', 4470)], [('SNB', 722)], [('op.', 1378)], [('RT', 1472)], [('RT', 1472), ('na', 149)], [('ME', 1805), ('s', 140)], [('ko', 2050), ('dif', 152)], [('USG', 2178), (

In [73]:
temp = procedura_test2["medvik_search"].apply(combine_searches)
temp[temp != {}]

Unnamed: 0
45      {'krve stolici': [('D001794', 'krevní tlak')],...
2520    {'oblast mammy': [('D007034', 'zadní hypotalam...
356     {'mastectomia partialis': [('D015412', 'segmen...
1046    {'adjuvantní chemobioterapi': [('D059248', 'ch...
4692    {'odběry Cov': [('D027724', 'asistovaná reprod...
468     {'konizaci čípku': [('D019092', 'konizace dělo...
30      {'radikální mastektomie': [('D015409', 'radiká...
1548    {'biopsie uzliny': [('D021701', 'biopsie senti...
Name: medvik_search, dtype: object

In [74]:
procedura_test2["combined_medvik"] = procedura_test2["medvik_search"].apply(combine_searches)
procedura_test2.head(3)

,label,text,original_text,about,medvik_search,combined_medvik
Unnamed: 0,,,,,,
2769,procedura,z core biopsie,z core biopsie,NaN,"{'z': [], 'core': [('D000088482', 'denzní váčk...",{}
45,procedura,krve ve stolici hemokult test,krve ve stolici - hemokult test,NaN,"{'krve': [('D000071997', 'kultivační vyšetření...","{'krve stolici': [('D001794', 'krevní tlak')],..."
2517,procedura,adjuvance,adjuvance,NaN,"{'adjuvance': [('D017358', 'skelet buněčné stě...",{}


#### Choosing best match

In [75]:
print_long_searches(procedura_test2["medvik_search"], limit=10)

[[('adjuv', 11), ('oblast', 41), ('mammy', 46)], [('mastectomia', 11), ('partialis', 27)], [('adjuv', 11)], [('doppler', 11)], [('disekcí', 12)], [('biopsie', 15)], [('biopsie', 15)], [('biopsie', 15), ('uzliny', 11), ('axille', 33)], [('bez', 16)], [('carpl', 18)], [('2', 18)], [('se', 19)], [('trombo', 27)], [('chemobioterapi', 29)], [('chemobioterapie', 29)], [('odběry', 35), ('Cov', 86)], [('prsou', 36), ('axill', 32)], [('prsou', 36)], [('krve', 61), ('ve', 19)], [('terapie', 84)]]


In [76]:
procedura_test2["medvik_search"] = procedura_test2["medvik_search"].apply(lambda x: drop_long_searches(x, 80))
procedura_test2["medvik_search"] = procedura_test2["medvik_search"].apply(drop_empty_searches)

In [77]:
if os.path.isfile("saved_search/procedura_test2_explanation.csv"):
    procedura_test2 = pd.read_csv("saved_search/procedura_test2_explanation.csv")
    procedura_test2.index = procedura_test2["Unnamed: 0"]
    procedura_test2.drop(["Unnamed: 0"], axis=1, inplace=True)
    procedura_test2["medvik_search"] = procedura_test2["medvik_search"].apply(from_string_to_dict)
    procedura_test2["combined_medvik"] = procedura_test2["combined_medvik"].apply(from_string_to_dict)
    procedura_test2["medvik_explanation"] = procedura_test2["medvik_explanation"].fillna("{}").apply(from_string_to_dict_to_tuple)
    
else:
    procedura_test2["medvik_explanation"] = "N/A"
    for i in procedura_test2.index:
        l = procedura_test2["medvik_search"][i]
        result = {}
        for text in l:
            message = message_for_GPT(text, l[text], medvik_find_by_code)
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
        procedura_test2.at[i, "medvik_explanation"] = result
        
        l = procedura_test2["combined_medvik"][i]
        for text in l:
            message = message_for_GPT(text, l[text], medvik_find_by_code)
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
            
        procedura_test2.at[i, "medvik_explanation"] = result
    procedura_test2.to_csv("saved_search/procedura_test2_explanation.csv")

procedura_test2.head(3)

,label,text,original_text,about,medvik_search,combined_medvik,medvik_explanation
Unnamed: 0,,,,,,,
2769,procedura,z core biopsie,z core biopsie,NaN,"{'core': [('D000088482', 'denzní váčky'), ('D0...",{},"{'core': None, 'biopsie': ('D001706', 'biopsie..."
45,procedura,krve ve stolici hemokult test,krve ve stolici - hemokult test,NaN,"{'ve': [('D000067491', 'téměř pochybení ve zdr...","{'krve stolici': [('D001794', 'krevní tlak')],...","{'krve': ('D000094902', 'krevní bankovnictví',..."
2517,procedura,adjuvance,adjuvance,NaN,"{'adjuvance': [('D017358', 'skelet buněčné stě...",{},{'adjuvance': None}


#### Results

In [78]:
asign = []
for j in procedura_test2.sample(35, random_state=42).index:   
    if procedura_test2["medvik_explanation"][j] is not None:
        x = procedura_test2["medvik_explanation"][j]
        asign.append((procedura_test2["text"][j], "Medvik", "{}".format([(e, x[e]) for e in x])))
    else:
        asign.append((procedura_test2["text"][j], "None", "Empty"))

i = 0
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Partially"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Wrong"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)
i += 1
assigment = "Right"
asign[i] = (asign[i][0], asign[i][1], asign[i][2], assigment)

In [79]:
print("Number of examples: {}".format(len(procedura_test2)))
print("Number of empty Linking for Medvik_combined_search: {}".format(procedura_test2["medvik_search"].apply(lambda x: len(x) == 0).sum()))

Number of examples: 35
Number of empty Linking for Medvik_combined_search: 0


In [80]:
print("Not assigned medvik:", len([i for i in asign if i[1] == "Medvik" and i[3] == ""]))
print("Mistakes from medvik:", len([i for i in asign if i[3] == 'Wrong' and i[1] == "Medvik"]))
print("Partially right from medvik:", len([i for i in asign if i[3] == 'Partially' and i[1] == "Medvik"]))
print("Right from medvik:", len([i for i in asign if i[3] == 'Right' and i[1] == "Medvik"]))

Not assigned medvik: 0
Mistakes from medvik: 13
Partially right from medvik: 10
Right from medvik: 12


Right: 34 %

Partially or Right: 63 %

### Label procedura 03 - Solving shortcuts & Limit for mistake

In [81]:
procedura_test = improved_procedura.sample(35, random_state=9)       
procedura_test.head(3)

,label,text,original_text
672,procedura,SNB,SNB
875,procedura,lokalizace TU a uzliny vlevo jod. zrno,lokalizace TU a uzliny vlevo jod. zrno
2952,procedura,RT CHT,"RT, CHT"


In [82]:
procedura_test[procedura_test["text"].apply(lambda x: "." in x)]

,label,text,original_text
875,procedura,lokalizace TU a uzliny vlevo jod. zrno,lokalizace TU a uzliny vlevo jod. zrno
625,procedura,parc. ME s disekcí axily,parc. ME s disekcí axily
3826,procedura,adjuvantní radioterapie na oblast mammy l.sin....,adjuvantní radioterapie na oblast mammy l.sin....
5350,procedura,2.čtení,2.čtení
202,procedura,SNB axillae l.dx,SNB axillae l.dx
1944,procedura,stomatolog. výkonům,stomatolog. výkonům
1335,procedura,stp. mastectomia partialis l. sin,stp. mastectomia partialis l. sin


In [83]:
procedura_test["shortcuts"] = procedura_test["text"].apply(shortcuts)
procedura_test["text"] = procedura_test["text"].apply(lambda x: " ".join([word for word in x.split(" ") if "." not in word]))

In [84]:
if os.path.isfile("saved_search/procedura_test3.csv"):
    procedura_test = pd.read_csv("saved_search/procedura_test3.csv")
    procedura_test.index = procedura_test["Unnamed: 0"]
    procedura_test.drop(["Unnamed: 0"], axis=1, inplace=True)
    procedura_test["medvik_search"] = procedura_test["medvik_search"].apply(from_string_to_dict)
    procedura_test["medvik_search_errors"] = procedura_test["medvik_search_errors"].apply(from_string_to_int_dict)

else:
    procedura_test["medvik_search"] = "N/A"
    for j in procedura_test.index:
        x = {}
        if procedura_test["text"][j] != "":
            x.update(search_words_mistake_bottom(procedura_test["text"][j],
                                              lambda x: medvik_combined_without_descr_search(x, procedury_anathomy), 
                                              lambda x, y: medvik_mistakes_search(x, y, database=procedury_anathomy),
                                              output_errors=True))
        if procedura_test["shortcuts"][j] != "": 
            x.update(search_words_mistake_bottom(procedura_test["shortcuts"][j],
                                              lambda x: shortcuts_medvik_without_descr_search(x, procedury_anathomy), 
                                              lambda x, y: shortcuts_medvik_mistakes_without_descr_search(x, y, database=procedury_anathomy),
                                              output_errors=True))
        procedura_test.at[j, "medvik_search"] = x

    procedura_test["medvik_search_errors"] = procedura_test["medvik_search"].apply(lambda x: {k[0]:k[1] for k in x})
    procedura_test["medvik_search"] = procedura_test["medvik_search"].apply(lambda x: {k[0]:x[k] for k in x})
    procedura_test.to_csv("saved_search/procedura_test3.csv")
    
procedura_test.head(3)

,label,text,original_text,about,shortcuts,medvik_search,medvik_search_errors
Unnamed: 0,,,,,,,
672,procedura,SNB,SNB,NaN,NaN,"{'SNB': [('D000067029', 'fyzický vzhled těla')...",{'SNB': 1}
875,procedura,lokalizace TU a uzliny vlevo zrno,lokalizace TU a uzliny vlevo jod. zrno,NaN,jod,"{'lokalizace': [('D001931', 'mapování mozku')]...","{'lokalizace': 0, 'TU': 0, 'a': 0, 'uzliny': 0..."
2952,procedura,RT CHT,"RT, CHT",NaN,NaN,"{'RT': [('D000018', 'slez'), ('D000025', 'potr...","{'RT': 0, 'CHT': 0}"


In [85]:
print_long_searches_len_dep(procedura_test["medvik_search"], limit=20)
procedura_test["medvik_search"] = procedura_test["medvik_search"].apply(lambda x: drop_long_searches_len_dep(x, 30))

[[('sin', 84, 28.0), ('stp', 1319, 439.6666666666667), ('l', 664, 664.0)], [('2', 36, 36.0)], [('na', 97, 48.5), ('a', 110, 110.0), ('l', 664, 664.0)], [('na', 97, 48.5)], [('na', 97, 48.5)], [('klipu', 281, 56.2)], [('SNB', 349, 116.33333333333333)], [('SNB', 349, 116.33333333333333), ('l', 664, 664.0)], [('PHK', 715, 238.33333333333334)], [('RT', 525, 262.5)], [('RT', 525, 262.5)], [('TU', 585, 292.5), ('a', 110, 110.0)], [('USG', 1093, 364.3333333333333)], [('USG', 1093, 364.3333333333333), ('a', 110, 110.0)], [('USG', 1093, 364.3333333333333)], [('ME', 1185, 592.5), ('s', 80, 80.0)]]


In [86]:
procedura_test["combined_medvik"] = procedura_test["medvik_search"].apply(combine_searches)
temp = procedura_test["medvik_search"].apply(combine_searches)
temp[temp != {}]

Unnamed: 0
517     {'gynekologické vyšetření': [('D058869', 'gyne...
449     {'komplexní léčba': [('D000073319', 'léčba obe...
3826    {'oblast mammy': [('D007034', 'zadní hypotalam...
5491    {'vyšetření chirurgické': [('D004384', 'duoden...
448     {'oblast mammy': [('D007034', 'zadní hypotalam...
1046    {'adjuvantní chemobioterapi': [('D017024', 'ad...
1335    {'mastectomia partialis': [('D015412', 'segmen...
Name: medvik_search, dtype: object

#### Choosing best match

In [87]:
print_long_searches(procedura_test["medvik_search"], limit=30)

[[('vyšetření', 47)], [('vyšetření', 47), ('chirurgické', 72)], [('vyšetření', 47)], [('reg', 50), ('lymf', 34)], [('léčba', 53)], [('výkonům', 64)], [('klip', 76)], [('sin', 84)]]


In [88]:
procedura_test["medvik_search"] = procedura_test["medvik_search"].apply(lambda x: drop_long_searches(x, 80))
procedura_test["medvik_search"] = procedura_test["medvik_search"].apply(drop_empty_searches)

if os.path.isfile("saved_search/procedura_test3_explanation.csv"):
    procedura_test = pd.read_csv("saved_search/procedura_test3_explanation.csv")
    procedura_test.index = procedura_test["Unnamed: 0"]
    procedura_test.drop(["Unnamed: 0"], axis=1, inplace=True)
    procedura_test["medvik_search"] = procedura_test["medvik_search"].apply(from_string_to_dict)
    procedura_test["combined_medvik"] = procedura_test["combined_medvik"].apply(from_string_to_dict)
    procedura_test["medvik_explanation"] = procedura_test["medvik_explanation"].fillna("{}").apply(from_string_to_dict_to_tuple)
    procedura_test["medvik_search_errors"] = procedura_test["medvik_search_errors"].apply(from_string_to_int_dict)

else:
    procedura_test["medvik_explanation"] = "N/A"
    for i in procedura_test.index:
        l = procedura_test["medvik_search"][i]
        result = {}
        for text in l:
            message = message_for_GPT(text, l[text], medvik_find_by_code)
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
        procedura_test.at[i, "medvik_explanation"] = result
        
        l = procedura_test["combined_medvik"][i]
        for text in l:
            message = message_for_GPT(text, l[text], medvik_find_by_code)
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
            
        procedura_test.at[i, "medvik_explanation"] = result
    procedura_test.to_csv("saved_search/procedura_test3_explanation.csv")

procedura_test["medvik_explanation"] = procedura_test["medvik_explanation"].apply(interprete_explanation)
procedura_test.head(3)

,label,text,original_text,about,shortcuts,medvik_search,medvik_search_errors,combined_medvik,medvik_explanation
Unnamed: 0,,,,,,,,,
672,procedura,SNB,SNB,NaN,NaN,{},{'SNB': 1},{},{}
875,procedura,lokalizace TU a uzliny vlevo zrno,lokalizace TU a uzliny vlevo jod. zrno,NaN,jod,"{'lokalizace': [('D001931', 'mapování mozku')]...","{'lokalizace': 0, 'TU': 0, 'a': 0, 'uzliny': 0...",{},"{'lokalizace': ('D001931', 'mapování mozku', '..."
2952,procedura,RT CHT,"RT, CHT",NaN,NaN,"{'CHT': [('D000068618', 'výběrové šlechtění'),...","{'RT': 0, 'CHT': 0}",{},{}


#### Results

In [89]:
procedura_test["asign"] = "N/A"

i = 0
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Empty"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Partially"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Empty"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Partially"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Partially"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Partially"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Empty"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Partially"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Empty"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Empty"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Partially"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Partially"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Partially"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Partially"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"


# print(procedura_test["original_text"].iloc[i])
# print(procedura_test["medvik_explanation"].iloc[i])
# print()
# print(procedura_test["medvik_search"].iloc[i])

In [90]:
procedura_test["asign"].value_counts()

asign
Right        14
Partially     9
Wrong         7
Empty         5
Name: count, dtype: int64

Right 40 % | Partially 66 % | Empty 14 %
- Bad filter
- Bad with established abbreviations (written in upper letters)

#### Testing Limit

In [91]:
print("RIGHT")
for i in procedura_test[procedura_test["asign"] == "Right"].sample(frac=1, random_state=32).index:
    x = procedura_test["medvik_search_errors"][i]
    print(procedura_test["original_text"][i])
    print(procedura_test["medvik_explanation"][i])
        
    temp2 = {k: (x[k], len(k), round(x[k]/len(k), 3)) for k in x if x[k] != 0}
    if len(temp2) != 0:
        print(temp2)
    else:
        print("Zero errors")
    print()

print("WRONG")
for i in procedura_test[procedura_test["asign"] != "Right"].sample(frac=1, random_state=32).index:
    x = procedura_test["medvik_search_errors"][i]
    print(procedura_test["original_text"][i])
    print(procedura_test["medvik_explanation"][i])
        
    temp2 = {k: (x[k], len(k), round(x[k]/len(k), 3)) for k in x if x[k] != 0}
    if len(temp2) != 0:
        print(temp2)
    else:
        print("Zero errors")
    print()

RIGHT
gynekologické vyšetření
{'gynekologické vyšetření': ('D058869', 'gynekologické vyšetření', 'Prohlédnutí a PALPACE ženských prsou, břicha a POHLAVNÍCH ORGÁNŮ a rovněž získání informací gynekologickou anamnézou.')}
Zero errors

PET/CT
{'PET': ('D000072078', 'PET/CT', 'Kombinovaná diagnostická technika, která kombinuje pozitronovou emisní tomografii a rentgenovou výpočetní tomografií. Získané obrazy obsahují jak poměrně podrobnou informaci o anatomické stavbě, tak i informaci o metabolické aktivitě tkání.'), 'CT': ('D014057', 'počítačová rentgenová tomografie', 'Tomografie využívající k rekonstrukci obrazu přenos rentgenových paprsků a počítačový algoritmus.')}
Zero errors

očkování vakcinou
{'očkování': ('D014611', 'vakcinace', 'Podávání vakcín ke stimulaci imunitní odpovědi hostitele. To zahrnuje každý přípravek určený k aktivní imunologické profylaxi.'), 'vakcinou': ('D016233', 'aktivní imunoterapie', 'Active immunization where vaccine is administered for therapeutic or preventiv

Limit 0.25 percentage of mistakes will filter more trash and let go maximum of results results.

In [92]:
for i in procedura_test[procedura_test["asign"] != "Right"].sample(frac=0.5, random_state=16).index:
    print(procedura_test["original_text"][i])
    print(procedura_test["medvik_explanation"][i])

    y = procedura_test["medvik_search"][i]
    {k for k in y if print("\n", k, ":", y[k][:20], end="")}
    print()
                           
    print(procedura_test["medvik_search_errors"][i])
    print("\n")

parc. ME s disekcí axily
{'disekcí': ('D004210', 'disekce', 'Oddělování částí těla či tkání podle jejich přirozených ohraničení. D. při operaci, d. při anatomické pitvě. (Cit. Velký lékařský slovník, http://lekarske.slovniky.cz/ 2014'), 'parc': ('D010314', 'parciální tromboplastinový čas', 'Doba potřebná k tomu, aby došlo ke sražení plazmy jako reakce na smíchání krevní plazmy s fosfolipidovou destičkovou náhradou (např. surové cefaliny, sójové fosfatidy). Jedná se o test vnitřní cesty (faktory VIII, IX, XI a XII) a společné cesty (fibrinogen, protrombin, faktory V a X) krevní koagulace. Slouží jako základní test srážení krve a používá se též k monitorování heparinové antikoagulační terapie.')}

 disekcí : [('D000069916', 'endoskopická mukózní resekce'), ('D004210', 'disekce'), ('D008197', 'lymfadenektomie'), ('D037981', 'krční disekce'), ('D042282', 'mikrodisekce'), ('D061346', 'laserová záchytná mikrodisekce')]
 axily : [('D000540', 'augmentace alveolárního výběžku'), ('D063166', 'os

Lots of mistakes is there because GPT didn't know the context.

### Label Procedura 04 - Improving message for GPT

#### Test Init

In [93]:
procedura_test = improved_procedura.sample(35, random_state=8)
procedura_test = make_shortcuts_table(procedura_test)

In [94]:
if os.path.isfile("saved_search/procedura_test4.csv"):
    procedura_test = medvik_search_all_read_csv("saved_search/procedura_test4.csv")
    procedura_test["medvik_search_errors"] = procedura_test["medvik_search_errors"].apply(from_string_to_int_dict)
else:
    procedura_test = medvik_with_mistake_all_search_table(procedura_test, procedury_anathomy, output_errors=True)
    procedura_test.to_csv("saved_search/procedura_test4.csv")

procedura_test["medvik_search"] = procedura_test["medvik_search"].apply(lambda x: drop_long_searches_len_dep(x, 30))
procedura_test["combined_medvik"] = procedura_test["medvik_search"].apply(combine_searches)
procedura_test["medvik_search"] = procedura_test["medvik_search"].apply(lambda x: drop_long_searches(x, 80))
procedura_test["medvik_search"] = procedura_test["medvik_search"].apply(drop_empty_searches)

procedura_test.head(3)

,label,text,original_text,about,shortcuts,medvik_search,medvik_search_errors,combined_medvik
Unnamed: 0,,,,,,,,
2350,procedura,adjuvantní radioterapii na oblast mammy lymfatik,"st.p. adjuvantní radioterapii, na oblast mammy...",NaN,st p l sin sv,"{'adjuvantní': [('D017024', 'adjuvantní chemot...","{'adjuvantní': 0, 'radioterapii': 0, 'na': 0, ...","{'oblast mammy': [('D007034', 'zadní hypotalam..."
2674,procedura,verifikováno MTS v axile,verifikováno MTS v axile,NaN,,"{'axile': [('D000080886', 'spatium parapharyng...","{'verifikováno': 4, 'MTS': 1, 'v': 0, 'axile': 1}",{}
2517,procedura,adjuvance,adjuvance,NaN,,"{'adjuvance': [('D017024', 'adjuvantní chemote...",{'adjuvance': 2},{}


#### Testing different messages to choosing by GPT
- The first message is previously used for method for linking.
- The second message has the same form as previous one, but it have additional option: if None of these.
- The last message has also the same form, but it adds also the content, where the term is.

In [95]:
if os.path.isfile("saved_search/procedura_test4_explanation_without_empty.csv"):
    procedura_without_empty = medvik_explanation_all_read_csv("saved_search/procedura_test4_explanation_without_empty.csv")
    procedura_without_empty["medvik_search_errors"] = procedura_without_empty["medvik_search_errors"].apply(from_string_to_int_dict)

else:
    procedura_without_empty = procedura_test.copy()
    procedura_without_empty["medvik_explanation"] = "N/A"
    for i in procedura_without_empty.index:
        result = {}
        combined_level = -np.infty
        is_find = set()
        l = procedura_without_empty["combined_medvik"][i]
        for text in sorted(l, key=lambda x: len(x.split(" ")), reverse=True):
            if combined_level > len(text.split(" ")) and all([w in is_find for w in text.split(" ")]):
                continue
                
            message = message_for_GPT(text, l[text], medvik_find_by_code)
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
            
            if result[text] is not None:
                is_find.update(text.split(" "))
                combined_level = len(text.split(" "))

        l = procedura_without_empty["medvik_search"][i]
        for text in l:
            if text in is_find:
                continue
                
            message = message_for_GPT(text, l[text], medvik_find_by_code)
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
            
        procedura_without_empty.at[i, "medvik_explanation"] = result
    procedura_without_empty.to_csv("saved_search/procedura_test4_explanation_without_empty.csv")

procedura_without_empty.head(3)

,label,text,original_text,about,shortcuts,medvik_search,medvik_search_errors,combined_medvik,medvik_explanation
Unnamed: 0,,,,,,,,,
2350,procedura,adjuvantní radioterapii na oblast mammy lymfatik,"st.p. adjuvantní radioterapii, na oblast mammy...",NaN,st p l sin sv,"{'adjuvantní': [('D000084262', 'hypertermická ...","{'adjuvantní': 0, 'radioterapii': 0, 'na': 0, ...","{'adjuvantní lymfatik': [('D011878', 'radioter...","{'adjuvantní lymfatik': None, 'oblast mammy': ..."
2674,procedura,verifikováno MTS v axile,verifikováno MTS v axile,NaN,,"{'axile': [('D000080886', 'spatium parapharyng...","{'verifikováno': 4, 'MTS': 1, 'v': 0, 'axile': 1}",{},"{'axile': ('D001365', 'axila', 'Oblast lidskéh..."
2517,procedura,adjuvance,adjuvance,NaN,,"{'adjuvance': [('D017358', 'skelet buněčné stě...",{'adjuvance': 1},{},"{'adjuvance': ('D017358', 'skelet buněčné stěn..."


In [96]:
if os.path.isfile("saved_search/procedura_test4_explanation_old_message.csv"):
    procedura_old_message = medvik_explanation_all_read_csv("saved_search/procedura_test4_explanation_old_message.csv")
    procedura_old_message["medvik_search_errors"] = procedura_old_message["medvik_search_errors"].apply(from_string_to_int_dict)

else:
    procedura_old_message = procedura_test.copy()
    procedura_old_message["medvik_explanation"] = "N/A"
    for i in procedura_old_message.index:
        result = {}
        combined_level = -np.infty
        is_find = set()
        l = procedura_old_message["combined_medvik"][i]
        for text in sorted(l, key=lambda x: len(x.split(" ")), reverse=True):
            if combined_level > len(text.split(" ")) and all([w in is_find for w in text.split(" ")]):
                continue
                
            message = message_for_GPT_with_empty(text, l[text], medvik_find_by_code)
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
            
            if result[text] is not None:
                is_find.update(text.split(" "))
                combined_level = len(text.split(" "))

        l = procedura_old_message["medvik_search"][i]
        for text in l:
            if text in is_find:
                continue
                
            message = message_for_GPT(text, l[text], medvik_find_by_code)
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
            
        procedura_old_message.at[i, "medvik_explanation"] = result
    procedura_old_message.to_csv("saved_search/procedura_test4_explanation_old_message.csv")

procedura_old_message.head(3)

,label,text,original_text,about,shortcuts,medvik_search,medvik_search_errors,combined_medvik,medvik_explanation
Unnamed: 0,,,,,,,,,
2350,procedura,adjuvantní radioterapii na oblast mammy lymfatik,"st.p. adjuvantní radioterapii, na oblast mammy...",NaN,st p l sin sv,"{'adjuvantní': [('D000084262', 'hypertermická ...","{'adjuvantní': 0, 'radioterapii': 0, 'na': 0, ...","{'adjuvantní lymfatik': [('D011878', 'radioter...","{'adjuvantní lymfatik': None, 'oblast mammy': ..."
2674,procedura,verifikováno MTS v axile,verifikováno MTS v axile,NaN,,"{'axile': [('D000080886', 'spatium parapharyng...","{'verifikováno': 4, 'MTS': 1, 'v': 0, 'axile': 1}",{},"{'axile': ('D000080886', 'spatium parapharynge..."
2517,procedura,adjuvance,adjuvance,NaN,,"{'adjuvance': [('D017358', 'skelet buněčné stě...",{'adjuvance': 1},{},"{'adjuvance': ('D017358', 'skelet buněčné stěn..."


In [97]:
if os.path.isfile("saved_search/procedura_test4_explanation.csv"):
    procedura_test = medvik_explanation_all_read_csv("saved_search/procedura_test4_explanation.csv")
    procedura_test["medvik_search_errors"] = procedura_test["medvik_search_errors"].apply(from_string_to_int_dict)

else:
    procedura_test["medvik_explanation"] = "N/A"
    for i in procedura_test.index:
        result = {}
        combined_level = -np.infty
        is_find = set()
        l = procedura_test["combined_medvik"][i]
        for text in sorted(l, key=lambda x: len(x.split(" ")), reverse=True):
            if combined_level > len(text.split(" ")) and all([w in is_find for w in text.split(" ")]):
                continue
                
            message = message_for_GPT_with_empty(text, l[text], medvik_find_by_code, context=procedura_test["original_text"][i])
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
            
            if result[text] is not None:
                is_find.update(text.split(" "))
                combined_level = len(text.split(" "))

        l = procedura_test["medvik_search"][i]
        for text in l:
            if text in is_find:
                continue
                
            message = message_for_GPT_with_empty(text, l[text], medvik_find_by_code, context=procedura_test["original_text"][i])
            response = send_to_GPT(message)
            result[text] = from_GPT(response, l[text], medvik_find_by_code)
            
        procedura_test.at[i, "medvik_explanation"] = result
        procedura_test.to_csv("saved_search/procedura_test4_explanation.csv")

procedura_test.head(3)

,label,text,original_text,about,shortcuts,medvik_search,medvik_search_errors,combined_medvik,medvik_explanation
Unnamed: 0,,,,,,,,,
2350,procedura,adjuvantní radioterapii na oblast mammy lymfatik,"st.p. adjuvantní radioterapii, na oblast mammy...",NaN,st p l sin sv,"{'adjuvantní': [('D000084262', 'hypertermická ...","{'adjuvantní': 0, 'radioterapii': 0, 'na': 0, ...","{'adjuvantní lymfatik': [('D011878', 'radioter...","{'adjuvantní lymfatik': None, 'oblast mammy': ..."
2674,procedura,verifikováno MTS v axile,verifikováno MTS v axile,NaN,,"{'axile': [('D000080886', 'spatium parapharyng...","{'verifikováno': 4, 'MTS': 1, 'v': 0, 'axile': 1}",{},"{'axile': ('D001366', 'arteria axillaris', 'Ar..."
2517,procedura,adjuvance,adjuvance,NaN,,"{'adjuvance': [('D017358', 'skelet buněčné stě...",{'adjuvance': 1},{},"{'adjuvance': ('D017358', 'skelet buněčné stěn..."


In [98]:
procedura_without_empty["medvik_explanation"] = procedura_without_empty["medvik_explanation"].apply(interprete_explanation)
procedura_old_message["medvik_explanation"] = procedura_old_message["medvik_explanation"].apply(interprete_explanation)
procedura_test["medvik_explanation"] = procedura_test["medvik_explanation"].apply(interprete_explanation)

#### Results

In [99]:
procedura_without_empty["asign"] = "N/A"

i = 0
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Partially"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Right"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Empty"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Empty"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Partially"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Partially"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Right"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Empty"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Right"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Partially"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Right"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Right"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Empty"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Empty"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Right"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Wrong"
i += 1
j = procedura_without_empty.index[i]
procedura_without_empty.at[j, "asign"] = "Empty"

# print(procedura_without_empty["original_text"].iloc[i])
# print(procedura_without_empty["medvik_explanation"].iloc[i])

In [100]:
procedura_old_message["asign"] = "N/A"

i = 0
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Partially"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Partially"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Partially"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Empty"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Empty"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Partially"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Partially"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Right"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Empty"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Right"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Partially"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Right"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Right"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Empty"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Empty"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Right"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Wrong"
i += 1
j = procedura_old_message.index[i]
procedura_old_message.at[j, "asign"] = "Empty"

# print(procedura_old_message["original_text"].iloc[i])
# print(procedura_old_message["medvik_explanation"].iloc[i])

In [101]:
procedura_test["asign"] = "N/A"

i = 0
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Partially"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Partially"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Empty"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Empty"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Partially"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Partially"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Empty"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Partially"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Empty"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Empty"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Right"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Wrong"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Partially"
i += 1
j = procedura_test.index[i]
procedura_test.at[j, "asign"] = "Empty"

# print(procedura_test["original_text"].iloc[i])
# print(procedura_test["medvik_explanation"].iloc[i])
# print()
# print(procedura_test["medvik_search"].iloc[i])
# print(procedura_test["combined_medvik"].iloc[i])

In [102]:
procedura_without_empty["asign"].value_counts()

asign
Wrong        19
Right         6
Empty         6
Partially     4
Name: count, dtype: int64

In [103]:
procedura_old_message["asign"].value_counts()

asign
Wrong        18
Partially     6
Empty         6
Right         5
Name: count, dtype: int64

In [104]:
procedura_test["asign"].value_counts()

asign
Wrong        13
Right        10
Partially     6
Empty         6
Name: count, dtype: int64

- Not so good results, lots terms we have not been able to link.
- We can see that, using context improves results. As we remove long nonsense, we create space for shorter which are correct.
- We can see that there is no difference between linking with non of these option and without.

### Label osobní anamnéza 

In [105]:
improved_osobni = data_improved_punctuation[(data_improved_punctuation["label"] == "osobní anamnéza")].copy()
print("Number of 'osobní anamnéza' label:", len(improved_osobni))
osobni_test = improved_osobni.sample(35, random_state=10)
osobni_test.head(3)

Number of 'osobní anamnéza' label: 214


,label,text,original_text
3555,osobní anamnéza,st.p. thyreoidectomiam,st.p. thyreoidectomiam
2516,osobní anamnéza,cysta v ZHK vlevo,cysta v ZHK vlevo
2155,osobní anamnéza,st.p. operaci močového měcýře,st.p. operaci močového měcýře


In [106]:
osobni_test = make_shortcuts_table(osobni_test)

#### Linking with mistakes

In [107]:
osobni_a = []
for child in offline_mshcz:
    try:
        d = [i for i in child.findall("{http://www.loc.gov/MARC21/slim}datafield") if i.attrib["tag"] == "072" ][0][0].text
        if d is not None and d[0] in ["A", "C", "E"]:
            osobni_a.append(child)
    except IndexError:
        pass                

In [108]:
print(len(osobni_a))

if os.path.isfile("saved_search/osobni_test.csv"):
    osobni_test = medvik_search_all_read_csv("saved_search/osobni_test.csv")
else:
    osobni_test = medvik_with_mistake_all_search_table(osobni_test, osobni_a)
    osobni_test.to_csv("saved_search/osobni_test.csv")

osobni_test.head(3)

9902


,label,text,original_text,about,shortcuts,medvik_search
Unnamed: 0,,,,,,
3555,osobní anamnéza,thyreoidectomiam,st.p. thyreoidectomiam,NaN,st p,"{'thyreoidectomiam': [('D013965', 'tyreoidekto..."
2516,osobní anamnéza,cysta v ZHK vlevo,cysta v ZHK vlevo,NaN,,"{'cysta': [('D003560', 'cysty')], 'v': [('D000..."
2155,osobní anamnéza,operaci močového měcýře,st.p. operaci močového měcýře,NaN,st p,"{'operaci': [('D000072656', 'lymfedém souvisej..."


In [109]:
print_long_searches_len_dep(osobni_test["medvik_search"], limit=25)
osobni_test["medvik_search"] = osobni_test["medvik_search"].apply(lambda x: drop_long_searches_len_dep(x, 70))

[[('typu', 122, 30.5), ('na', 181, 90.5), ('II', 101, 50.5)], [('typ', 115, 38.333333333333336)], [('příl', 154, 38.5)], [('žl', 129, 64.5)], [('na', 181, 90.5)], [('CHCE', 443, 110.75)], [('LCHCE', 1024, 204.8), ('p', 3420, 3420.0)], [('v', 210, 210.0), ('ZHK', 516, 172.0)], [('st', 683, 341.5), ('p', 3420, 3420.0)], [('st', 683, 341.5), ('p', 3420, 3420.0)], [('st', 683, 341.5), ('p', 3420, 3420.0)], [('st', 683, 341.5), ('p', 3420, 3420.0)], [('st', 683, 341.5), ('p', 3420, 3420.0)], [('LDK', 1199, 399.6666666666667), ('st', 683, 341.5), ('p', 3420, 3420.0)], [('hy', 1527, 763.5)], [('stp', 2608, 869.3333333333334)], [('op', 2022, 1011.0), ('p', 3420, 3420.0)], [('op', 2022, 1011.0), ('st', 683, 341.5), ('p', 3420, 3420.0)], [('ME', 2467, 1233.5), ('s', 249, 249.0), ('st', 683, 341.5), ('p', 3420, 3420.0)], [('i', 8948, 8948.0), ('v', 1760, 1760.0), ('s', 3485, 3485.0)]]


In [110]:
temp = osobni_test["medvik_search"].apply(combine_searches)
temp[temp != {}]

Unnamed: 0
2155    {'močového měcýře': [('D000094463', 'transuret...
1167    {'typu II': [('D013915', 'trombastenie'), ('D0...
3081    {'štítné žl': [('D050033', 'dysgeneze štítné ž...
4972    {'konisace čípku': [('D019092', 'konizace dělo...
4630    {'konizaci čípku': [('D019092', 'konizace dělo...
1744    {'mutace leiden': [('D020016', 'rezistence k a...
2428    {'arteriální hypertenze': [('D006973', 'hypert...
2690    {'ulcerozní kolitis': [('D011350', 'proktokoli...
Name: medvik_search, dtype: object

In [111]:
osobni_test["combined_medvik"] = osobni_test["medvik_search"].apply(combine_searches)
osobni_test.head(3)

,label,text,original_text,about,shortcuts,medvik_search,combined_medvik
Unnamed: 0,,,,,,,
3555,osobní anamnéza,thyreoidectomiam,st.p. thyreoidectomiam,NaN,st p,"{'thyreoidectomiam': [('D013965', 'tyreoidekto...",{}
2516,osobní anamnéza,cysta v ZHK vlevo,cysta v ZHK vlevo,NaN,,"{'cysta': [('D003560', 'cysty')], 'v': [], 'ZH...",{}
2155,osobní anamnéza,operaci močového měcýře,st.p. operaci močového měcýře,NaN,st p,"{'operaci': [('D000072656', 'lymfedém souvisej...","{'močového měcýře': [('D000094463', 'transuret..."


#### Choosing best match

In [112]:
print_long_searches(osobni_test["medvik_search"], limit=30)

[[('hypretenze', 31)], [('benigní', 42)], [('vrozená', 44)], [('operace', 44)], [('leukémií', 44)], [('plic', 46)], [('oper', 57), ('žl', 129)], [('typ', 115)], [('typu', 122), ('II', 101)], [('cystami', 151)], [('příl', 154)], [('onemocnění', 174)]]


In [113]:
osobni_test["medvik_search"] = osobni_test["medvik_search"].apply(lambda x: drop_long_searches(x, 90))
osobni_test["medvik_search"] = osobni_test["medvik_search"].apply(drop_empty_searches)

if os.path.isfile("saved_search/osobni_test_explanation.csv"):
    osobni_test = medvik_explanation_all_read_csv("saved_search/osobni_test_explanation.csv")
else:
    osobni_test = medvik_choose_GPT_table(osobni_test)
    osobni_test.to_csv("saved_search/osobni_test_explanation.csv")

osobni_test.head(3)

,label,text,original_text,about,shortcuts,medvik_search,combined_medvik,medvik_explanation
Unnamed: 0,,,,,,,,
3555,osobní anamnéza,thyreoidectomiam,st.p. thyreoidectomiam,NaN,st p,"{'thyreoidectomiam': [('D013965', 'tyreoidekto...",{},"{'thyreoidectomiam': ('D013965', 'tyreoidektom..."
2516,osobní anamnéza,cysta v ZHK vlevo,cysta v ZHK vlevo,NaN,,"{'cysta': [('D003560', 'cysty')], 'vlevo': [('...",{},"{'cysta': ('D003560', 'cysty', 'Patologická du..."
2155,osobní anamnéza,operaci močového měcýře,st.p. operaci močového měcýře,NaN,st p,"{'operaci': [('D000072656', 'lymfedém souvisej...","{'močového měcýře': [('D000283', 'aplikace int...","{'močového měcýře': ('D001743', 'močový měchýř..."


In [114]:
osobni_test["medvik_explanation"] = osobni_test["medvik_explanation"].apply(interprete_explanation)

#### Results

In [115]:
osobni_test["asign"] = "N/A"

i = 0
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Right"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Partially"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Right"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Right"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Right"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Partially"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Wrong"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Right"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Right"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Partially"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Empty"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Right"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Partially"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Right"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Wrong"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Wrong"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Wrong"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Right"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Wrong"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Wrong"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Empty"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Right"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Right"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Partially"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Right"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Partially"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Right"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Wrong"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Right"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Wrong"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Empty"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Partially"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Wrong"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Right"
i += 1
j = osobni_test.index[i]
osobni_test.at[j, "asign"] = "Partially"


# print(osobni_test["original_text"].iloc[i])
# print(osobni_test["medvik_explanation"].iloc[i])
# print()
# print(osobni_test["medvik_search"].iloc[i])
# print(osobni_test["combined_medvik"].iloc[i])

In [116]:
osobni_test["asign"].value_counts()

asign
Right        15
Wrong         9
Partially     8
Empty         3
Name: count, dtype: int64

Right: 43 % | Partially or Right: 67 % | Empty: 9 %

### Label symptom

In [117]:
improved_symptom = data_improved_punctuation[(data_improved_punctuation["label"] == "symptom")].copy()
print("Number of 'symptom' label:", len(improved_symptom))
symptom_test = improved_symptom.sample(35, random_state=1)
symptom_test = make_shortcuts_table(symptom_test)

symptom_test.head(5)

Number of 'symptom' label: 587


,label,text,original_text,shortcuts
5955,symptom,vpravo prominující tu léze v rozhraní HKK,vpravo prominující tu léze v rozhraní HKK,
2707,symptom,mutace v genu NBN,mutace v genu NBN,
182,symptom,vpravo vtažení kůže vysoko v ZHK laterálně,vpravo vtažení kůže vysoko v ZHK laterálně,
5045,symptom,nevolnosti,nevolnosti,
5719,symptom,karieznímu a defktnímu chrupu,karieznímu a defktnímu chrupu,


#### Linking with mistakes

In [118]:
symptomy = []
for child in offline_mshcz:
    try:
        d = [i for i in child.findall("{http://www.loc.gov/MARC21/slim}datafield") if i.attrib["tag"] == "072" ][0][0].text
        if d is not None and d[0] in ["A", "G", "C", "D", "E"]:
            symptomy.append(child)
    except IndexError:
        pass                
print(len(symptomy))

22086


In [119]:
if os.path.isfile("saved_search/symptom_test.csv"):
    symptom_test = medvik_search_all_read_csv("saved_search/symptom_test.csv")
else:
    symptom_test = medvik_with_mistake_all_search_table(symptom_test, symptomy)
    symptom_test.to_csv("saved_search/symptom_test.csv")

symptom_test.head(3)

,label,text,original_text,about,shortcuts,medvik_search
Unnamed: 0,,,,,,
5955,symptom,vpravo prominující tu léze v rozhraní HKK,vpravo prominující tu léze v rozhraní HKK,NaN,,"{'vpravo': [('D000069584', 'jednostranný karci..."
2707,symptom,mutace v genu NBN,mutace v genu NBN,NaN,,"{'mutace': [('D009154', 'mutace')], 'v': [('D0..."
182,symptom,vpravo vtažení kůže vysoko v ZHK laterálně,vpravo vtažení kůže vysoko v ZHK laterálně,NaN,,"{'vpravo': [('D000069584', 'jednostranný karci..."


In [120]:
print_long_searches_len_dep(symptom_test["medvik_search"], limit=20)
symptom_test["medvik_search"] = symptom_test["medvik_search"].apply(lambda x: drop_long_searches_len_dep(x, 30))

[[('neostře', 213, 30.428571428571427)], [('lož', 110, 36.666666666666664)], [('k', 93, 93.0)], [('mamy', 416, 104.0)], [('7mm', 860, 286.6666666666667)], [('v', 310, 310.0), ('HKK', 319, 106.33333333333333)], [('v', 310, 310.0), ('NBN', 4261, 1420.3333333333333)], [('v', 310, 310.0), ('ZHK', 681, 227.0)], [('v', 310, 310.0), ('HKK', 319, 106.33333333333333)], [('v', 310, 310.0), ('obl', 120, 40.0)], [('v', 310, 310.0), ('až', 104, 52.0), ('do', 62, 31.0)], [('DKK', 1040, 346.6666666666667)], [('dkk', 1040, 346.6666666666667)], [('dkk', 1040, 346.6666666666667)], [('s', 490, 490.0), ('lemem', 223, 44.6)], [('s', 490, 490.0)], [('a', 714, 714.0)]]


In [121]:
temp = symptom_test["medvik_search"].apply(combine_searches)
temp[temp != {}]

Unnamed: 0
182     {'vpravo laterálně': [('D000069584', 'jednostr...
2743    {'pozitivní resekční': [('D000072662', 'resekč...
2685    {'pozitivní resekční': [('D000072662', 'resekč...
Name: medvik_search, dtype: object

In [122]:
symptom_test["combined_medvik"] = symptom_test["medvik_search"].apply(combine_searches)
symptom_test.head(3)

,label,text,original_text,about,shortcuts,medvik_search,combined_medvik
Unnamed: 0,,,,,,,
5955,symptom,vpravo prominující tu léze v rozhraní HKK,vpravo prominující tu léze v rozhraní HKK,NaN,,"{'vpravo': [('D000069584', 'jednostranný karci...",{}
2707,symptom,mutace v genu NBN,mutace v genu NBN,NaN,,"{'mutace': [('D009154', 'mutace')], 'v': [], '...",{}
182,symptom,vpravo vtažení kůže vysoko v ZHK laterálně,vpravo vtažení kůže vysoko v ZHK laterálně,NaN,,"{'vpravo': [('D000069584', 'jednostranný karci...","{'vpravo laterálně': [('D000069584', 'jednostr..."


#### Choosing best match

In [123]:
print_long_searches(symptom_test["medvik_search"], limit=20)

[[('vysoko', 21), ('laterálně', 36)], [('parciální', 22)], [('mezi', 31), ('stac', 29)], [('ledvin', 37)], [('dosahuje', 38), ('bazi', 28)], [('dosahuje', 38)], [('genu', 40)], [('jater', 40)], [('vlevo', 41)], [('vel', 47)], [('bolesi', 74)]]


In [124]:
symptom_test["medvik_search"] = symptom_test["medvik_search"].apply(lambda x: drop_long_searches(x, 80))
symptom_test["medvik_search"] = symptom_test["medvik_search"].apply(drop_empty_searches)

if os.path.isfile("saved_search/symptom_test_explanation.csv"):
    symptom_test = medvik_explanation_all_read_csv("saved_search/symptom_test_explanation.csv")
else:
    symptom_test = medvik_choose_GPT_table(symptom_test)
    symptom_test.to_csv("saved_search/symptom_test_explanation.csv")

symptom_test.head(3)

,label,text,original_text,about,shortcuts,medvik_search,combined_medvik,medvik_explanation
Unnamed: 0,,,,,,,,
5955,symptom,vpravo prominující tu léze v rozhraní HKK,vpravo prominující tu léze v rozhraní HKK,NaN,,"{'vpravo': [('D000069584', 'jednostranný karci...",{},"{'vpravo': ('D055458', 'faktory určující pravo..."
2707,symptom,mutace v genu NBN,mutace v genu NBN,NaN,,"{'mutace': [('D009154', 'mutace')], 'genu': [(...",{},"{'mutace': ('D009154', 'mutace', 'Jakákoli det..."
182,symptom,vpravo vtažení kůže vysoko v ZHK laterálně,vpravo vtažení kůže vysoko v ZHK laterálně,NaN,,"{'vpravo': [('D000069584', 'jednostranný karci...","{'vpravo laterálně': [('D000069584', 'jednostr...","{'vpravo laterálně': None, 'vpravo': ('D000073..."


In [125]:
symptom_test["medvik_explanation"] = symptom_test["medvik_explanation"].apply(interprete_explanation)

#### Results

In [126]:
symptom_test["asign"] = "N/A"

i = 0
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Right"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Partially"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Partially"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Partially"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Partially"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Right"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Partially"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Empty"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Partially"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Partially"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Empty"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Right"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Empty"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Partially"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Right"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Empty"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Right"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Partially"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Right"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Partially"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Right"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = symptom_test.index[i]
symptom_test.at[j, "asign"] = "Partially"


# print(symptom_test["original_text"].iloc[i])
# print(symptom_test["medvik_explanation"].iloc[i])
# print()
# print(symptom_test["medvik_search"].iloc[i])

In [127]:
symptom_test["asign"].value_counts()

asign
Wrong        13
Partially    11
Right         7
Empty         4
Name: count, dtype: int64

Right: 20 % | Partially: 51 % | Empty 12 %

Observations:
- Text with lot of words, mostly not in base form.
- Lots of shortcuts
- Lot of not medical specific words - will help extracting potential entities

### Label NE osobní anamnéza - Converting negative samples to positives

In [128]:
improved_ne_osobni = data_improved_punctuation[data_improved_punctuation["label"] == "NE osobní anamnéza"].copy()
print("Number of 'NE osobní label' label:", len(improved_ne_osobni))

Number of 'NE osobní label' label: 61


In [129]:
NE_label = data_improved_punctuation[data_improved_punctuation["label"].apply(lambda x: "NE" in x)]

hard_negative = set( [x[0] for x in NE_label["text"].apply(
                lambda x: [y for y in x.split(" ") if is_negative_word(y)]) if len(x) >= 1])
soft_negative = set([x[0] for x in NE_label["text"].apply(
                lambda x: [y for y in x.split(" ") if is_soft_negative_word(y)]) if len(x) >= 1])

print(soft_negative)
print()

random.seed(42)
print(sorted(hard_negative.difference(soft_negative), key=lambda x: random.randint(0, 100))[:20])

{'0', 'ne', 'O', 'negat.', 'neg', 'negativní', 'neguje', 'negat', 'bez'}

['nezvětšení', 'nepovleklý', 'nebo', 'nehmatá', 'nezhubla', 'neurologická', 'neléčí', 'nerozšířeno', 'nenápadná', 'nezv.', 'neuvádí', 'nebolestivé', 'nebol', 'neurologicky', 'nezvěětšená', 'nepozoruje', 'nebolí', 'nezaznamenala', 'nebolest', 'nevýrazně']


In [130]:
ne_osobni_test = improved_ne_osobni.sample(35, random_state=25)
ne_osobni_test = make_negation_table(ne_osobni_test)
ne_osobni_test = make_shortcuts_table(ne_osobni_test)
ne_osobni_test.head(3)

,label,text,original_text,neg,shortcuts
759,NE osobní anamnéza,HAK nebere,HAK nebere,False,
862,NE osobní anamnéza,transfuze,transfuze 0,True,
923,NE osobní anamnéza,ITP,ITP,False,


#### Linking to MSHCZ

In [131]:
print(len(osobni_a))

if os.path.isfile("saved_search/ne_osobni_test.csv"):
    ne_osobni_test = medvik_search_all_read_csv("saved_search/ne_osobni_test.csv")
else:
    ne_osobni_test = medvik_with_mistake_all_search_table(ne_osobni_test, osobni_a)
    ne_osobni_test.to_csv("saved_search/ne_osobni_test.csv")

ne_osobni_test.head(3)

9902


,label,text,original_text,about,neg,shortcuts,medvik_search
Unnamed: 0,,,,,,,
759,NE osobní anamnéza,HAK nebere,HAK nebere,NaN,False,,"{'HAK': [('D001035', 'afakie'), ('D001036', 'a..."
862,NE osobní anamnéza,transfuze,transfuze 0,NaN,True,,"{'transfuze': [('D000087526', 'podvázání pupeč..."
923,NE osobní anamnéza,ITP,ITP,NaN,False,,"{'ITP': [('D000012', 'abetalipoproteinemie'), ..."


In [132]:
print_long_searches_len_dep(ne_osobni_test["medvik_search"], limit=20)
ne_osobni_test["medvik_search"] = ne_osobni_test["medvik_search"].apply(lambda x: drop_long_searches_len_dep(x, limit=70))

[[('je', 374, 187.0)], [('s', 249, 249.0), ('se', 49, 24.5)], [('a', 254, 254.0)], [('PE', 1671, 835.5)], [('ITP', 2977, 992.3333333333334)], [('ITP', 2977, 992.3333333333334)], [('CA', 2002, 1001.0)], [('fce', 4322, 1440.6666666666667), ('syst', 198, 49.5)]]


In [133]:
temp = ne_osobni_test["medvik_search"].apply(combine_searches)
ne_osobni_test["combined_medvik"] = ne_osobni_test["medvik_search"].apply(combine_searches)
temp[temp != {}]

Unnamed: 0
2193    {'potraty spont': [('D000022', 'samovolný potr...
1882    {'gynekologická onemocnění': [('D005831', 'gyn...
2656    {'gynekologická onemocnění': [('D005831', 'gyn...
1687    {'COVID 19': [('D000086742', 'testování na COV...
4231    {'onemocnění jater': [('D008107', 'nemoci jate...
964     {'ničím se': [('D000077299', 'nozokomiální pne...
2536    {'neurologická onemocnění': [('D009422', 'nemo...
367     {'gynekologická onemocnění': [('D005831', 'gyn...
2038    {'LK syst': [('D018487', 'dysfunkce levé srdeč...
Name: medvik_search, dtype: object

In [134]:
print_long_searches(ne_osobni_test["medvik_search"], limit=50)

[[('kompl', 67)], [('onemocnění', 174)], [('onemocnění', 174)], [('onemocnění', 174)], [('onemocnění', 174)], [('onemocnění', 174)], [('syst', 198)]]


In [135]:
ne_osobni_test["medvik_search"] = ne_osobni_test["medvik_search"].apply(lambda x: drop_long_searches(x, 90))
ne_osobni_test["medvik_search"] = ne_osobni_test["medvik_search"].apply(drop_empty_searches)

if os.path.isfile("saved_search/ne_osobni_test_explanation.csv"):
    ne_osobni_test = medvik_explanation_all_read_csv("saved_search/ne_osobni_test_explanation.csv")
else:
    ne_osobni_test = medvik_choose_GPT_table(ne_osobni_test)
    ne_osobni_test.to_csv("saved_search/ne_osobni_test_explanation.csv")

ne_osobni_test["medvik_explanation"] = ne_osobni_test["medvik_explanation"].apply(interprete_explanation)
ne_osobni_test.head(3)

,label,text,original_text,about,neg,shortcuts,medvik_search,combined_medvik,medvik_explanation
Unnamed: 0,,,,,,,,,
759,NE osobní anamnéza,HAK nebere,HAK nebere,NaN,False,,"{'HAK': [('D001035', 'afakie'), ('D001036', 'a...",{},"{'HAK': ('D038642', 'syndrom třeseného dítěte'..."
862,NE osobní anamnéza,transfuze,transfuze 0,NaN,True,,"{'transfuze': [('D000087526', 'podvázání pupeč...",{},"{'transfuze': ('D001803', 'krevní transfuze', ..."
923,NE osobní anamnéza,ITP,ITP,NaN,False,,{},{},{}


#### Results

In [136]:
ne_osobni_test["asign"] = "N/A"

i = 0
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Empty"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Partially"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Partially"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Partially"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Partially"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Partially"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Empty"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Partially"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Empty"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Partially"
i += 1
j = ne_osobni_test.index[i]
ne_osobni_test.at[j, "asign"] = "Right"


# print(ne_osobni_test["original_text"].iloc[i])
# print(ne_osobni_test["medvik_explanation"].iloc[i])
# print()
# print(ne_osobni_test["medvik_search"].iloc[i])

In [137]:
ne_osobni_test["asign"].value_counts()

asign
Right        19
Partially     7
Wrong         6
Empty         3
Name: count, dtype: int64

Right: 54 % | Partially: 74 % | Empty: 9 %

Observations
- Not so good in filtering
- Bad with shortcuts

### Label NE symptom

In [138]:
improved_ne_symptom = data_improved_punctuation[(data_improved_punctuation["label"] == "NE symptom")].copy()
print("Number of 'NE symptom' label:", len(improved_ne_symptom))

ne_symptom_test = improved_ne_symptom.sample(35, random_state=23)
ne_symptom_test = make_negation_table(ne_symptom_test)
ne_symptom_test = make_shortcuts_table(ne_symptom_test)
ne_symptom_test.head(3)

Number of 'NE symptom' label: 1024


,label,text,original_text,neg,shortcuts
4163,NE symptom,kůže klidná,kůže klidná,False,
1281,NE symptom,KI 100 lucidní,KI 100% lucidní,False,
2348,NE symptom,spráná LK,spráná systol.i diastol.funkce LK,False,systol i diastol funkce


#### Linking to MSHCZ

In [139]:
print(len(symptomy))

if os.path.isfile("saved_search/ne_symptom_test.csv"):
    ne_symptom_test = medvik_search_all_read_csv("saved_search/ne_symptom_test.csv")
else:
    ne_symptom_test = medvik_with_mistake_all_search_table(ne_symptom_test, symptomy)
    ne_symptom_test.to_csv("saved_search/ne_symptom_test.csv")

ne_symptom_test.head(3)

22086


,label,text,original_text,about,neg,shortcuts,medvik_search
Unnamed: 0,,,,,,,
4163,NE symptom,kůže klidná,kůže klidná,NaN,False,,"{'kůže': [('D012867', 'kůže')], 'klidná': [('D..."
1281,NE symptom,KI 100 lucidní,KI 100% lucidní,NaN,False,,"{'KI': [('D000066491', 'klinické rozhodování')..."
2348,NE symptom,spráná LK,spráná systol.i diastol.funkce LK,NaN,False,systol i diastol funkce,"{'spráná': [('D003951', 'chybná diagnóza'), ('..."


In [140]:
print_long_searches_len_dep(ne_symptom_test["medvik_search"], limit=20)
ne_symptom_test["medvik_search"] = ne_symptom_test["medvik_search"].apply(lambda x: drop_long_searches_len_dep(x, limit=30))

[[('norm', 117, 29.25)], [('do', 62, 31.0)], [('komp', 332, 83.0)], [('k', 93, 93.0), ('v', 310, 310.0), ('normě', 121, 24.2)], [('na', 337, 168.5)], [('V', 310, 310.0)], [('v', 310, 310.0), ('normě', 121, 24.2)], [('v', 310, 310.0)], [('je', 747, 373.5)], [('je', 747, 373.5)], [('KI', 912, 456.0)], [('SNB', 1419, 473.0)], [('a', 714, 714.0)], [('i', 20118, 20118.0)]]


In [141]:
temp = ne_symptom_test["medvik_search"].apply(combine_searches)
ne_symptom_test["combined_medvik"] = ne_symptom_test["medvik_search"].apply(combine_searches)
temp[temp != {}]

Unnamed: 0
2348    {'LK systol': [('D018487', 'dysfunkce levé srd...
1022    {'covid 19': [('D000086663', 'vakcíny proti CO...
884     {'chuť jídlu': [('D054439', 'ghrelin'), ('D001...
Name: medvik_search, dtype: object

In [142]:
print_long_searches(ne_symptom_test["medvik_search"], limit=30)

[[('funkce', 31)], [('bil', 34)], [('perif', 40)], [('otoku', 68)], [('patrné', 101)], [('norm', 117)], [('normě', 121)], [('normě', 121), ('lab', 42)]]


In [143]:
ne_symptom_test["medvik_search"] = ne_symptom_test["medvik_search"].apply(lambda x: drop_long_searches(x, 80))
ne_symptom_test["medvik_search"] = ne_symptom_test["medvik_search"].apply(drop_empty_searches)

if os.path.isfile("saved_search/ne_symptom_test_explanation.csv"):
    ne_symptom_test = medvik_explanation_all_read_csv("saved_search/ne_symptom_test_explanation.csv")
else:
    ne_symptom_test = medvik_choose_GPT_table(ne_symptom_test)
    ne_symptom_test.to_csv("saved_search/ne_symptom_test_explanation.csv")

ne_symptom_test["medvik_explanation"] = ne_symptom_test["medvik_explanation"].apply(interprete_explanation)
ne_symptom_test.head(3)

,label,text,original_text,about,neg,shortcuts,medvik_search,combined_medvik,medvik_explanation
Unnamed: 0,,,,,,,,,
4163,NE symptom,kůže klidná,kůže klidná,NaN,False,,"{'kůže': [('D012867', 'kůže')], 'klidná': [('D...",{},"{'kůže': ('D012867', 'kůže', 'Orgán pokrývajíc..."
1281,NE symptom,KI 100 lucidní,KI 100% lucidní,NaN,False,,"{'100': [('D004121', 'dimethylsulfoxid'), ('D0...",{},"{'100': ('D009418', 'proteiny S100', 'A family..."
2348,NE symptom,spráná LK,spráná systol.i diastol.funkce LK,NaN,False,systol i diastol funkce,"{'spráná': [('D003951', 'chybná diagnóza'), ('...","{'LK systol': [('D018487', 'dysfunkce levé srd...","{'systol LK diastol': ('D018487', 'dysfunkce l..."


#### Results

In [144]:
ne_symptom_test["asign"] = "N/A"

i = 0
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Right"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Partially"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Right"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Right"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Right"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Partially"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Right"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Partially"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Right"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Right"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Partially"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Partially"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Right"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Partially"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Right"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Partially"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Right"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Partially"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Right"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Partially"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Partially"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Right"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Right"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Right"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Right"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_symptom_test.index[i]
ne_symptom_test.at[j, "asign"] = "Partially"


# print(ne_symptom_test["original_text"].iloc[i])
# print(ne_symptom_test["medvik_explanation"].iloc[i])
# print()
# print(ne_symptom_test["medvik_search"].iloc[i])
# print(ne_symptom_test["combined_medvik"].iloc[i])

In [145]:
ne_symptom_test["asign"].value_counts()

asign
Right        15
Partially    11
Wrong         9
Name: count, dtype: int64

Right: 43 % | Partially: 74 % | Empty: 0 %

Observations
- Shortcuts
- It should filter better

### Label medikace 01 - Linking medikace to MSHCZ

In [146]:
improved_medikace = data_improved_punctuation[(data_improved_punctuation["label"] == "medikace")].copy()
print("Number of 'medikace' label:", len(improved_medikace))

medikace_test = improved_medikace.sample(35, random_state=54)
medikace_test = make_shortcuts_table(medikace_test)
medikace_test.head(3)

Number of 'medikace' label: 305


,label,text,original_text,shortcuts
5604,medikace,DEPAKINE CHRONO,DEPAKINE CHRONO,
2539,medikace,NACHT EC,NACHT EC,
1929,medikace,neoadjuv CHT na bazi antracyklinů a taxanů,neoadjuv CHT na bazi antracyklinů a taxanů,


In [147]:
medikace = []
for child in offline_mshcz:
    try:
        d = [i for i in child.findall("{http://www.loc.gov/MARC21/slim}datafield") if i.attrib["tag"] == "072" ][0][0].text
        if d is not None and d[0] in ["D", "E"]:
            medikace.append(child)
    except IndexError:
        pass      
        
print(len(medikace))

13410


In [148]:
if os.path.isfile("saved_search/medikace_test.csv"):
    medikace_test = medvik_search_all_read_csv("saved_search/medikace_test.csv")
else:
    medikace_test = medvik_with_mistake_all_search_table(medikace_test, medikace)
    medikace_test.to_csv("saved_search/medikace_test.csv")

medikace_test.head(3)

,label,text,original_text,about,shortcuts,medvik_search
Unnamed: 0,,,,,,
5604,medikace,DEPAKINE CHRONO,DEPAKINE CHRONO,NaN,,"{'DEPAKINE': [('D014635', 'kyselina valproová'..."
2539,medikace,NACHT EC,NACHT EC,NaN,,"{'NACHT': [('D000071199', 'protein NLRP3')], '..."
1929,medikace,neoadjuv CHT na bazi antracyklinů a taxanů,neoadjuv CHT na bazi antracyklinů a taxanů,NaN,,"{'neoadjuv': [('D020360', 'neoadjuvantní terap..."


In [149]:
print_long_searches_len_dep(medikace_test["medvik_search"], limit=20)
medikace_test["medvik_search"] = medikace_test["medvik_search"].apply(lambda x: drop_long_searches_len_dep(x, 30))

[[('D', 73, 73.0)], [('na', 187, 93.5), ('a', 488, 488.0)], [('na', 187, 93.5)], [('80mg', 560, 140.0)], [('st', 430, 215.0), ('p', 4514, 4514.0), ('NACT', 1145, 286.25)]]


In [150]:
temp = medikace_test["medvik_search"].apply(combine_searches)
medikace_test["combined_medvik"] = medikace_test["medvik_search"].apply(combine_searches)
temp[temp != {}]

Series([], Name: medvik_search, dtype: object)

In [151]:
print_long_searches(medikace_test["medvik_search"], limit=10)

[[('forte', 11)], [('ditahiden', 14)], [('bazi', 15)], [('bazi', 15)], [('elicea', 21)], [('10', 28)], [('vigantol', 48)], [('taxHea', 56)], [('detralex', 100)]]


In [152]:
medikace_test["medvik_search"] = medikace_test["medvik_search"].apply(lambda x: drop_long_searches(x, 25))
medikace_test["medvik_search"] = medikace_test["medvik_search"].apply(drop_empty_searches)

if os.path.isfile("saved_search/medikace_test_explanation.csv"):
    medikace_test = medvik_explanation_all_read_csv("saved_search/medikace_test_explanation.csv")
else:
    medikace_test = medvik_choose_GPT_table(medikace_test)
    medikace_test.to_csv("saved_search/medikace_test_explanation.csv")

medikace_test["medvik_explanation"] = medikace_test["medvik_explanation"].apply(interprete_explanation)
medikace_test.head(3)

,label,text,original_text,about,shortcuts,medvik_search,combined_medvik,medvik_explanation
Unnamed: 0,,,,,,,,
5604,medikace,DEPAKINE CHRONO,DEPAKINE CHRONO,NaN,,"{'DEPAKINE': [('D014635', 'kyselina valproová'...",{},"{'DEPAKINE': ('D014635', 'kyselina valproová',..."
2539,medikace,NACHT EC,NACHT EC,NaN,,"{'NACHT': [('D000071199', 'protein NLRP3')], '...",{},"{'NACHT': ('D000071199', 'protein NLRP3', 'An ..."
1929,medikace,neoadjuv CHT na bazi antracyklinů a taxanů,neoadjuv CHT na bazi antracyklinů a taxanů,NaN,,"{'neoadjuv': [('D020360', 'neoadjuvantní terap...",{},"{'neoadjuv': ('D020360', 'neoadjuvantní terapi..."


#### Results

In [153]:
medikace_test["asign"] = "N/A"

i = 0
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Partially"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Empty"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Empty"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Empty"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Empty"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Empty"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Empty"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Empty"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Empty"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Empty"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Empty"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Empty"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Empty"

# print(medikace_test["original_text"].iloc[i])
# print(medikace_test["medvik_explanation"].iloc[i])
# print()
# print(medikace_test["medvik_search"].iloc[i])

In [154]:
medikace_test["asign"].value_counts()

asign
Right        17
Empty        12
Wrong         5
Partially     1
Name: count, dtype: int64

Right: 49 % | Partially: 51 % | Empty 34 %
    
Observations
- Not so good filter (for example weekly don't filtered)
- Many medicines not in database
- Lots of right thanks to short text

### Label medikace 02 - Linking medikace to SUKL, Limit for mistake
Many medicines is unknown for MSHCZ, for this reason I will use SUKL databaze, where I will search. Because GPT is not suitable for linking chemicals with drugs I will use only mathematical methods for linking.

In [155]:
products = pd.read_csv("databaze/DLP20240229/dlp_lecivepripravky.csv",
                       encoding="cp1250", delimiter=";")[["KOD_SUKL", "NAZEV", "SILA", "FORMA", "ATC_WHO", "NAZEV_REG"]]
products = products[["NAZEV", "ATC_WHO"]].drop_duplicates()
products.head(3)

,NAZEV,ATC_WHO
0,ACYLCOFFIN,N02BA51
1,DILURAN,S01EC01
2,HYDROCHLOROTHIAZID LÉČIVA,C03AA03


In [156]:
substances_with_ATC = pd.read_csv("databaze/DLP20240229/dlp_atc.csv", encoding="cp1250", delimiter=";")

temp = substances_with_ATC[["ATC", "NAZEV_EN"]]
temp = temp.rename(columns={"NAZEV_EN":"NAZEV"})
temp.index = temp.index + len(substances_with_ATC)
substances_with_ATC_concat = pd.concat([substances_with_ATC[["ATC", "NAZEV"]], temp])

substances_with_ATC_concat.head(3)

,ATC,NAZEV
0,A,TRÁVICÍ TRAKT A METABOLISMUS
1,A01,STOMATOLOGICKÉ PŘÍPRAVKY
2,A01A,STOMATOLOGICKÉ PŘÍPRAVKY


In [157]:
# Linking to atc name by code
def count_to_tuple(l):
    un, count = np.unique(l, return_counts=True)
    result = []
    for i, _ in enumerate(un):
        result.append((un[i], count[i]))
    return result


def link_atc_preserve_count(list_atc):
    count = count_to_tuple(list_atc)
    atc = list(set(list_atc))
    r = substances_with_ATC[substances_with_ATC["ATC"].apply(lambda x: x in atc)]
    return [(r["ATC"][i], r["NAZEV"][i], [x[1] for x in count if x[0] == r["ATC"][i]][0]) for i in r.index]
    

# Search chemicals
def sukl_chemicals_search(string, database=substances_with_ATC_concat):
    string = string.lower()

    find = database[database["NAZEV"].apply(
        lambda x: string == x.lower())]
    find = database[database["NAZEV"].apply(
        lambda x: (" " + string + " ") in (" " + x.lower() + " "))] if len(find) == 0 else find
    find = database[database["NAZEV"].apply(
        lambda x: string in x.lower())] if len(find) == 0 else find
    
    return list(find["ATC"])


def sukl_chemicals_mistake_search(string, mistakes, database=substances_with_ATC_concat):
    string = patternize(string)
    pattern = regex.compile(f"({string}){{e<={mistakes}}}", regex.IGNORECASE)
    test = lambda x: pattern.search(x) is not None
    find = database[database["NAZEV"].apply(test)]
    return list(find["ATC"])


def sukl_find_chemicals_search(string, output_errors=True,
                               database=substances_with_ATC_concat, limit=0.5):
    return search_words_mistake_bottom(string,
                         lambda x: link_atc_preserve_count(
                             sukl_chemicals_search(x, database=database)),
                         lambda x, y: link_atc_preserve_count(
                             sukl_chemicals_mistake_search(x, y, database=database)),
                         limit=limit,
                         output_errors=output_errors)

# Search drugs
def sukl_drugs_search(string):
    string = string.lower()
    find = products[products["NAZEV"].apply(lambda x: string == x.lower())]
    find = products[products["NAZEV"].apply(
        lambda x: (" " + string + " ") in (" " + x.lower() + " "))] if len(find) == 0 else find
    find = products[products["NAZEV"].apply(lambda x: string in x.lower())] if len(find) == 0 else find
    return list(find["ATC_WHO"])


def sukl_drugs_mistake_search(string, mistakes):
    string = patternize(string)
    pattern = regex.compile(f"({string}){{e<={mistakes}}}", regex.IGNORECASE)
    test = lambda x: pattern.search(x) is not None
    find = products[products["NAZEV"].apply(test)]
    return list(find["ATC_WHO"])


def sukl_find_drugs_search(string, output_errors=True, limit=0.5):
    return search_words_mistake_bottom(string,
                         lambda x: link_atc_preserve_count(sukl_drugs_search(x)),
                         lambda x, y: link_atc_preserve_count(sukl_drugs_mistake_search(x, y)),
                         limit=limit,
                         output_errors=output_errors)


# Chemical shortcuts search
def sukl_chemicals_shortcuts_search(string, database=substances_with_ATC_concat):
    string = string.lower()
    find = database[database["NAZEV"].apply(lambda x: string == x[:len(string)].lower())]
    find = database[database["NAZEV"].apply(lambda x: (" " + string) in x.lower()
                                           )] if len(find) == 0 else find
    return list(find["ATC"])


def sukl_chemicals_shortcuts_mistake_search(string, mistakes, database=substances_with_ATC_concat):
    string = patternize(string)
    pattern_start = regex.compile(f"({string}){{e<={mistakes}}}", regex.IGNORECASE)
    pattern_in = regex.compile(f"( {string}){{e<={mistakes}}}", regex.IGNORECASE)
    test = lambda x: (pattern_start.search(x[:(len(string) + mistakes)]) is not None or
                      pattern_in.search(x) is not None)
    find = database[database["NAZEV"].apply(test)]
    return list(find["ATC"])


def sukl_find_chemicals_shortcuts_search(string, output_errors=True,
                                         database=substances_with_ATC_concat, limit=0.5):
    return search_words_mistake_bottom(string,
                         lambda x: link_atc_preserve_count(
                             sukl_chemicals_shortcuts_search(x, database=database)),
                         lambda x, y: link_atc_preserve_count(
                             sukl_chemicals_shortcuts_mistake_search(x, y, database=database)),
                         limit=limit,
                         output_errors=output_errors)


# Drugs shortcuts search
def sukl_drugs_shortcuts_search(string):
    string = string.lower()
    find = products[products["NAZEV"].apply(lambda x: string == x[:len(string)].lower())]
    find = products[products["NAZEV"].apply(lambda x: (" " + string) in x.lower())] if len(find) == 0 else find
    return list(find["ATC_WHO"])
    

def sukl_drugs_shortcuts_mistake_search(string, mistakes):
    string = patternize(string)
    pattern_start = regex.compile(f"({string}){{e<={mistakes}}}", regex.IGNORECASE)
    pattern_in = regex.compile(f"( {string}){{e<={mistakes}}}", regex.IGNORECASE)
    test = lambda x: (pattern_start.search(x[:(len(string) + mistakes)]) is not None or
                      pattern_in.search(x) is not None)
    find = products[products["NAZEV"].apply(test)]
    return list(find["ATC_WHO"])


def sukl_find_drugs_shortcuts_search(string, output_errors=True, limit=0.5):
    return search_words_mistake_bottom(string,
                     lambda x: link_atc_preserve_count(sukl_drugs_shortcuts_search(x)),
                     lambda x, y: link_atc_preserve_count(sukl_drugs_shortcuts_mistake_search(x, y)),
                     limit=limit,
                     output_errors=output_errors)

In [158]:
# Statistically choosing best match
def combine_preserve_count(linked_text):
    combined = combine_searches({k: map(lambda x: (x[0], x[1]), linked_text[k]) for k in linked_text})
    result = {}
    for k in combined:
        links = []
        for l in combined[k]:
            sum_ = sum(
                next(filter(lambda y: y[0]==l[0], linked_text[x]))[2]
                    for x in k.split(" "))
            
            links.append((l[0], l[1], sum_))
        result[k] = links

    return result


def combine_2_searched_databaze(priority1, priority2, mistakes1, mistakes2):
    mistakes1 = mistakes1.copy()
    mistakes2 = mistakes2.copy()
    for j in priority1:
        if len(j.split(" ")) >= 2:
            mistakes1[j] = sum([mistakes1[x] for x in j.split(" ")])
    for j in priority2:
        if len(j.split(" ")) >= 2:
            mistakes2[j] = sum([mistakes2[x] for x in j.split(" ")])
    
    dict_with_priority = ({(len(k.split(" ")), -mistakes1.pop(k, 10), 2, k) : priority1[k] for k in priority1} |
                          {(len(k.split(" ")), -mistakes2.pop(k, 10), 1, k) : priority2[k] for k in priority2} )
    result = {}
    in_result = set()   
    for record in sorted(dict_with_priority, reverse=True):
        if any([x not in in_result for x in record[3].split(" ")]):
            for x in record[3].split(" "):
                in_result.add(x)
            result[record[3]] = dict_with_priority[record]
    return result


def prioritize_sukl(one_search, complete_search):
    in_count = sum([1 for k in complete_search if one_search[1] in k[1]])
    return (one_search[2], in_count, random.randint(0, 100))

In [159]:
def sukl_search_all_read_csv(path):
    db = pd.read_csv(path)
    db.index = db["Unnamed: 0"]
    db.drop(["Unnamed: 0"], axis=1, inplace=True)
    
    db["sukl_search_chemicals"] = db["sukl_search_chemicals"].apply(from_string_to_dict)
    db["sukl_chemicals_errors"] = db["sukl_chemicals_errors"].apply(from_string_to_int_dict)
    db["sukl_search_chemicals"] = db["sukl_search_chemicals"].apply(
        lambda x: {k : list(map(lambda e: (e[0], e[1], int(e[2])), x[k])) for k in x})
    
    db["sukl_search_drugs"] = db["sukl_search_drugs"].apply(from_string_to_dict)
    db["sukl_drugs_errors"] = db["sukl_drugs_errors"].apply(from_string_to_int_dict)
    db["sukl_search_drugs"] = db["sukl_search_drugs"].apply(
        lambda x: {k : list(map(lambda e: (e[0], e[1], int(e[2])), x[k])) for k in x})

    return db

#### Linking

In [160]:
medikace_test = improved_medikace.sample(35, random_state=21)
medikace_test = make_shortcuts_table(medikace_test)
medikace_test.head(3)

,label,text,original_text,shortcuts
3848,medikace,tamoxifenu,tamoxifenu,
2984,medikace,herceptinu,herceptinu,
4807,medikace,neoadjuvantní chemoterapie paclitaxel weekly,neoadjuvantní chemoterapie - paclitaxel weekly,


In [161]:
if os.path.isfile("saved_search/medikace_test2.csv"):
    medikace_test = sukl_search_all_read_csv("saved_search/medikace_test2.csv")
    
else:
    medikace_test["sukl_search_chemicals"] = medikace_test["text"].apply(lambda x: sukl_find_chemicals_search(x, output_errors=True))

    medikace_test["sukl_search_chemicals"] = medikace_test.apply(lambda x: x.sukl_search_chemicals |
                (sukl_find_chemicals_shortcuts_search(x.shortcuts ,output_errors=True) if x.shortcuts != "" else {}), axis=1)

    medikace_test["sukl_search_drugs"] = medikace_test["text"].apply(lambda x: sukl_find_drugs_search(x, output_errors=True))
    medikace_test["sukl_search_drugs"] = medikace_test.apply(lambda x: x.sukl_search_drugs |
                (sukl_find_drugs_shortcuts_search(x.shortcuts ,output_errors=True) if x.shortcuts != "" else {}), axis=1)

    medikace_test["sukl_chemicals_errors"] = medikace_test["sukl_search_chemicals"].apply(lambda x: {k[0]:k[1] for k in x})
    medikace_test["sukl_search_chemicals"] = medikace_test["sukl_search_chemicals"].apply(lambda x: {k[0]:x[k] for k in x})
    medikace_test["sukl_drugs_errors"] = medikace_test["sukl_search_drugs"].apply(lambda x: {k[0]:k[1] for k in x})
    medikace_test["sukl_search_drugs"] = medikace_test["sukl_search_drugs"].apply(lambda x: {k[0]:x[k] for k in x})
    
    medikace_test.to_csv("saved_search/medikace_test2.csv")

medikace_test.head(3)

,label,text,original_text,about,shortcuts,sukl_search_chemicals,sukl_search_drugs,sukl_chemicals_errors,sukl_drugs_errors
Unnamed: 0,,,,,,,,,
3848,medikace,tamoxifenu,tamoxifenu,NaN,NaN,"{'tamoxifenu': [('L02BA01', 'TAMOXIFEN', 2)]}","{'tamoxifenu': [('L02BA01', 'TAMOXIFEN', 1)]}",{'tamoxifenu': 1},{'tamoxifenu': 1}
2984,medikace,herceptinu,herceptinu,NaN,NaN,"{'herceptinu': [('A16AA07', 'METRELEPTIN', 2),...","{'herceptinu': [('L01FD01', 'TRASTUZUMAB', 1)]}",{'herceptinu': 4},{'herceptinu': 1}
4807,medikace,neoadjuvantní chemoterapie paclitaxel weekly,neoadjuvantní chemoterapie - paclitaxel weekly,NaN,NaN,"{'neoadjuvantní': [('B02BD', 'KOAGULAČNÍ FAKTO...","{'neoadjuvantní': [('A05AA01', 'KYSELINA CHENO...","{'neoadjuvantní': 7, 'chemoterapie': 1, 'pacli...","{'neoadjuvantní': 7, 'chemoterapie': 6, 'pacli..."


In [162]:
print_long_searches_len_dep(medikace_test["sukl_search_chemicals"], limit=10)
print()
print_long_searches_len_dep(medikace_test["sukl_search_drugs"], limit=10)

medikace_test["sukl_search_chemicals"] = medikace_test["sukl_search_chemicals"].apply(
    lambda x: drop_long_searches_len_dep(x, limit=25))
medikace_test["sukl_search_drugs"] = medikace_test["sukl_search_drugs"].apply(
    lambda x: drop_long_searches_len_dep(x, limit=40))

[[('weekly', 62, 10.333333333333334)], [('CBDCA', 61, 12.2)], [('režimem', 90, 12.857142857142858), ('AC', 1383, 691.5)], [('LYRICA', 112, 18.666666666666668)], [('vit', 62, 20.666666666666668), ('D', 3422, 3422.0)], [('3', 42, 42.0)], [('COMIRNATY', 516, 57.333333333333336)], [('PTX', 283, 94.33333333333333)], [('AC', 1383, 691.5), ('4x', 960, 480.0), ('st', 39, 19.5), ('p', 693, 693.0), ('NACT', 147, 36.75)], [('v', 1682, 1682.0), ('weekly', 62, 10.333333333333334)]]

[[('10', 25, 12.5)], [('Neo', 41, 13.666666666666666)], [('vit', 65, 21.666666666666668), ('D', 811, 811.0)], [('PTX', 100, 33.333333333333336)], [('3', 34, 34.0)], [('AC', 300, 150.0)], [('AC', 300, 150.0), ('st', 32, 16.0), ('p', 191, 191.0), ('NACT', 71, 17.75)], [('v', 719, 719.0)]]


In [163]:
medikace_test["sukl_chemicals_combine"] = medikace_test["sukl_search_chemicals"].apply(combine_preserve_count)
medikace_test[medikace_test["sukl_chemicals_combine"] != {}]["sukl_chemicals_combine"]

Series([], Name: sukl_chemicals_combine, dtype: object)

In [164]:
medikace_test["sukl_drugs_combine"] = medikace_test["sukl_search_drugs"].apply(combine_preserve_count)
medikace_test[medikace_test["sukl_drugs_combine"] != {}]["sukl_drugs_combine"]

Unnamed: 0
4807    {'neoadjuvantní chemoterapie': [('V06XX', 'POT...
3546    {'calcium vit': [('A12AX', 'VÁPNÍK, KOMBINACE ...
1703         {'orcal Neo': [('C08CA01', 'AMLODIPIN', 2)]}
Name: sukl_drugs_combine, dtype: object

In [165]:
print_long_searches(medikace_test["sukl_search_chemicals"], limit=5)
print()
print_long_searches(medikace_test["sukl_search_drugs"], limit=10)

medikace_test["sukl_search_chemicals"] = medikace_test["sukl_search_chemicals"].apply(lambda x: drop_long_searches(x, limit=10))
medikace_test["sukl_search_drugs"] = medikace_test["sukl_search_drugs"].apply(lambda x: drop_long_searches(x, limit=15))

[[('adjuvanci', 6), ('podáme', 6), ('weekly', 62)], [('ranisan', 7)], [('kalnormin', 7)], [('dexametason', 16)], [('utrogestan', 18)], [('neoadjvu', 23), ('režimem', 90)], [('herceptinu', 26)], [('detralex', 28)], [('cipralex', 34)], [('st', 39)], [('orcal', 47), ('Neo', 27)], [('CBDCA', 61)], [('vit', 62)], [('flamigel', 63)], [('terapii', 64)], [('neoadjuvantní', 66), ('weekly', 62)], [('ryzodeg', 66)], [('LYRICA', 112)]]

[[('stp1xAC', 11)], [('flamigel', 11)], [('ranisan', 12)], [('podáme', 16), ('weekly', 40)], [('antiemetika', 19)], [('CBDCA', 21)], [('chemoterapie', 22), ('weekly', 40)], [('10', 25), ('mg', 19)], [('neoadjvu', 26), ('režimem', 43)], [('st', 32), ('NACT', 71)], [('3', 34), ('mg', 19)], [('Neo', 41)], [('vit', 65)], [('PTX', 100)]]


In [166]:
medikace_test["sukl_mathematical_explanation"] = "N/A"
for index in medikace_test.index:
    union_chem = drop_empty_searches(medikace_test["sukl_search_chemicals"][index] | medikace_test["sukl_chemicals_combine"][index])
    union_drugs = drop_empty_searches(medikace_test["sukl_search_drugs"][index] | medikace_test["sukl_drugs_combine"][index])

    medikace_test.at[index, "sukl_mathematical_explanation"] = combine_2_searched_databaze(union_chem, union_drugs,
                     medikace_test["sukl_chemicals_errors"][index],
                     medikace_test["sukl_drugs_errors"][index])

random.seed(10)
medikace_test["sukl_mathematical_explanation"] = (
    medikace_test["sukl_mathematical_explanation"].apply(
        lambda x: {k: sorted(x[k], key=lambda y: prioritize_sukl(y, x[k]), reverse=True)[0]
                   for k in x if len(x[k]) != 0}))

medikace_test.head(3)

,label,text,original_text,about,shortcuts,sukl_search_chemicals,sukl_search_drugs,sukl_chemicals_errors,sukl_drugs_errors,sukl_chemicals_combine,sukl_drugs_combine,sukl_mathematical_explanation
Unnamed: 0,,,,,,,,,,,,
3848,medikace,tamoxifenu,tamoxifenu,NaN,NaN,"{'tamoxifenu': [('L02BA01', 'TAMOXIFEN', 2)]}","{'tamoxifenu': [('L02BA01', 'TAMOXIFEN', 1)]}",{'tamoxifenu': 1},{'tamoxifenu': 1},{},{},"{'tamoxifenu': ('L02BA01', 'TAMOXIFEN', 2)}"
2984,medikace,herceptinu,herceptinu,NaN,NaN,{'herceptinu': []},"{'herceptinu': [('L01FD01', 'TRASTUZUMAB', 1)]}",{'herceptinu': 4},{'herceptinu': 1},{},{},"{'herceptinu': ('L01FD01', 'TRASTUZUMAB', 1)}"
4807,medikace,neoadjuvantní chemoterapie paclitaxel weekly,neoadjuvantní chemoterapie - paclitaxel weekly,NaN,NaN,"{'neoadjuvantní': [], 'chemoterapie': [('D06',...","{'neoadjuvantní': [('A05AA01', 'KYSELINA CHENO...","{'neoadjuvantní': 7, 'chemoterapie': 1, 'pacli...","{'neoadjuvantní': 7, 'chemoterapie': 6, 'pacli...",{},"{'neoadjuvantní chemoterapie': [('V06XX', 'POT...","{'neoadjuvantní chemoterapie': ('V06XX', 'POTR..."


#### Results

In [167]:
medikace_test["asign"] = "N/A"

i = 0
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Partially" #2 - Wrong neoadjuvantní terapie - not a medicine, drug was right (better filter)
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong" #3 - Wrong  - word taxol is in chemicals BETAXOLOL, but also it is drug TAXOL which we should find
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Partially"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Partially"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"

# print(medikace_test["original_text"].iloc[i])
# print(medikace_test["sukl_mathematical_explanation"].iloc[i])

In [168]:
medikace_test["asign"].value_counts()

asign
Right        21
Wrong        11
Partially     3
Name: count, dtype: int64

Right 60 % | Partially 69 %

In [169]:
print("RIGHT")
for i in medikace_test[medikace_test["asign"] == "Right"].sample(10, random_state=32).index:
    x = medikace_test["sukl_chemicals_errors"][i]
    y = medikace_test["sukl_search_chemicals"][i]
    temp = {k : y[k] for k in y if x[k] != 0}
    if len(temp) != 0:
        print(temp)
    temp2 = {k: (x[k], len(k), round(x[k]/len(k), 3)) for k in x if x[k] != 0}
    if len(temp2) != 0:
        print(temp2)
    if len(temp) + len(temp2) != 0:
        print()

print("WRONG")
for i in medikace_test[medikace_test["asign"] != "Right"].sample(10, random_state=32).index:
    x = medikace_test["sukl_chemicals_errors"][i]
    y = medikace_test["sukl_search_chemicals"][i]
    temp = {k : y[k] for k in y if x[k] != 0}
    if len(temp) != 0:
        print(temp)
    temp2 = {k: (x[k], len(k), round(x[k]/len(k), 3)) for k in x if x[k] != 0}
    if len(temp2) != 0:
        print(temp2)
    if len(temp) + len(temp2) != 0:
        print()

RIGHT
{'LYRICA': []}
{'LYRICA': (2, 6, 0.333)}

{'tritace': [('A16AX13', 'URIDIN-TRIACETÁT', 2)]}
{'tritace': (1, 7, 0.143)}

{'berodual': [('B06AC06', 'BEROTRALSTAT', 2), ('L04AC12', 'BRODALUMAB', 2)]}
{'berodual': (2, 8, 0.25)}

{'ondasetronem': [('A04AA01', 'ONDANSETRON', 2)]}
{'ondasetronem': (3, 12, 0.25)}

{'dexametason': []}
{'dexametason': (1, 11, 0.091)}

{'trastuzumabu': [('L01FD01', 'TRASTUZUMAB', 2), ('L01FD03', 'TRASTUZUMAB EMTANSIN', 2), ('L01FD04', 'TRASTUZUMAB DERUXTEKAN', 2), ('L01FD05', 'TRASTUZUMAB DUOKARMAZIN', 2), ('L01FY01', 'PERTUZUMAB A TRASTUZUMAB', 2)]}
{'trastuzumabu': (1, 12, 0.083)}

{'tamoxifenu': [('L02BA01', 'TAMOXIFEN', 2)]}
{'tamoxifenu': (1, 10, 0.1)}

{'COMIRNATY': []}
{'COMIRNATY': (3, 9, 0.333)}

{'utrogestan': []}
{'utrogestan': (3, 10, 0.3)}

{'orcal': []}
{'orcal': (1, 5, 0.2)}

WRONG
{'neoadjuvantní': [], 'chemoterapie': [('D06', 'ANTIBIOTIKA A CHEMOTERAPEUTIKA PRO POUŽITÍ V DERMATOLOGII', 1), ('D06B', 'CHEMOTERAPEUTIKA PRO LOKÁLNÍ APLIKACI', 1

Limit 0.25 will be enough

In [170]:
print("RIGHT")
for i in medikace_test[medikace_test["asign"] == "Right"].sample(10, random_state=13).index:
    x = medikace_test["sukl_drugs_errors"][i]
    y = medikace_test["sukl_search_drugs"][i]
    temp = {k : y[k] for k in y if x[k] != 0}
    if len(temp) != 0:
        print(temp)
    temp2 = {k: (x[k], len(k), round(x[k]/len(k), 3)) for k in x if x[k] != 0}
    if len(temp2) != 0:
        print(temp2)
    if len(temp) + len(temp2) != 0:
        print()

print("WRONG")
for i in medikace_test[medikace_test["asign"] != "Right"].sample(10, random_state=13).index:
    x = medikace_test["sukl_drugs_errors"][i]
    y = medikace_test["sukl_search_drugs"][i]
    temp = {k : y[k] for k in y if x[k] != 0}
    if len(temp) != 0:
        print(temp)
    temp2 = {k: (x[k], len(k), round(x[k]/len(k), 3)) for k in x if x[k] != 0}
    if len(temp2) != 0:
        print(temp2)
    if len(temp) + len(temp2) != 0:
        print()

RIGHT
{'ondasetronem': [('A04AA01', 'ONDANSETRON', 5)]}
{'ondasetronem': (3, 12, 0.25)}

{'tamoxifenu': [('L02BA01', 'TAMOXIFEN', 1)]}
{'tamoxifenu': (1, 10, 0.1)}

{'herceptinu': [('L01FD01', 'TRASTUZUMAB', 1)]}
{'herceptinu': (1, 10, 0.1)}

{'trastuzumabu': [('A12BA01', 'CHLORID DRASELNÝ', 1), ('B05XA01', 'CHLORID DRASELNÝ', 1), ('B05XA03', 'CHLORID SODNÝ', 2), ('B05XA07', 'CHLORID VÁPENATÝ', 2), ('C05BA53', 'HEPARIN, KOMBINACE', 1), ('N02AX02', 'TRAMADOL', 1), ('V07AB', 'ROZPOUŠTĚDLA A ŘEDIDLA, VČETNĚ IRIGAČNÍCH ROZTOKŮ', 2), ('V11', 'FYTOFARMAKA A ŽIVOČIŠNÉ PRODUKTY (ČESKÁ ATC SKUPINA)', 1), ('V12', 'HOMEOPATIKA (ČESKÁ ATC SKUPINA)', 1)]}
{'trastuzumabu': (6, 12, 0.5)}

WRONG
{'bisulepin': [('N05AL01', 'SULPIRID', 1)]}
{'bisulepin': (3, 9, 0.333)}

{'NACT': []}
{'NACT': (1, 4, 0.25)}

{'neoadjvu': [], 'režimem': []}
{'neoadjvu': (4, 8, 0.5), 'režimem': (3, 7, 0.429)}

{'CBDCA': []}
{'CBDCA': (2, 5, 0.4)}

{'PTX': []}
{'PTX': (1, 3, 0.333)}

{'antiHER2': [('J06AA03', 'SÉRUM PROTI HA

limit 0.25 will be enough

### Label medikace 03 - Extended SUKL

#### New chemicals database

In [171]:
substances_sukl_code = pd.read_csv("databaze/DLP20240229/dlp_latky.csv", encoding="cp1250", delimiter=";"
           )[["KOD_LATKY", "NAZEV_INN", "NAZEV_EN", "NAZEV"]]
content_drugs = pd.read_csv("databaze/DLP20240229/dlp_slozeni.csv", encoding="cp1250", delimiter=";")
products_whole = pd.read_csv("databaze/DLP20240229/dlp_lecivepripravky.csv",
                       encoding="cp1250", delimiter=";")[["KOD_SUKL", "NAZEV", "SILA", "FORMA", "ATC_WHO", "NAZEV_REG"]]

drug_code_to_substance_code = content_drugs[["KOD_SUKL", "KOD_LATKY"]].dropna()
drug_code_to_substance_code["KOD_LATKY"] = drug_code_to_substance_code["KOD_LATKY"].apply(lambda x: round(x))
drug_code_to_substance_code.index = drug_code_to_substance_code["KOD_SUKL"]
drug_code_to_substance_code = drug_code_to_substance_code[["KOD_LATKY"]]

drug_code_to_atc = products_whole.copy()
drug_code_to_atc.index = drug_code_to_atc["KOD_SUKL"]
drug_code_to_atc = drug_code_to_atc.drop(["KOD_SUKL"], axis=1)

substance_code_to_atc_duplicated = drug_code_to_substance_code.join(drug_code_to_atc, how="left")[["KOD_LATKY", "NAZEV", "ATC_WHO"]]
substance_code_to_atc_duplicated["KOD_SUKL"] = substance_code_to_atc_duplicated.index
substance_code_to_atc_duplicated.index = substance_code_to_atc_duplicated["KOD_LATKY"]
substance_code_to_atc_duplicated = substance_code_to_atc_duplicated.drop(["KOD_LATKY"], axis=1)

substance_code_to_atc = substance_code_to_atc_duplicated.drop([-1, 0]).groupby(["KOD_LATKY"], group_keys=True).apply(
    lambda x: x["ATC_WHO"].value_counts().sort_values(ascending=False).index[0])

substance_code_to_atc.head(3)

KOD_LATKY
1        V11
2    M02AA10
8    V10XA01
dtype: object

In [172]:
synonyms = pd.read_csv("databaze/DLP20240229/dlp_synonyma.csv", encoding="cp1250", delimiter=";")
temp = synonyms["KOD_LATKY"].apply(lambda x: substance_code_to_atc[x] if x in substance_code_to_atc else "N/A")
synonyms_linked = synonyms[temp != "N/A"].copy()
synonyms_linked["ATC"] = temp[temp != "N/A"]
synonyms_linked = synonyms_linked[~(synonyms_linked[["NAZEV", "ATC"]].duplicated())]

name_to_atc = pd.DataFrame()
name_to_atc["NAME"] = "N/A"
name_to_atc["ATC"] = "N/A"

length = len(name_to_atc)
temp = pd.DataFrame()
temp["NAME"] = synonyms_linked["NAZEV"].reset_index(drop=True)
temp["ATC"] = synonyms_linked["ATC"].reset_index(drop=True)
temp.index = temp.index + length
name_to_atc = pd.concat([name_to_atc, temp])

length = len(name_to_atc)
temp = pd.DataFrame()
temp["NAME"] = substances_with_ATC_concat["NAZEV"].reset_index(drop=True)
temp["ATC"] = substances_with_ATC_concat["ATC"].reset_index(drop=True)
temp.index = temp.index + length
name_to_atc = pd.concat([name_to_atc, temp])

name_to_atc = name_to_atc.dropna()
name_to_atc = name_to_atc.rename(columns={"NAME": "NAZEV"})
name_to_atc.head(3)

,NAZEV,ATC
0,"ARTEMISIA ABSINTHIUM L., HERBA",V11
1,HERBA ARTEMISIAE ABSINTHII,V11
2,ABSINTHII SUMMITAS,V11


#### Linking

In [173]:
medikace_test = improved_medikace.sample(35, random_state=21)
medikace_test = make_shortcuts_table(medikace_test)
medikace_test.head(3)

,label,text,original_text,shortcuts
3848,medikace,tamoxifenu,tamoxifenu,
2984,medikace,herceptinu,herceptinu,
4807,medikace,neoadjuvantní chemoterapie paclitaxel weekly,neoadjuvantní chemoterapie - paclitaxel weekly,


In [174]:
if os.path.isfile("saved_search/medikace_test3.csv"):
    medikace_test = sukl_search_all_read_csv("saved_search/medikace_test3.csv")
    
else:
    medikace_test["sukl_search_chemicals"] = medikace_test["text"].apply(
        lambda x: sukl_find_chemicals_search(x, output_errors=True, database=name_to_atc, limit=0.15))
    medikace_test["sukl_search_chemicals"] = medikace_test.apply(
        lambda x: x.sukl_search_chemicals | 
        (sukl_find_chemicals_shortcuts_search(x.shortcuts, output_errors=True, database=name_to_atc, limit=0.15)
                 if x.shortcuts != "" else {}), axis=1)

    medikace_test["sukl_search_drugs"] = medikace_test["text"].apply(
        lambda x: sukl_find_drugs_search(x, output_errors=True, limit=0.25))
    medikace_test["sukl_search_drugs"] = medikace_test.apply(lambda x: x.sukl_search_drugs |
                (sukl_find_drugs_shortcuts_search(x.shortcuts ,output_errors=True, limit=0.25)
                         if x.shortcuts != "" else {}), axis=1)

    medikace_test["sukl_chemicals_errors"] = medikace_test["sukl_search_chemicals"].apply(lambda x: {k[0]:k[1] for k in x})
    medikace_test["sukl_search_chemicals"] = medikace_test["sukl_search_chemicals"].apply(lambda x: {k[0]:x[k] for k in x})
    medikace_test["sukl_drugs_errors"] = medikace_test["sukl_search_drugs"].apply(lambda x: {k[0]:k[1] for k in x})
    medikace_test["sukl_search_drugs"] = medikace_test["sukl_search_drugs"].apply(lambda x: {k[0]:x[k] for k in x})
    
    medikace_test.to_csv("saved_search/medikace_test3.csv")

medikace_test.head(3)

,label,text,original_text,about,shortcuts,sukl_search_chemicals,sukl_search_drugs,sukl_chemicals_errors,sukl_drugs_errors
Unnamed: 0,,,,,,,,,
3848,medikace,tamoxifenu,tamoxifenu,NaN,NaN,"{'tamoxifenu': [('L02BA01', 'TAMOXIFEN', 3)]}","{'tamoxifenu': [('L02BA01', 'TAMOXIFEN', 1)]}",{'tamoxifenu': 0},{'tamoxifenu': 1}
2984,medikace,herceptinu,herceptinu,NaN,NaN,{'herceptinu': []},"{'herceptinu': [('L01FD01', 'TRASTUZUMAB', 1)]}",{'herceptinu': 2},{'herceptinu': 1}
4807,medikace,neoadjuvantní chemoterapie paclitaxel weekly,neoadjuvantní chemoterapie - paclitaxel weekly,NaN,NaN,"{'neoadjuvantní': [], 'chemoterapie': [('D06',...","{'neoadjuvantní': [], 'chemoterapie': [], 'pac...","{'neoadjuvantní': 2, 'chemoterapie': 1, 'pacli...","{'neoadjuvantní': 4, 'chemoterapie': 4, 'pacli..."


In [175]:
print_long_searches_len_dep(medikace_test["sukl_search_chemicals"], limit=10)
print()
print_long_searches_len_dep(medikace_test["sukl_search_drugs"], limit=10)

medikace_test["sukl_search_chemicals"] = medikace_test["sukl_search_chemicals"].apply(
    lambda x: drop_long_searches_len_dep(x, limit=25))
medikace_test["sukl_search_drugs"] = medikace_test["sukl_search_drugs"].apply(
    lambda x: drop_long_searches_len_dep(x, limit=40))

[[('orcal', 123, 24.6), ('Neo', 52, 17.333333333333332)], [('vit', 160, 53.333333333333336), ('D', 4333, 4333.0)], [('PTX', 657, 219.0)], [('10', 808, 404.0), ('mg', 70, 35.0)], [('AC', 2211, 1105.5)], [('AC', 2211, 1105.5), ('st', 163, 81.5), ('p', 1207, 1207.0), ('NACT', 562, 140.5)], [('3', 1588, 1588.0), ('mg', 70, 35.0)], [('v', 2446, 2446.0)]]

[[('10', 25, 12.5)], [('Neo', 41, 13.666666666666666)], [('vit', 65, 21.666666666666668), ('D', 811, 811.0)], [('PTX', 100, 33.333333333333336)], [('3', 34, 34.0)], [('AC', 300, 150.0)], [('AC', 300, 150.0), ('st', 32, 16.0), ('p', 191, 191.0), ('NACT', 71, 17.75)], [('v', 719, 719.0)]]


In [176]:
medikace_test["sukl_chemicals_combine"] = medikace_test["sukl_search_chemicals"].apply(combine_preserve_count)
medikace_test[medikace_test["sukl_chemicals_combine"] != {}]["sukl_chemicals_combine"]

Unnamed: 0
1703    {'orcal Neo': [('G01AA51', 'NYSTATIN, KOMBINAC...
Name: sukl_chemicals_combine, dtype: object

In [177]:
medikace_test["sukl_drugs_combine"] = medikace_test["sukl_search_drugs"].apply(combine_preserve_count)
medikace_test[medikace_test["sukl_drugs_combine"] != {}]["sukl_drugs_combine"]

Unnamed: 0
3546    {'calcium vit': [('A12AX', 'VÁPNÍK, KOMBINACE ...
1703         {'orcal Neo': [('C08CA01', 'AMLODIPIN', 2)]}
Name: sukl_drugs_combine, dtype: object

In [178]:
print_long_searches(medikace_test["sukl_search_chemicals"], limit=5)
print()
print_long_searches(medikace_test["sukl_search_drugs"], limit=10)

medikace_test["sukl_search_chemicals"] = medikace_test["sukl_search_chemicals"].apply(lambda x: drop_long_searches(x, limit=10))
medikace_test["sukl_search_drugs"] = medikace_test["sukl_search_drugs"].apply(lambda x: drop_long_searches(x, limit=15))

[[('adjuvanci', 6)], [('ryzodeg', 10)], [('antiHER2', 12), ('terapii', 64)], [('lexaurin', 15)], [('tritace', 25)], [('CHT', 29)], [('stilnox', 30)], [('ranisan', 64)], [('orcal', 123), ('Neo', 52)]]

[[('flamigel', 11)], [('ranisan', 12)], [('podáme', 16)], [('CBDCA', 21)], [('10', 25), ('mg', 19)], [('st', 32), ('NACT', 71)], [('3', 34), ('mg', 19)], [('Neo', 41)], [('vit', 65)], [('PTX', 100)]]


In [179]:
medikace_test["sukl_mathematical_explanation"] = "N/A"
for index in medikace_test.index:
    union_chem = drop_empty_searches(medikace_test["sukl_search_chemicals"][index] | medikace_test["sukl_chemicals_combine"][index])
    union_drugs = drop_empty_searches(medikace_test["sukl_search_drugs"][index] | medikace_test["sukl_drugs_combine"][index])

    medikace_test.at[index, "sukl_mathematical_explanation"] = combine_2_searched_databaze(union_chem, union_drugs,
                     medikace_test["sukl_chemicals_errors"][index],
                     medikace_test["sukl_drugs_errors"][index])

random.seed(10)
medikace_test["sukl_mathematical_explanation"] = (
    medikace_test["sukl_mathematical_explanation"].apply(
        lambda x: {k: sorted(x[k], key=lambda y: prioritize_sukl(y, x[k]), reverse=True)[0]
                   for k in x if len(x[k]) != 0}))

medikace_test.head(3)

,label,text,original_text,about,shortcuts,sukl_search_chemicals,sukl_search_drugs,sukl_chemicals_errors,sukl_drugs_errors,sukl_chemicals_combine,sukl_drugs_combine,sukl_mathematical_explanation
Unnamed: 0,,,,,,,,,,,,
3848,medikace,tamoxifenu,tamoxifenu,NaN,NaN,"{'tamoxifenu': [('L02BA01', 'TAMOXIFEN', 3)]}","{'tamoxifenu': [('L02BA01', 'TAMOXIFEN', 1)]}",{'tamoxifenu': 0},{'tamoxifenu': 1},{},{},"{'tamoxifenu': ('L02BA01', 'TAMOXIFEN', 3)}"
2984,medikace,herceptinu,herceptinu,NaN,NaN,{'herceptinu': []},"{'herceptinu': [('L01FD01', 'TRASTUZUMAB', 1)]}",{'herceptinu': 2},{'herceptinu': 1},{},{},"{'herceptinu': ('L01FD01', 'TRASTUZUMAB', 1)}"
4807,medikace,neoadjuvantní chemoterapie paclitaxel weekly,neoadjuvantní chemoterapie - paclitaxel weekly,NaN,NaN,"{'neoadjuvantní': [], 'chemoterapie': [('D06',...","{'neoadjuvantní': [], 'chemoterapie': [], 'pac...","{'neoadjuvantní': 2, 'chemoterapie': 1, 'pacli...","{'neoadjuvantní': 4, 'chemoterapie': 4, 'pacli...",{},{},"{'paclitaxel': ('L01CD01', 'PAKLITAXEL', 2), '..."


#### Result

In [180]:
medikace_test["asign"] = "N/A"

i = 0
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Empty"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Empty"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Partially"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"
i += 1
j = medikace_test.index[i]
medikace_test.at[j, "asign"] = "Right"

# print(medikace_test["original_text"].iloc[i])
# print(medikace_test["sukl_mathematical_explanation"].iloc[i])

In [181]:
medikace_test["asign"].value_counts()

asign
Right        27
Wrong         5
Empty         2
Partially     1
Name: count, dtype: int64

Right: 77 % | Partially 80 % | 6 % Empty

### Label NE medikace

In [182]:
improved_ne_medikace = data_improved_punctuation[(data_improved_punctuation["label"] == "NE medikace")].copy()
print("Number of 'NE medikace' label:", len(improved_ne_medikace))

ne_medikace_test = improved_ne_medikace.sample(frac=1, random_state=235)
ne_medikace_test = make_negation_table(ne_medikace_test)
ne_medikace_test = make_shortcuts_table(ne_medikace_test)
ne_medikace_test.head(3)

Number of 'NE medikace' label: 16


,label,text,original_text,neg,shortcuts
53,NE medikace,antikoncepce,antikoncepce: 0,True,
5463,NE medikace,HRT,bez HRT,True,
2463,NE medikace,hormonální léčba HRT dříve,hormonální léčba: HRT dříve,False,


In [183]:
if os.path.isfile("saved_search/ne_medikace_test.csv"):
    ne_medikace_test = sukl_search_all_read_csv("saved_search/ne_medikace_test.csv")
    
else:
    ne_medikace_test["sukl_search_chemicals"] = ne_medikace_test["text"].apply(
        lambda x: sukl_find_chemicals_search(x, output_errors=True, database=name_to_atc, limit=0.15))
    ne_medikace_test["sukl_search_chemicals"] = ne_medikace_test.apply(
        lambda x: x.sukl_search_chemicals | 
        (sukl_find_chemicals_shortcuts_search(x.shortcuts, output_errors=True, database=name_to_atc, limit=0.15)
                 if x.shortcuts != "" else {}), axis=1)

    ne_medikace_test["sukl_search_drugs"] = ne_medikace_test["text"].apply(
        lambda x: sukl_find_drugs_search(x, output_errors=True, limit=0.25))
    ne_medikace_test["sukl_search_drugs"] = ne_medikace_test.apply(lambda x: x.sukl_search_drugs |
                (sukl_find_drugs_shortcuts_search(x.shortcuts ,output_errors=True, limit=0.25)
                         if x.shortcuts != "" else {}), axis=1)
    
    ne_medikace_test["sukl_chemicals_errors"] = ne_medikace_test["sukl_search_chemicals"].apply(lambda x: {k[0]:k[1] for k in x})
    ne_medikace_test["sukl_search_chemicals"] = ne_medikace_test["sukl_search_chemicals"].apply(lambda x: {k[0]:x[k] for k in x})
    ne_medikace_test["sukl_drugs_errors"] = ne_medikace_test["sukl_search_drugs"].apply(lambda x: {k[0]:k[1] for k in x})
    ne_medikace_test["sukl_search_drugs"] = ne_medikace_test["sukl_search_drugs"].apply(lambda x: {k[0]:x[k] for k in x})
    ne_medikace_test.to_csv("saved_search/ne_medikace_test.csv")

ne_medikace_test.head(3)

,label,text,original_text,neg,shortcuts,sukl_search_chemicals,sukl_search_drugs,sukl_chemicals_errors,sukl_drugs_errors
Unnamed: 0,,,,,,,,,
53,NE medikace,antikoncepce,antikoncepce: 0,True,NaN,{'antikoncepce': []},{'antikoncepce': []},{'antikoncepce': 2},{'antikoncepce': 4}
5463,NE medikace,HRT,bez HRT,True,NaN,"{'HRT': [('A01A', 'STOMATOLOGICKÉ PŘÍPRAVKY', ...","{'HRT': [('A01AD02', 'BENZYDAMIN', 2), ('A02AD...",{'HRT': 1},{'HRT': 1}
2463,NE medikace,hormonální léčba HRT dříve,hormonální léčba: HRT dříve,False,NaN,"{'hormonální': [('G03A', 'HORMONÁLNÍ KONTRACEP...","{'hormonální': [], 'léčba': [('A11DA01', 'THIA...","{'hormonální': 0, 'léčba': 0, 'HRT': 1, 'dříve...","{'hormonální': 3, 'léčba': 2, 'HRT': 1, 'dříve..."


In [184]:
print_long_searches_len_dep(ne_medikace_test["sukl_search_chemicals"], limit=5)
print()
print_long_searches_len_dep(ne_medikace_test["sukl_search_drugs"], limit=5)

ne_medikace_test["sukl_search_chemicals"] = ne_medikace_test["sukl_search_chemicals"].apply(
    lambda x: drop_long_searches_len_dep(x, limit=25))
ne_medikace_test["sukl_search_drugs"] = ne_medikace_test["sukl_search_drugs"].apply(
    lambda x: drop_long_searches_len_dep(x, limit=40))

[[('zometa', 54, 9.0)], [('akt', 309, 103.0), ('již', 342, 114.0)], [('HRT', 1244, 414.6666666666667)], [('HRT', 1244, 414.6666666666667)], [('HRT', 1244, 414.6666666666667)]]

[[('léčbu', 35, 7.0)], [('léčba', 36, 7.2), ('HRT', 170, 56.666666666666664)], [('léčba', 36, 7.2)], [('léčba', 36, 7.2)], [('FA', 95, 47.5)], [('HRT', 170, 56.666666666666664)], [('HRT', 170, 56.666666666666664)]]


In [185]:
ne_medikace_test["sukl_chemicals_combine"] = ne_medikace_test["sukl_search_chemicals"].apply(combine_preserve_count)
ne_medikace_test[ne_medikace_test["sukl_chemicals_combine"] != {}]["sukl_chemicals_combine"]

Series([], Name: sukl_chemicals_combine, dtype: object)

In [186]:
ne_medikace_test["sukl_drugs_combine"] = ne_medikace_test["sukl_search_drugs"].apply(combine_preserve_count)
ne_medikace_test[ne_medikace_test["sukl_drugs_combine"] != {}]["sukl_drugs_combine"]

Series([], Name: sukl_drugs_combine, dtype: object)

In [187]:
print_long_searches(ne_medikace_test["sukl_search_chemicals"], limit=5)
print()
print_long_searches(ne_medikace_test["sukl_search_drugs"], limit=10)

ne_medikace_test["sukl_search_chemicals"] = ne_medikace_test["sukl_search_chemicals"].apply(lambda x: drop_long_searches(x, limit=10))
ne_medikace_test["sukl_search_drugs"] = ne_medikace_test["sukl_search_drugs"].apply(lambda x: drop_long_searches(x, limit=15))

[[('vysazeno', 12)], [('léčbu', 13)], [('tramal', 16)], [('zometa', 54)]]

[[('snesla', 18)], [('léčbu', 35)], [('léčba', 36)], [('léčba', 36)], [('léčba', 36)]]


In [188]:
ne_medikace_test["sukl_mathematical_explanation"] = "N/A"
for index in ne_medikace_test.index:
    union_chem = drop_empty_searches(ne_medikace_test["sukl_search_chemicals"][index] | ne_medikace_test["sukl_chemicals_combine"][index])
    union_drugs = drop_empty_searches(ne_medikace_test["sukl_search_drugs"][index] | ne_medikace_test["sukl_drugs_combine"][index])

    ne_medikace_test.at[index, "sukl_mathematical_explanation"] = combine_2_searched_databaze(union_chem, union_drugs,
                     ne_medikace_test["sukl_chemicals_errors"][index],
                     ne_medikace_test["sukl_drugs_errors"][index])

random.seed(23)
ne_medikace_test["sukl_mathematical_explanation"] = (
    ne_medikace_test["sukl_mathematical_explanation"].apply(
        lambda x: {k: sorted(x[k], key=lambda y: prioritize_sukl(y, x[k]), reverse=True)[0]
                   for k in x if len(x[k]) != 0}))

ne_medikace_test.head(3)

,label,text,original_text,neg,shortcuts,sukl_search_chemicals,sukl_search_drugs,sukl_chemicals_errors,sukl_drugs_errors,sukl_chemicals_combine,sukl_drugs_combine,sukl_mathematical_explanation
Unnamed: 0,,,,,,,,,,,,
53,NE medikace,antikoncepce,antikoncepce: 0,True,NaN,{'antikoncepce': []},{'antikoncepce': []},{'antikoncepce': 2},{'antikoncepce': 4},{},{},{}
5463,NE medikace,HRT,bez HRT,True,NaN,{'HRT': []},{'HRT': []},{'HRT': 1},{'HRT': 1},{},{},{}
2463,NE medikace,hormonální léčba HRT dříve,hormonální léčba: HRT dříve,False,NaN,"{'hormonální': [('G03A', 'HORMONÁLNÍ KONTRACEP...","{'hormonální': [], 'léčba': [], 'HRT': [], 'dř...","{'hormonální': 0, 'léčba': 0, 'HRT': 1, 'dříve...","{'hormonální': 3, 'léčba': 2, 'HRT': 1, 'dříve...",{},{},"{'léčba': ('V10B', 'PALIATIVNÍ LÉČBA BOLESTI (..."


#### Result

In [189]:
ne_medikace_test["asign"] = "N/A"

i = 0
j = ne_medikace_test.index[i]
ne_medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_medikace_test.index[i]
ne_medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_medikace_test.index[i]
ne_medikace_test.at[j, "asign"] = "Partially"
i += 1
j = ne_medikace_test.index[i]
ne_medikace_test.at[j, "asign"] = "Partially"
i += 1
j = ne_medikace_test.index[i]
ne_medikace_test.at[j, "asign"] = "Right"
i += 1
j = ne_medikace_test.index[i]
ne_medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_medikace_test.index[i]
ne_medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_medikace_test.index[i]
ne_medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_medikace_test.index[i]
ne_medikace_test.at[j, "asign"] = "Right"
i += 1
j = ne_medikace_test.index[i]
ne_medikace_test.at[j, "asign"] = "Wrong"
i += 1
j = ne_medikace_test.index[i]
ne_medikace_test.at[j, "asign"] = "Partially"
i += 1
j = ne_medikace_test.index[i]
ne_medikace_test.at[j, "asign"] = "Right"
i += 1
j = ne_medikace_test.index[i]
ne_medikace_test.at[j, "asign"] = "Right"
i += 1
j = ne_medikace_test.index[i]
ne_medikace_test.at[j, "asign"] = "Right"
i += 1
j = ne_medikace_test.index[i]
ne_medikace_test.at[j, "asign"] = "Empty"
i += 1
j = ne_medikace_test.index[i]
ne_medikace_test.at[j, "asign"] = "Right"


# print(ne_medikace_test["original_text"].iloc[i])
# print(ne_medikace_test["sukl_mathematical_explanation"].iloc[i])

# print()
# print(ne_medikace_test["sukl_search_drugs"].iloc[i])
# print(ne_medikace_test["sukl_drugs_errors"].iloc[i])
# print(ne_medikace_test["sukl_search_chemicals"].iloc[i])
# print(ne_medikace_test["sukl_chemicals_errors"].iloc[i])

In [190]:
ne_medikace_test["asign"].value_counts()

asign
Wrong        6
Right        6
Partially    3
Empty        1
Name: count, dtype: int64

In [191]:
ne_medikace_test.apply(lambda x: print(f"{x.text}\n{x.sukl_mathematical_explanation}\n{x.asign}\n{x.sukl_search_chemicals}\n{x.sukl_search_drugs}\n"), axis=1)
None

antikoncepce
{}
Wrong
{'antikoncepce': []}
{'antikoncepce': []}

HRT
{}
Wrong
{'HRT': []}
{'HRT': []}

hormonální léčba HRT dříve
{'léčba': ('V10B', 'PALIATIVNÍ LÉČBA BOLESTI (LÁTKY VYCHYTÁVANÉ V KOSTECH)', 1), 'hormonální': ('G03A', 'HORMONÁLNÍ KONTRACEPTIVA K SYSTÉMOVÉ APLIKACI', 1), 'dříve': ('J07BX02', 'VAKCÍNY PROTI EBOLE', 1)}
Partially
{'hormonální': [('G03A', 'HORMONÁLNÍ KONTRACEPTIVA K SYSTÉMOVÉ APLIKACI', 1), ('H', 'SYSTÉMOVÁ HORMONÁLNÍ LÉČIVA KROMĚ POHLAVNÍCH HORMONŮ A INSULINŮ', 1), ('L02', 'HORMONÁLNÍ LÉČIVA POUŽÍVANÁ V ONKOLOGII', 1)], 'léčba': [('V10B', 'PALIATIVNÍ LÉČBA BOLESTI (LÁTKY VYCHYTÁVANÉ V KOSTECH)', 1)], 'HRT': [], 'dříve': [('J07BX02', 'VAKCÍNY PROTI EBOLE', 1)]}
{'hormonální': [], 'léčba': [], 'HRT': [], 'dříve': [('B05XA31', 'ELEKTROLYTY V KOMBINACI S JINÝMI LÉČIVY', 1), ('C05CA53', 'DIOSMIN, KOMBINACE', 2), ('G03AA18', 'DROSPIRENON A ESTETROL', 1), ('G03DB08', 'DIENOGEST', 1), ('L01BA04', 'PEMETREXED', 1), ('N01AH03', 'SUFENTANIL', 1), ('N07CA52', 'CINARIZ

Right: 37.5 % | Partially  56 % | Empty 6 %

- Lots of shortcuts
- Lots of wrongly labeled data.

## Whole search
Whole search by label

In [192]:
def whole_way(table, index, database, lenght_dep_limit, drop_limit, output_exp=False):
    # Search
    search = {}
    if table["text"][index] != "":
        search.update(search_words_mistake_bottom(table["text"][index],
                                              lambda x: medvik_combined_without_descr_search(x, database), 
                                              lambda x, y: medvik_mistakes_search(x, y, database=database),
                                              limit=0.25))
    if table["shortcuts"][index] != "":
        search.update(search_words_mistake_bottom(table["shortcuts"][index],
                                              lambda x: shortcuts_medvik_without_descr_search(x, database), 
                                              lambda x, y: shortcuts_medvik_mistakes_without_descr_search(
                                                  x, y, database=database),
                                              limit=0.25))

    # Drop & Combine
    search = drop_long_searches_len_dep(search, lenght_dep_limit)
    combined_search = combine_searches(search)
    search = drop_long_searches(search, drop_limit)
    search = drop_empty_searches(search)

    if output_exp:
        table.at[index, "search"] = search
        table.at[index, "combined"] = combined_search

    # Choose best Match
    result = {}
    combined_level = -np.infty
    is_find = set()
    for text in sorted(combined_search, key=lambda x: len(x.split(" ")), reverse=True):
        if combined_level > len(text.split(" ")) and all([w in is_find for w in text.split(" ")]):
            continue
        message = message_for_GPT(text, combined_search[text], medvik_find_by_code, context=table["original_text"][index])
        response = send_to_GPT(message)
        result[text] = from_GPT(response, combined_search[text], medvik_find_by_code)
        if result[text] is not None:
            is_find.update(text.split(" "))
            combined_level = len(text.split(" "))

    for text in search:
        if text in is_find:
            continue
        message = message_for_GPT(text, search[text], medvik_find_by_code, context=table["original_text"][index])
        response = send_to_GPT(message)
        result[text] = from_GPT(response, search[text], medvik_find_by_code)
        
    return interprete_explanation(result)

#### Procedura
Hyperparameters:
- Search only Anatomy and Analytical (A), diagnostic and therapeutic techniques and devices (E)
- drop length dependent before combining limit: 30
- drop not word limit: 80

In [193]:
whole_way_procedura = (lambda table, index, output_exp=False: whole_way(table, index, procedury_anathomy, 30, 80, output_exp=output_exp))

#### Os. Anamnéza
Hyperparameters:
- Search only: Anatomy and Analytical (A), Diseases (C), diagnostic and therapeutic techniques and devices (E)
- drop length dependent before combining limit: 70
- drop not word limit: 90

In [194]:
whole_way_os_anamneza = (lambda table, index: whole_way(table, index, osobni_a, 70, 90))

#### Symptomy
Hyperparameters:
- Search only: Anatomy (A), Diseases (C), Chemicals and Drugs (D), Analytical, Diagnostic and Therapeutic Techniques and Equipment (E), Phenomena and Processes (G)
- drop length dependent before combining limit: 30
- drop not word limit: 80

In [195]:
whole_way_symptomy = (lambda table, index, output_exp=False: whole_way(table, index, symptomy, 30, 80, output_exp=output_exp))

#### NE Os. Anamnéza
Hyperparameters:
- Search only: Anatomy and Analytical (A), Diseases (C), diagnostic and therapeutic techniques and devices (E)
- drop length dependent before combining limit: 70
- drop not word limit: 90

In [196]:
def whole_way_ne_os_anamneza(table, index):
    is_negation = False
    if any([True for x in table["text"][index].split(" ") if is_soft_negative_word(x)]):
        is_negation = True

    table.at[index, "text"] = " ".join([x.strip(":") for x in table["text"][index].split(" ")
                                                      if not(is_negative_word(x))])

    return (whole_way_os_anamneza(table, index), is_negation)

#### NE Symptomy

In [197]:
def whole_way_ne_symptomy(table, index):
    is_negation = False
    if any([True for x in table["text"][index].split(" ") if is_soft_negative_word(x)]):
        is_negation = True

    table.at[index, "text"] = " ".join([x.strip(":") for x in table["text"][index].split(" ")
                                                      if not(is_negative_word(x))])

    return (whole_way_symptomy(table, index), is_negation)

#### Medikace

Hyperparameters:

- Search: SUKL drugs, SUKL atc, SUKL synonyms, SUKL substances
- drop length dependent before combining limit: 25 (chemicals), 40 (drugs)
- drop not word limit: 10 (chemicals), 15 (drugs)


In [198]:
def whole_way_medikace(table, index):
    text = table["text"][index]
    shortcuts = table["shortcuts"][index]

    # Search
    search_chemicals = {}
    search_chemicals.update(sukl_find_chemicals_search(text,
                                                       output_errors=True, 
                                                       database=name_to_atc,
                                                       limit=0.15) if text != "" else {})
    search_chemicals.update(sukl_find_chemicals_shortcuts_search(shortcuts,
                                                                 output_errors=True,
                                                                 database=name_to_atc,
                                                                 limit=0.15) if shortcuts != "" else {})
    search_drugs = {}
    search_drugs.update(sukl_find_drugs_search(text,
                                               output_errors=True,
                                               limit=0.25) if text != "" else {})
    search_drugs.update(sukl_find_drugs_shortcuts_search(shortcuts,
                                                         output_errors=True,
                                                         limit=0.15) if shortcuts != "" else {})
    
    search_chemicals_errors = (lambda x: {k[0]:k[1] for k in x})(search_chemicals)
    search_chemicals = (lambda x: {k[0]:x[k] for k in x})(search_chemicals)
    search_drugs_errors = (lambda x: {k[0]:k[1] for k in x})(search_drugs)
    search_drugs = (lambda x: {k[0]:x[k] for k in x})(search_drugs)

    # Drop & Combine
    search_chemicals = drop_long_searches_len_dep(search_chemicals, 25)
    search_drugs = drop_long_searches_len_dep(search_drugs, 40)
    combined_chemicals = combine_preserve_count(search_chemicals)
    combined_drugs = combine_preserve_count(search_drugs)

    # Choosing best match
    search_chemicals = drop_long_searches(search_chemicals, 10)
    search_drugs = drop_long_searches(search_drugs, 15)
    search_chemicals = drop_empty_searches(search_chemicals)
    search_drugs = drop_empty_searches(search_drugs)

    comb_searches = combine_2_searched_databaze(search_chemicals | combined_chemicals,
                                                search_drugs | combined_drugs,
                                                search_chemicals_errors,
                                                search_drugs_errors)
    
    return {k: sorted(comb_searches[k], key=lambda y: prioritize_sukl(y, comb_searches[k]), reverse=True)[0] 
              for k in comb_searches 
              if len(comb_searches[k]) != 0}

#### NE Medikace

In [199]:
def whole_way_ne_medikace(table, index):
    is_negation = False
    if any([True for x in table["text"][index].split(" ") if is_soft_negative_word(x)]):
        is_negation = True

    table.at[index, "text"] = " ".join([x.strip(":") for x in table["text"][index].split(" ")
                                                      if not(is_soft_negative_word(x))])
    
    return (whole_way_medikace(table, index), is_negation)

#### Example of usage

In [200]:
i = improved_medikace.sample(frac=1, random_state=23).index[0]
print(improved_medikace["original_text"][i])
whole_way_medikace(make_shortcuts_table(improved_medikace), i)

depakine Crono


{'depakine Crono': ('N03AG01', 'KYSELINA VALPROOVÁ', 4)}

In [201]:
i = improved_ne_medikace.sample(frac=1, random_state=21).index[0]
print(improved_ne_medikace["original_text"][i])
whole_way_ne_medikace(make_shortcuts_table(improved_ne_medikace), i)

zometě


({'zometě': ('M05BA08', 'KYSELINA ZOLEDRONOVÁ', 1)}, False)

In [202]:
i = improved_symptom.sample(frac=1, random_state=20).index[0]
print(improved_symptom["original_text"][i])
whole_way_symptomy(make_shortcuts_table(improved_symptom), i)

DKK brnění prstů


KeyboardInterrupt: 

## Labelling

In [ ]:
atc_drug_bank = pd.read_csv("databaze/drugbank_to_atc.csv", index_col=[0])
atc_drug_bank.head(3)

In [ ]:
temp = atc_drug_bank[atc_drug_bank["atc"].duplicated(keep=False)]
temp.index = temp["atc"]
temp2 = substances_with_ATC.copy()
temp2.index = temp2["ATC"]
temp.join(temp2).drop(["ATC", "code", "NT", "NAZEV_EN"], axis=1).sort_index().head(20)

In [ ]:
def link_drug_bank(drug_code):
    if "DBSALT" in drug_code:
        return f"https://go.drugbank.com/salts/{drug_code}"
    return f"https://go.drugbank.com/drugs/{drug_code}"


def description_atc(atc_code):
    list_des = atc_drug_bank[atc_drug_bank["atc"] == atc_code].apply(
        lambda x: (x.code, x.description, link_drug_bank(x.code)), axis=1)
    
    if len(list_des) == 0:
        return ""
        
    result_string = ["Látky vedené v databázi DrugBank spojené s tímto ATC kódem:\n"]
    for code, des, link in list_des:
        result_string.append(f"{code}: {link} \n{des}\n")

    return "".join(result_string)

In [ ]:
link_medvik = lambda code: f"https://www.medvik.cz/bmc/link.do?id={code}"
remove_cit = lambda des: regex.sub(" ?\((C|c)it\.([^\)])*\)?.", "",  des)
link_drugbank = lambda code: f"https://go.drugbank.com/atc/{code}"


def prepare_labeling_medvik(db, expl_column="explanation"):
    result = db[db[expl_column] != {}].copy()
    result["Link to MSHCZ"] = result[expl_column].apply(
        lambda dictr: [(k, dictr[k][0], dictr[k][1], remove_cit(dictr[k][2]), link_medvik(dictr[k][0])) for k in dictr ]
    )
    result = result[["label", "original_text", "Link to MSHCZ"]]    
    return result


def prepare_labeling_sukl(db, expl_column="explanation"):
    result = db[db[expl_column] != {}].copy()
    result["Link to MSHCZ"] = result[expl_column].apply(
        lambda dictr: [(k, dictr[k][0], dictr[k][1], description_atc(dictr[k][0]), link_drugbank(dictr[k][0])) for k in dictr]
    )
    result = result[["label", "original_text", "Link to MSHCZ"]]    
    return result

print(prepare_labeling_medvik(symptom_test, expl_column="medvik_explanation").iloc[7]["Link to MSHCZ"])
print()
print(prepare_labeling_sukl(medikace_test, expl_column="sukl_mathematical_explanation").iloc[0]["Link to MSHCZ"])

In [ ]:
def predict_for_labeling_table(db, whole_way_func, n=35, random_state=None, negative=False):
    to_link = db.sample(frac=1, random_state=random_state)
    count_success, index = 0, 0
    to_link["explanation"] = "N/A"
    if negative:
        to_link["negation"] = False
    
    while count_success < n:
        if len(to_link) <= index:
            break
        j = to_link.index[index]
        
        result = whole_way_func(to_link, j)
        to_link.at[j, "explanation"] = result if not negative else result[0]
        if negative:
            to_link.at[j, "negation"] = result[1]
            
        if to_link["explanation"][j] != {}:
            count_success += 1

        index += 1
    
    return to_link.iloc[:index]

## Generate results for doctors

In [ ]:
# temp = predict_for_labeling_table(make_shortcuts_table(improved_medikace),
#                                                     whole_way_medikace,
#                                                     n=35,
#                                                     random_state=125)

# temp.to_csv("labeling_doctors/medikace_with_empty.csv")
# prepare_labeling_sukl(temp).to_csv("labeling_doctors/medikace_labeling.csv")

In [ ]:
# temp = predict_for_labeling_table(make_shortcuts_table(improved_ne_medikace),
#                                                     whole_way_ne_medikace,
#                                                     n=35,
#                                                     random_state=121,
#                                                     negative=True)

# temp.to_csv("labeling_doctors/ne_medikace_with_empty.csv")
# prepare_labeling_sukl(temp).to_csv("labeling_doctors/ne_medikace_labeling.csv")

In [ ]:
# temp = predict_for_labeling_table(make_shortcuts_table(improved_procedura),
#                                                     whole_way_procedura,
#                                                     n=35,
#                                                     random_state=124)

# temp.to_csv("labeling_doctors/procedura_with_empty.csv")
# prepare_labeling_medvik(temp).to_csv("labeling_doctors/procedura_labeling.csv")

In [ ]:
# temp = predict_for_labeling_table(make_shortcuts_table(improved_symptom),
#                                                     whole_way_symptomy,
#                                                     n=35,
#                                                     random_state=123)

# temp.to_csv("labeling_doctors/symptom_with_empty.csv")
# prepare_labeling_medvik(temp).to_csv("labeling_doctors/symptom_labeling.csv")

In [ ]:
# temp = predict_for_labeling_table(make_shortcuts_table(improved_ne_symptom),
#                                                     whole_way_ne_symptomy,
#                                                     n=35,
#                                                     random_state=119,
#                                                     negative=True)

# temp.to_csv("labeling_doctors/ne_symptom_with_empty.csv")
# prepare_labeling_medvik(temp).to_csv("labeling_doctors/ne_symptom_labeling.csv")

In [ ]:
# temp = predict_for_labeling_table(make_shortcuts_table(improved_osobni),
#                                                     whole_way_os_anamneza,
#                                                     n=35,
#                                                     random_state=120)

# temp.to_csv("labeling_doctors/osobni_with_empty.csv")
# prepare_labeling_medvik(temp).to_csv("labeling_doctors/osobni_labeling.csv")

In [ ]:
# temp = predict_for_labeling_table(make_shortcuts_table(improved_ne_osobni),
#                                                     whole_way_ne_os_anamneza,
#                                                     n=35,
#                                                     random_state=122,
#                                                     negative=True)

# temp.to_csv("labeling_doctors/ne_osobni_with_empty.csv")
# prepare_labeling_medvik(temp).to_csv("labeling_doctors/ne_osobni_labeling.csv")